In [1]:
# %matplotlib widget
# %matplotlib inline
%matplotlib qt5

In [2]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.style as style
from mpl_toolkits.axes_grid1 import make_axes_locatable

from qick import *
from qick.helpers import gauss
from tqdm import tqdm_notebook as tqdm

import time
import os
import sys
sys.path.append('/home/xilinx/jupyter_notebooks/')
import scipy as sp
from scipy.fft import fft, fftfreq
import json
from copy import deepcopy

from slab.instruments import *
from slab.experiment import Experiment
from slab.datamanagement import SlabFile
from slab import get_next_filename, AttrDict

# Figure params
plt.rcParams['figure.figsize'] = [10,6]
plt.rcParams.update({'font.size': 14})
default_colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
# style.use('default')
style.use('S:\Connie\prx.mplstyle')

import Pyro4.util

Could not import QickInstrument (will only work if running on rfsoc). If running on rfsoc, try install/update qick package
Warning failed to import AWG81180A
Warning failed to import Tek5014
Warning failed to import Tek70001
Warning failed to import M8195A


c:\ProgramData\Anaconda3\lib\site-packages\IPython\qt.py:12: ShimWarning: The `IPython.qt` package has been deprecated since IPython 4.0. You should import from qtconsole instead.
  warn("The `IPython.qt` package has been deprecated since IPython 4.0. "


Could not load InstrumentManagerWindow
Could not load labbrick
Could not load Autonics TM4 Driver
Could not load Oxford Trition driver


# Load experiments

In [3]:
# %aimport experiments
import experiments as meas

imported experiments.clifford_averager_program
imported experiments.fitting
imported experiments.four_qubit.fourQ_state_tomo
imported experiments.qram_protocol_timestepped
imported experiments.single_qubit.acstarkshift_spectroscopy
imported experiments.single_qubit.amplitude_rabi
imported experiments.single_qubit.length_rabi
imported experiments.single_qubit.pulse_probe_spectroscopy
imported experiments.single_qubit.resonator_spectroscopy
imported experiments.single_qubit.rfsoc_tof_calibration
imported experiments.single_qubit.single_shot
imported experiments.single_qubit.t1
imported experiments.single_qubit.t2_ramsey
imported experiments.three_qubit.threeQ_state_tomo
imported experiments.two_qubit.amplitude_rabi_EgGf
imported experiments.two_qubit.amplitude_rabi_f0g1
imported experiments.two_qubit.amprabi_opt_ctrl_state_prep
imported experiments.two_qubit.crosstalk_echo_calib
imported experiments.two_qubit.length_rabi_EgGf
imported experiments.two_qubit.length_rabi_F0G1
imported exper

In [4]:
"""Reopen saved data"""
def prev_data(expt_path, filename=None):
    temp_data_file = expt_path
    if filename is not None: temp_data_file = os.path.join(expt_path, filename)
    print(temp_data_file)
    with SlabFile(temp_data_file) as a:
        attrs = dict()
        for key in list(a.attrs):
            attrs.update({key:json.loads(a.attrs[key])})
        keys = list(a)
        temp_data = dict()
        for key in keys:
            temp_data.update({key:np.array(a[key])})
        print(f'Extracted data and attrs {list(a.attrs)}')
    return temp_data, attrs

In [5]:
data_path = 'S:\\QRAM\\qram_4QR2\\'
expt_path = os.path.join(data_path, 'data', 'data_241025')
print('Data will be stored in', expt_path)

Data will be stored in S:\QRAM\qram_4QR2\data\data_241025


In [6]:
# config_file = 'config_zcu216.yml'
# config_file = 'config_q3diamond.yml'
config_file = 'config_q3diamond_full688and638_reset.yml'
config_path = os.path.join('s:\\Connie\\experiments\\qram_tprocv1_expts\\configs\\', config_file)
print('Config will be', config_path)

Config will be s:\Connie\experiments\qram_tprocv1_expts\configs\config_q3diamond_full688and638_reset.yml


# Set the qubits/resonators to probe

## [Auto0] SET 2Q QUBIT SWAP PARAMETERS

In [7]:
qubits = [2, 1] # order doesn't matter if you specify qDrive correctly
init_state = '|0>|1>'
wrong_init_state = '|1>|1>'
setup_ZZ = None
# measure_qubits = [qubits[0], qubits[1], 3]
measure_qubits = qubits
qDrive = qubits[0] # normal new protocol

# qubits = [3, 1] # order doesn't matter if you specify qDrive correctly
# init_state = '|1>|1>' # for everything else
# wrong_init_state = '|0>|1>'
# setup_ZZ = 0 # for the amp rabi chevron
# # measure_qubits = [qubits[0], qubits[1], 2]
# measure_qubits = qubits
# qDrive = qubits[0] # normal new protocol

# init_state = '|0>|2>'
# qDrive = qubits[1]


# f0-g1
# # init_state = '|0>|0>'
# # qDrive = 1 # resonator f0-g1

# Setup experiment hardware

Set in config:
- LO alias, frequency, power
- SOC dac channels, nyquist zones

In [8]:
import yaml
with open(config_path, 'r') as cfg_file:
    yaml_cfg = yaml.safe_load(cfg_file)
yaml_cfg = AttrDict(yaml_cfg)

In [9]:
# im = InstrumentManager(ns_address='192.168.14.1') # Chicago lab
# im = InstrumentManager(ns_address='192.168.137.1') # SLAC lab
# im = InstrumentManager(ns_address='10.108.30.56') # Spilker Fermium
im = InstrumentManager(ns_address='10.108.30.75') # Spilker Vibranium

In [10]:
soc = QickConfig(im[yaml_cfg['aliases']['soc']].get_cfg())
print(soc)
# print(soc.description())
# print(soc.dump_cfg())

QICK running on ZCU216, software version 0.2.272

Firmware configuration (built Sat Oct  5 10:21:47 2024):

	Global clocks (MHz): tProcessor 430.080, RF reference 245.760

	7 signal generator channels:
	0:	axis_signal_gen_v6 - envelope memory 32768 samples (5.128 us)
		fs=6389.760 MHz, fabric=399.360 MHz, 32-bit DDS, range=6389.760 MHz
		DAC tile 2, blk 0 is 0_230, on JHC3
	1:	axis_signal_gen_v6 - envelope memory 32768 samples (5.128 us)
		fs=6389.760 MHz, fabric=399.360 MHz, 32-bit DDS, range=6389.760 MHz
		DAC tile 2, blk 1 is 1_230, on JHC4
	2:	axis_signal_gen_v6 - envelope memory 32768 samples (5.128 us)
		fs=6389.760 MHz, fabric=399.360 MHz, 32-bit DDS, range=6389.760 MHz
		DAC tile 2, blk 2 is 2_230, on JHC3
	3:	axis_signal_gen_v6 - envelope memory 32768 samples (5.128 us)
		fs=6389.760 MHz, fabric=399.360 MHz, 32-bit DDS, range=6389.760 MHz
		DAC tile 2, blk 3 is 3_230, on JHC4
	4:	axis_signal_gen_v6 - envelope memory 32768 samples (4.762 us)
		fs=6881.280 MHz, fabric=430.080 MH

In [ ]:
lo1 = im[yaml_cfg.aliases.readout_LO]
lo1.open_device()
print(lo1.get_id())

# TURN OFF
lo1.set_standby(True)
lo1.set_output_state(False)

# # TURN ON
# lo1.set_standby(False)
# lo1.set_output_state(True)
# lo_freq = float(yaml_cfg.hw.lo.readout.frequency)
# lo_power = float(yaml_cfg.hw.lo.readout.power)
# lo1.set_frequency(lo_freq) # Set LO frequency
# print('set freq to', lo_freq)
# lo1.set_power(lo_power) # Set LO power
# print('set power to', lo_power)

# ZZ shifted pi pulses

## Pulse probe coupling spectroscopy

Sweep qubit A frequency after applying pi pulse to qubit B.

In [101]:
qubits = [1, 2] # for eg-gf swap, always list the 2nd qubit here as the 2nd qubit in the order eg-gf

In [ ]:
qZZspec = meas.PulseProbeCouplingSpectroscopyExperiment(
    soccfg=soc,
    path=expt_path,
    prefix=f"qubit_coupling_spectroscopy_qubit{qubits[0]}{qubits[1]}",
    config_file=config_path,
)

span = 1
npts = 100

qZZspec.cfg.expt = dict(
    start=qZZspec.cfg.device.qubit.f_Q1_ZZ[qubits[1]]-span/2,
    # start=qZZspec.cfg.device.qubit.f_ge[qubits[0]]-span/2,
    # start=3458.5-span/2,
    step=span/npts,
    expts=npts, # Number of experiments stepping from start
    reps=100, # Number of averages per point
    rounds=10, # Number of start to finish sweeps to average over
    pulseB=True, # Whether to apply pi pulse to qubit B
    length=1, # qubit 0 probe constant pulse length [us]
    gain=100, # qubit 0 const pulse gain
    pulse_type='gauss',
    qubits=qubits,
)

# qZZspec.cfg.device.readout.relax_delay = [10, 10, 10, 10] # Wait time between experiments [us]

print(qZZspec.cfg)

qZZspec.go(analyze=False, display=False, progress=True, save=False)

In [ ]:
# # TESTING
# qZZspec = meas.PulseProbeEgGfSpectroscopyExperiment(config_file=config_path)
# temp_data, attrs = prev_data(expt_path, '00002_qubit_coupling_spectroscopy_qubit01.h5')
# data = qZZspec.analyze(data=temp_data, fit=True, sign=[[1, -1], [-1, -1]])
# qZZspec.display(data=data, fit=True)

qZZspec.analyze(fit=True, signs=[1, -1, 1])
qZZspec.display(fit=True, signs=[1, -1, 1])

In [ ]:
qZZspec.save_data()

## Length rabi for Q1 with QA in e

In [ ]:
lengthrabiZZ = meas.LengthRabiPiZZExperiment(
    soccfg=soc,
    path=expt_path,
    prefix=f"length_rabi_ZZ_qubit{qubits[0]}{qubits[1]}",
    config_file=config_path,
)

lengthrabiZZ.cfg.expt = dict(
    start=0, # pulse length (sigma) [us]
    step=soc.cycles2us(1), # [us]
    # step=0.001,
    expts=101,
    reps=1000,
    gain=20000, # qubit gain [DAC units],
    pulse_type='gauss',
    qubits=qubits,
)
# lengthrabiZZ.cfg.device.readout.relax_delay = 100 # Wait time between experiments [us]

print(lengthrabiZZ.cfg)
# lengthrabi.im[lengthrabi.cfg.aliases.soc].reset()
lengthrabiZZ.go(analyze=False, display=False, progress=True, save=False)

In [ ]:
# # TESTING
# temp_data, attrs = prev_data(expt_path, '00000_voltage_sweep_qubit_spectroscopy_qubit3.h5')
# data = lengthrabiZZ.analyze(data=temp_data, fit=True)

lengthrabiZZ.analyze(fit=True)
lengthrabiZZ.display(fit=True)

In [ ]:
lengthrabiZZ.save_data()

# 2Q swap frequencies

Estimate 2Q swap frequencies analytically

In [ ]:
f_ge = np.array(yaml_cfg.device.qubit.f_ge) + np.array(yaml_cfg.hw.soc.dacs.qubit.mixer_freq)
f_Q1_ZZ = np.array(yaml_cfg.device.qubit.f_Q1_ZZ) + np.array(yaml_cfg.hw.soc.dacs.qubit.mixer_freq)
f_ef = np.array(yaml_cfg.device.qubit.f_ef) + np.array(yaml_cfg.hw.soc.dacs.qubit.mixer_freq)

print('f_EgGf: need indices 0, 2, 3')
print(f_ge[1] + f_ef[1] - f_ge)
print('f_EgGf_Q: indices 2, 3')
print(f_ge[2] + f_ef[2] - f_ge[1])
print(f_ge[3] + f_ef[3] - f_Q1_ZZ[0])
print('f_EgGf for Q3 if switch is in g')
print(f_ge[3] + f_ef[3] - f_ge[1])
print('f_EgGf for Q2 if switch is in 3')
print(f_ge[2] + f_ef[2] - f_Q1_ZZ[0])

## Pulse probe eg-gf spectroscopy

In [ ]:
qEgGfspec = meas.PulseProbeEgGfSpectroscopyExperiment(
    soccfg=soc,
    path=expt_path,
    prefix=f"qubit_EgGf_spectroscopy_qubit{qubits[0]}{qubits[1]}",
    config_file=config_path,
)

span = 40
npts = 150

qEgGfspec.cfg.expt = dict(
    # start=qEgGfspec.cfg.device.qubit.f_ge[1] - span/2, # Eg-Gf probe frequency [MHz]
    start=qEgGfspec.cfg.device.qubit.f_EgGf[qubits[0]] - span/2, # Eg-Gf probe frequency [MHz]
    # start=1900,
    step=span/npts,
    expts=npts, # Number of experiments stepping from start
    reps=100, # Number of averages per point
    rounds=10, # Number of start to finish sweeps to average over
    length=2, # Eg-Gf probe constant pulse length [us]
    gain=3000, # Eg-Gf const pulse gain
    qubits=qubits,
)
# qEgGfspec.cfg.device.readout.relax_delay = [50]*4
# qEgGfspec.cfg.device.readout.relax_delay = [100, 100, 100, 100]
# qEgGfspec.cfg.device.readout.length = [5, 5, 5, 5]

print(qEgGfspec.cfg)

qEgGfspec.go(analyze=False, display=False, progress=True, save=False)

In [ ]:
# # TESTING
# qEgGfspec = meas.PulseProbeEgGfSpectroscopyExperiment(config_file=config_path)
# temp_data, attrs = prev_data(expt_path, '00002_qubit_EgGf_spectroscopy_qubit01.h5')
# data = qEgGfspec.analyze(data=temp_data, fit=True, sign=[[1, -1], [-1, -1]])
# qEgGfspec.display(data=data, fit=True)

# qEgGfspec.analyze(fit=False)
sign = [[1, -1], [-1, 1]] # [iA, qA], [iB, qB]
qEgGfspec.analyze(fit=True, sign=sign)
qEgGfspec.display(fit=True, sign=sign)

In [ ]:
qEgGfspec.save_data()

### Pulse probe EgGF power sweep

In [ ]:
sweepspecEgGf = meas.PulseProbeEgGfSweepSpectroscopyExperiment(
    soccfg=soc,
    path=expt_path,
    prefix=f"pulse_probe_ef_len_sweep_spectroscopy_qubit{qubits[0]}{qubits[1]}",
    config_file=config_path,
)

span_f = 75
npts_f = 150
# npts_f = 10

span_len = 20
npts_len = 10
# npts_len = 3

sweepspecEgGf.cfg.expt = dict(
    start_f=sweepspecEgGf.cfg.device.qubit.f_EgGf[qubits[0]] - span_f/2, # Eg-Gf probe frequency [MHz]
    step_f=span_f/npts_f, # min step ~1 Hz
    expts_f=npts_f, # Number experiments stepping freq from start
    start_len=1,
    step_len=int(span_len/npts_len), # len step size
    expts_len=npts_len, # Number experiments stepping len from start
    reps=500,
    rounds=40, # Number averages per point
    gain=30000, # ef const pulse length [us]
    qubits=qubits,
)
sweepspecEgGf.cfg.device.readout.relax_delay = [100, 100, 100, 100]
sweepspecEgGf.cfg.device.readout.length = [5, 5, 5, 5]
sweepspecEgGf.go(analyze=False, display=False, progress=True, save=False)

In [ ]:
# # TESTING
# temp_data, attrs = prev_data(expt_path, '00000_voltage_sweep_qubit_spectroscopy_qubit3.h5')
# data = rpowspec.analyze(data=temp_data, lowgain=2000, highgain=18000)

sweepspecEgGf.analyze()
sweepspecEgGf.display()

In [ ]:
3452+3318-4763

## [Auto1] Rabi eg-gf chevron (freq vs. amplitude)

In [139]:
rabiEgGfFreqGainChevron = meas.EgGfFreqGainChevronExperiment(
    soccfg=soc,
    path=expt_path,
    prefix=f"rabi_EgGf_freqgain_chevron_qubit{qubits[0]}{qubits[1]}",
    config_file=config_path,
)

span_gain = 7000
# span_gain = 7000
# npts_gain = 10
# span_gain = 0
# npts_gain = 25
npts_gain = 15

# span_f = 26 # MHz
# npts_f = 50
span_f = 20
npts_f = 40

rabiEgGfFreqGainChevron.cfg.expt = dict(
    start_gain=12000, # qubit gain [dac level]
    # start_gain=18000, # qubit gain [dac level]
    step_gain=int(span_gain/npts_gain), # [dac level]
    expts_gain=npts_gain,
    # start_f=2012 - span_f/2, # start freq [MHz],
    # start_f=rabiEgGfFreqGainChevron.cfg.device.qubit.f_EgGf[qubits[0]] - span_f/2, # start freq [MHz],
    start_f=rabiEgGfFreqGainChevron.cfg.device.qubit.f_EgGf_Q_half[qubits[0]] - span_f/2, # start freq [MHz],
    # start_f=rabiEgGfFreqGainChevron.cfg.device.qubit.f_EgGf_Q[qubits[0]] - 5 - span_f/2, # start freq [MHz],
    # start_f=5120,
    step_f=span_f/npts_f, # freq step, 
    expts_f=npts_f, # number of different freq experiments, 
    reps=500,
    rounds=1,
    pi_EgGf_sigma=0.230, # gaussian sigma/const pulse length, overrides config [us]
    pulse_type='flat_top',
    # pulse_type='const',
    setup_ZZ=setup_ZZ,
    # setup_ZZ=None,
    qDrive=qDrive,
    # qDrive=1,
    qubits=qubits,
    measure_qubits=measure_qubits,

    test_pi_half=True,


    # full_mux_expt=True,
    # resonator_reset=[0, 2, 3],
)

# rabiEgGfFreqGainChevron.cfg.device.readout.readout_length = [15, 15] # [us]
# rabiEgGfFreqGainChevron.cfg.device.readout.relax_delay = [600, 50] # Wait time between experiments [us]

print('qDrive', qDrive)
print(rabiEgGfFreqGainChevron.cfg)

rabiEgGfFreqGainChevron.go(analyze=False, display=False, progress=True, save=False)

qDrive 2
{'device': {'readout': {'trig_offset': 150, 'lo_sideband': [1, 1, 1, 1], 'frequency': [-389.975, 5795.761666663333, 502.368233333334, -228.9], 'gain': [0.5, 0.6, 0.4, 0.3], 'readout_length': [5, 20, 10, 8], 'frequency_ef': [-391.91408405, 5795.76166666, 500.44439227, -230.30732892], 'gain_ef': [0.05, 0.6, 0.4, 0.25], 'readout_length_ef': [1, 20, 10, 8], 'relax_delay': [450, 450, 450, 450]}, 'qubit': {'f_ge': [4111.103663892308, 4105.758870288914, 4110.961390890517, 4110.561006068113, 3442.9791477220547, 3448.3312965131795, 3447.2256002913127, 3447.022898974775, 4762.736987583428, 4761.772206046936, 4762.882027802732, 4762.340227746316, 4380.990256112639, 4380.2241341449835, 4380.999038792971, 4381.537145425829], 'f_ef': [3884.9668600899413, 0.0, 0.0, 0.0, 0.0, 3347.846225737386, 0.0, 0.0, 4571.9056482340775, 4571.009319367402, 4572.904716852475, 0.0, 4211.704645679207, 4206.869581627414, 0.0, 4205.123751627149], 'f_EgGf': [2686.53594683, 0, 2017.3, 2414.92850999], 'f_EgGf_Q_ha

  0%|          | 0/40 [00:00<?, ?it/s]

In [149]:
# # TESTING
# rabiEgGfFreqGainChevron = meas.EgGfFreqGainChevronExperiment(config_file=config_path)
# temp_data, attrs = prev_data('s:\QRAM\qram_4QR2\data\data_240508\\00001_rabi_EgGf_freqgain_chevron_qubit31.h5')
# rabiEgGfFreqGainChevron.cfg.expt = attrs['config']['expt']
# # print(temp_data.keys())
# # rabiEgGfFreqGainChevron.display(data=temp_data, saveplot=True) #, plot_freq=2024.5, plot_gain=2125)
# data = rabiEgGfFreqGainChevron.analyze(data=temp_data, fit=True)
# rabiEgGfFreqGainChevron.display(data=data, fit=True)

data = rabiEgGfFreqGainChevron.analyze(data=None, fit=True)
rabiEgGfFreqGainChevron.display(fit=True, data=data, range_start=0, range_end=-5) #, plot_freq=5883.9, plot_gain=21000)


# plt.figure()
# plot_gain = 15000
# # plot_gain = rabiEgGfFreqGainChevron.cfg.device.qubit.pulses.pi_EgGf_Q.gain[qubits[0]]
# gain_ind = np.argmin(np.abs(data['gainpts']-plot_gain))
# # for i in range(len(data['lenpts'])):
# #     plt.plot(data['freqpts'], data['avgi'][0][:, i])

# amps_freq_cut_qA = data['amps'][0][:, gain_ind]
# amps_freq_cut_qB = data['amps'][1][:, gain_ind]

# p_ampsA = data['fitamps'][0][gain_ind]
# # print(p_ampsA)
# fit_amps_freq_cut_qA = meas.fitting.rabifunc(data['freqpts'], *p_ampsA)
# plt.suptitle(f"Cut through gain {data['gainpts'][gain_ind]}")
# plt.subplot(211, title=f'QA ({qubits[0]})')
# plt.plot(data['freqpts'], amps_freq_cut_qA, '.-', label='qA', color=default_colors[0])
# plt.plot(data['freqpts'], fit_amps_freq_cut_qA, color=default_colors[0])
# # plt.axvline(freq_max_amp_qA, color=default_colors[0], linestyle='--')
# plt.ylabel('Amplitude [ADC units]')

# plt.subplot(212, title=f'QB ({qubits[1]})')
# plt.plot(data['freqpts'], amps_freq_cut_qB, '.-', label='qB', color=default_colors[1])
# # plt.plot(data['freqpts'], meas.fitting.quadraticfunc(data['freqpts'], *p_ampsB), color=default_colors[1])
# # plt.axvline(freq_min_amp_qB, color=default_colors[1], linestyle='--')
# plt.xlabel('Frequency [MHz]')
# plt.ylabel('Amplitude [ADC units]')
# plt.tight_layout()
# plt.show()


QA: max at gain 12000 freq 5885.6717699723895
QB: min at gain 12466 freq 5885.6717699723895
fit_gain_sweep = [12000, 12466, 12932, 13398, 13864, 14330, 14796, 15262, 15728, 16194, 16660, 17126, 17592, 18058, 18524]
fit_freqs= [5885.794489663239, 5885.612675998018, 5885.434649062326, 5885.260408856163, 5885.08995537953, 5884.923288632426, 5884.760408614851, 5884.601315326807, 5884.446008768291, 5884.294488939304, 5884.146755839847, 5884.002809469919, 5883.862649829522, 5883.726276918653, 5883.5936907373125]


In [150]:
rabiEgGfFreqGainChevron.save_data()

Saving S:\QRAM\qram_4QR2\data\data_241025\00003_rabi_EgGf_freqgain_chevron_qubit21.h5


'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00003_rabi_EgGf_freqgain_chevron_qubit21.h5'

## Length rabi chevron eg-gf (freq vs. len)

1. Once picked a good gain/len/freq, make sure the freq is really dialed in by doing a cut through the length.
2. Calibrate simultaneous pulses by doing the same thing while playing the other pulse

In [ ]:
rabiEgGfFreqLenChevron = meas.EgGfFreqLenChevronExperiment(
    soccfg=soc,
    path=expt_path,
    prefix=f"rabi_EgGf_freqlen_chevron_qubit{qubits[0]}{qubits[1]}",
    config_file=config_path,
)

span_f = 5 # MHz
# span_f = 20 # MHz
# npts_f = 100
npts_f = 20

start_len = 0.0
# start_len = rabiEgGfFreqLenChevron.cfg.device.qubit.pulses.pi_EgGf_Q.half_sigma[qubits[0]]
# span_len = 0.300
span_len = 2*rabiEgGfFreqLenChevron.cfg.device.qubit.pulses.pi_EgGf_Q.sigma[qubits[0]]
npts_len = 15
# start_len = rabiEgGfFreqLenChevron.cfg.device.qubit.pulses.pi_EgGf_Q.sigma[qubits[0]]
# span_len = 0
# npts_len = 1

print(measure_qubits)
print(qubits)
  
rabiEgGfFreqLenChevron.cfg.expt = dict(
    # start_f=rabiEgGfFreqLenChevron.cfg.device.qubit.f_EgGf[qubits[0]] - span_f/2 - 20, # start freq [MHz],
    start_f=rabiEgGfFreqLenChevron.cfg.device.qubit.f_EgGf_Q_half[qubits[0]] - 0.5*span_f, # start freq [MHz],
    # start_f=5136.0-0.5*span_f,
    step_f=span_f/npts_f, # freq step, 
    # expts_f=1,
    expts_f=npts_f+1, # number of different freq experiments, 
    start_len=start_len, # pulse length (sigma) [us]
    step_len=span_len/npts_len, # [us]
    expts_len=npts_len,
    # gain=15000,
    # reps=800,
    reps=300,
    loops=1,
    pulse_type='flat_top',
    post_process=None, #'threshold',
    # singleshot_reps=10000,
    # sigma_ramp_cycles=2,
    measure_f=None,
    init_state=init_state,
    qDrive=qDrive,
    qubits=qubits,
    measure_qubits=measure_qubits,

    test_pi_half=True,
    readout_cool=False,

    full_mux_expt=full_mux_expt,
    resonator_reset=resonator_reset,
    use_robust_pulses=True,
)

# rabiEgGfFreqLenChevron.cfg.device.readout.readout_length = [15, 15] # [us]
# rabiEgGfFreqLenChevron.cfg.device.readout.relax_delay = [600, 50] # Wait time between experiments [us]

print(rabiEgGfFreqLenChevron.cfg)

try:
    rabiEgGfFreqLenChevron.go(analyze=False, display=False, progress=True, save=False)
except Exception:
    print("Pyro traceback:")
    print("".join(Pyro4.util.getPyroTraceback()))

In [97]:
data = rabiEgGfFreqLenChevron.analyze(data=None, fit=False)
rabiEgGfFreqLenChevron.display(data=data, fit=False, signs=[-1, -1, -1, 1])

# rabiEgGfFreqLenChevron.data = data

# # print('Saved', rabiEgGfFreqLenChevron.save_data())

# plt.figure(figsize=(9, 8))
# if qubits[0] == 3: sign = 1
# elif qubits[0] == 2: sign = -1
# good_ZZ_freq = rabiEgGfFreqLenChevron.cfg.device.qubit.f_EgGf_Q[qubits[0]]
# # bad_ZZ_freq = rabiEgGfFreqLenChevron.cfg.device.qubit.f_EgGf_Q[qubits[0]]+sign*rabiEgGfFreqLenChevron.cfg.device.qubit.ZZs[1]

# plot_len = rabiEgGfFreqLenChevron.cfg.device.qubit.pulses.pi_EgGf_Q.sigma[qubits[0]]
# # plot_len = rabiEgGfFreqLenChevron.cfg.device.qubit.pulses.pi_EgGf.sigma[qubits[0]]
# len_ind = np.argmin(np.abs(data['lenpts']-plot_len))
# # for i in range(len(data['lenpts'])):
# #     plt.plot(data['freqpts'], data['avgi'][0][:, i])

# amps_freq_cut_qA = data['amps'][0][:, len_ind]
# amps_freq_cut_qB = data['amps'][1][:, len_ind]
# p_ampsA, pCov_ampsA = meas.fitting.fitquadratic(data['freqpts'], amps_freq_cut_qA)
# p_ampsB, pCov_ampsB = meas.fitting.fitquadratic(data['freqpts'], amps_freq_cut_qB)
# fit_amps_freq_cut_qA = meas.fitting.quadraticfunc(data['freqpts'], *p_ampsA)
# fit_amps_freq_cut_qB = meas.fitting.quadraticfunc(data['freqpts'], *p_ampsB)
# freq_max_amp_qA = data['freqpts'][np.argmax(fit_amps_freq_cut_qA)]
# freq_min_amp_qB = data['freqpts'][np.argmin(fit_amps_freq_cut_qB)]
# print('QA freq with max amp', freq_max_amp_qA)
# print('QB freq with min amp', freq_min_amp_qB)
# plt.suptitle(f"Cut through length {data['lenpts'][len_ind]:.3} us")
# plt.subplot(211, title=f'QA ({qubits[0]})')
# plt.plot(data['freqpts'], amps_freq_cut_qA, '.-', label='qA', color=default_colors[0])
# plt.plot(data['freqpts'], fit_amps_freq_cut_qA, color=default_colors[0])
# plt.axvline(freq_max_amp_qA, color=default_colors[0], linestyle='--')
# plt.ylabel('Amplitude [ADC units]')

# plt.subplot(212, title=f'QB ({qubits[1]})')
# plt.plot(data['freqpts'], amps_freq_cut_qB, '.-', label='qB', color=default_colors[1])
# plt.plot(data['freqpts'], meas.fitting.quadraticfunc(data['freqpts'], *p_ampsB), color=default_colors[1])
# plt.axvline(freq_min_amp_qB, color=default_colors[1], linestyle='--')
# # plt.axvline(5890.45, color='r', linestyle='--')
# # plt.axvline(bad_ZZ_freq, color='r', linestyle='--')
# plt.xlabel('Frequency [MHz]')
# plt.ylabel('Amplitude [ADC units]')
# plt.tight_layout()
# plt.show()

# fig, ax = plt.subplots(1, 1, figsize=(6, 6))

# ax.plot(data['freqpts'], amps_freq_cut_qA+amps_freq_cut_qB, '.-', label='qA+qB', color=default_colors[0])


In [ ]:
# filename = 's:\Connie\experiments\qramLL_4QR2\data\data_240218\\00123_rabi_EgGf_freqlen_chevron_qubit21.h5'
# filename = 's:\Connie\experiments\qramLL_4QR2\data\data_240218\\00025_rabi_EgGf_freqlen_chevron_qubit31.h5'

rabiEgGfFreqLenChevron = meas.EgGfFreqLenChevronExperiment(
    soccfg=None,
    path=expt_path,
    prefix=f"rabi_EgGf_freqlen_chevron_qubit{qubits[0]}{qubits[1]}",
    config_file=config_path,
)
temp_data, attrs = prev_data(filename, None)
data = temp_data
data = deepcopy(data)

# # print(temp_data['amps'].shape)
# # print(data['amps'].shape)
# for fit_axis in ['amps', 'avgi', 'avgq']:
#     data[fit_axis] = np.concatenate((temp_data[fit_axis], data[fit_axis]), axis=1)
# data['freqpts'] = np.concatenate((temp_data['freqpts'], data['freqpts']), axis=None)

# print(data['freqpts'])
# rabiEgGfFreqLenChevron.display(data=data, fit=False, signs=[-1, 1, -1, 1])

rabiEgGfFreqLenChevron.data = data
rabiEgGfFreqLenChevron.cfg = AttrDict(attrs['config'])
qubits = rabiEgGfFreqLenChevron.cfg.expt.qubits
if qubits[0] == 3: sign = 1
elif qubits[0] == 2: sign = -1
good_ZZ_freq = rabiEgGfFreqLenChevron.cfg.device.qubit.f_EgGf_Q[qubits[0]]
bad_ZZ_freq = rabiEgGfFreqLenChevron.cfg.device.qubit.f_EgGf_Q[qubits[0]]+sign*rabiEgGfFreqLenChevron.cfg.device.qubit.ZZs[1]

plt.style.use('dark_background')

rows = 2
cols = 1
index = rows*100 + cols*10
fig, axs = plt.subplots(rows, cols, figsize=(9*cols, 7), gridspec_kw={'height_ratios': [1, 1]}, sharex='col')
ax1, ax2 = axs
plt.suptitle("$|eg\\rangle-|gf\\rangle$"+ f" Frequency vs. Length (Q{qubits[0]}/Q{qubits[1]})")

plt.sca(ax1)
data['amps'][0] = (data['amps'][0] - min(data['amps'][0])) / (max(data['amps'][0]) - min(data['amps'][0]))
rabiEgGfFreqLenChevron.plot_rabi_chevron(data, 'amps', plot_xpts=1000*data['lenpts'], plot_ypts=data['freqpts'], plot_rabi=False, q_index=0, show_cbar=False)
# plt.axhline(bad_ZZ_freq, color='r', linestyle='--')
# print(((data['freqpts'][-1] - data['freqpts'][0])//4))
# yticks = int(np.average(data['freqpts'])) + (data['freqpts'][-1] - data['freqpts'][0])//3*np.arange(-1, 2) 
yticks = [round(good_ZZ_freq), round(bad_ZZ_freq)] 
plt.yticks(ticks=yticks, fontsize=24)
plt.ylabel('Frequency\n(MHz)', fontsize=24)
divider = make_axes_locatable(ax1)
cax = divider.append_axes("right", size="5%", pad=0.05)
cbar = plt.colorbar(cax=cax, ticks=[0, 0.5, 1])
cbar.ax.tick_params(labelsize=16)
cbar.set_label(f'Population (Scaled)', size='large')
# cbar.ax.set_title(f'Q{qubits[0]}')

plt.sca(ax2)
# data['amps'][1] = (data['amps'][1] - min(data['amps'][1])) / (max(data['amps'][1]) - min(data['amps'][1]))
data['amps'][1] = (data['amps'][1] - max(data['amps'][1])) / (min(data['amps'][1]) - max(data['amps'][1]))
rabiEgGfFreqLenChevron.plot_rabi_chevron(data, 'amps', plot_xpts=1000*data['lenpts'], plot_ypts=data['freqpts'], plot_rabi=False, q_index=1, show_cbar=False)
# plt.axhline(bad_ZZ_freq, color='r', linestyle='--')
# yticks = int(np.average(data['freqpts'])) + (data['freqpts'][-1] - data['freqpts'][0])//3*np.arange(-1, 2) 
yticks = [round(good_ZZ_freq), round(bad_ZZ_freq)] 
plt.ylabel('Frequency\n(MHz)', fontsize=24)
plt.xlabel('Length (ns)', fontsize=24)
plt.yticks(ticks=yticks, fontsize=24)
plt.xticks(fontsize=24)
divider = make_axes_locatable(ax2)
cax = divider.append_axes("right", size="5%", pad=0.05)
cbar = plt.colorbar(cax=cax, ticks=[0, 0.5, 1])
cbar.ax.tick_params(labelsize=16)
cbar.set_label(f'Population (Scaled)', size='large')
# cbar.ax.set_title(f'Q{qubits[1]}')

plt.subplots_adjust(hspace=0.07)
# plt.tight_layout()

plot_filename = filename[:-3]+'.svg'
plt.savefig(plot_filename, format='svg', bbox_inches='tight', transparent=True)
print('Saved', plot_filename)

plt.show()




# # filename = 's:\Connie\experiments\qramLL_4QR2\data\data_240218\\00122_rabi_EgGf_freqlen_chevron_qubit21.h5'
# # temp_data, attrs = prev_data(filename, None)
# # data = temp_data
# # data = deepcopy(data)

# # # print(temp_data['amps'].shape)
# # # print(data['amps'].shape)
# # for fit_axis in ['amps', 'avgi', 'avgq']:
# #     data[fit_axis] = np.concatenate((temp_data[fit_axis], data[fit_axis]), axis=1)
# # data['freqpts'] = np.concatenate((temp_data['freqpts'], data['freqpts']), axis=None)

# # print(data['freqpts'])
# # rabiEgGfFreqLenChevron.display(data=data, fit=False, signs=[-1, 1, -1, 1])

# rabiEgGfFreqLenChevron.data = data
# rabiEgGfFreqLenChevron.cfg = AttrDict(attrs['config'])
# qubits = rabiEgGfFreqLenChevron.cfg.expt.qubits
# if qubits[0] == 3: sign = 1
# elif qubits[0] == 2: sign = -1
# good_ZZ_freq = rabiEgGfFreqLenChevron.cfg.device.qubit.f_EgGf_Q[qubits[0]]
# bad_ZZ_freq = rabiEgGfFreqLenChevron.cfg.device.qubit.f_EgGf_Q[qubits[0]]+sign*rabiEgGfFreqLenChevron.cfg.device.qubit.ZZs[1]

# plot_len = rabiEgGfFreqLenChevron.cfg.device.qubit.pulses.pi_EgGf_Q.sigma[qubits[0]]
# len_ind = np.argmin(np.abs(data['lenpts']-plot_len))


# fig, axs = plt.subplots(rows, cols, figsize=(6*cols, 7), gridspec_kw={'height_ratios': [1, 1]}, sharex='col')
# ax1, ax2 = axs
# plt.suptitle(f"Q{qubits[0]} Cut Through Length {data['lenpts'][len_ind]:.3} ns")

# plt.sca(ax1)
# data['amps'][0] = (data['amps'][0] - min(data['amps'][0])) / (max(data['amps'][0]) - min(data['amps'][0]))
# plt.plot(data['freqpts'], data['amps'][0][:, len_ind], '.-')
# # plt.axvline(bad_ZZ_freq, color='r', linestyle='--')
# plt.ylabel('Population\n(Scaled)', fontsize=24)
# plt.yticks(ticks=[0, 0.5, 1.0], fontsize=24)

# plt.sca(ax2)
# data['amps'][1] = (data['amps'][1] - min(data['amps'][1])) / (max(data['amps'][1]) - min(data['amps'][1]))
# plt.plot(data['freqpts'], data['amps'][1][:, len_ind], '.-')
# # plt.axvline(bad_ZZ_freq, color='r', linestyle='--')
# plt.ylabel('Population\n(Scaled)', fontsize=24)
# plt.xlabel('Frequency (MHz)', fontsize=24)
# plt.yticks(ticks=[0, 0.5, 1.0], fontsize=24)
# # xticks = int(np.average(data['freqpts'])) + (data['freqpts'][-1] - data['freqpts'][0])//3*np.arange(-1, 2) 
# xticks = [round(good_ZZ_freq), round(bad_ZZ_freq)] 
# plt.xticks(ticks=xticks, fontsize=24)

# plt.subplots_adjust(hspace=0.07)

# plot_filename = filename[:-3]+f'_cut.svg'
# plt.savefig(plot_filename, format='svg', bbox_inches='tight', transparent=True)
# print('Saved', plot_filename)

# plt.show()

## Length Rabi eg-gf single sweep

In [ ]:
lengthrabiEgGf = meas.LengthRabiEgGfExperiment(
    soccfg=soc,
    path=expt_path,
    prefix=f"length_rabi_EgGf_qubit{qubits[0]}{qubits[1]}",
    config_file=config_path,
)

# start_len = lengthrabiEgGf.cfg.device.qubit.pulses.pi_EgGf_Q.sigma[qDrive]
# span_len = 0
# npts_len = 10

start_len = 0
# span_len = 3.75*lengthrabiEgGf.cfg.device.qubit.pulses.pi_EgGf_Q.sigma[qDrive]
span_len = 2*lengthrabiEgGf.cfg.device.qubit.pulses.pi_EgGf_Q.sigma[qDrive]
npts_len = 30
# start_len = 0.999*lengthrabiEgGf.cfg.device.qubit.pulses.pi_EgGf_Q.sigma[qDrive]
# span_len = 0.001*lengthrabiEgGf.cfg.device.qubit.pulses.pi_EgGf_Q.sigma[qDrive]
# npts_len = 50


lengthrabiEgGf.cfg.expt = dict(
    start=start_len, # pulse length (sigma) [us]
    # step=0,
    # expts=1,
    step=span_len/npts_len, # [us]
    expts=npts_len+1,
    reps=1000,
    # reps_f=25000,
    # reps_f=1000,
    # reps=4000,
    # loops=5,
    pulse_type='flat_top',
    post_process=None,
    # post_process=None,
  
    # singleshot_reps=40000,
    # singleshot_reps=1000,
    measure_f=None,
    # measure_f=[qDrive],
    init_state=init_state,
    # init_state=wrong_init_state,
    qDrive=qDrive,
    qubits=qubits,
    measure_qubits=measure_qubits,

    test_pi_half=True,
    readout_cool=False,

    full_mux_expt=full_mux_expt,
    resonator_reset=resonator_reset,
    use_robust_pulses=True,
)

# lengthrabiEgGf.cfg.device.readout.readout_length = [10,5] # [us]
print(lengthrabiEgGf.cfg)
# lengthrabiEgGf.im[lengthrabiEgGf.cfg.aliases.soc].reset()
try:
    lengthrabiEgGf.acquire(progress=True, debug=False)
except Exception:
    print("Pyro traceback:")
    print("".join(Pyro4.util.getPyroTraceback()))

In [ ]:
lengthrabiEgGf.save_data()

In [ ]:
# # # TESTING
# # lengthrabiEgGf = meas.LengthRabiEgGfExperiment(config_file=config_path)
# # temp_data, attrs = prev_data(expt_path, '00000_length_rabi_qubit01.h5')
# # data = lengthrabiEgGf.analyze(data=temp_data, fit=True)
# # lengthrabiEgGf.display(data=data, fit=False)

# lengthrabiEgGf.analyze(fit=True, post_select_calib=False, post_select_experiment=False, separate_correction=False, ps_qubits=[0, 1, 2], verbose=False)

# # plt.figure()
# # print(lengthrabiEgGf.cfg.expt.measure_qubits)
# # plt.plot(lengthrabiEgGf.data['xpts'], lengthrabiEgGf.data['amps'][0], label='q1')
# # plt.plot(lengthrabiEgGf.data['xpts'], lengthrabiEgGf.data['amps'][1], label='q2')
# # plt.ylabel('rotated amplitude')
# # plt.legend()
# # plt.show()

# # plt.figure()
# # # 00, 01, 10, 11
# # plt.plot(lengthrabiEgGf.data['xpts'], np.average(lengthrabiEgGf.data['counts_raw'][0, :, :, 2], axis=0) + np.average(lengthrabiEgGf.data['counts_raw'][0, :, :, 3], axis=0), label='q1 e counts')
# # plt.plot(lengthrabiEgGf.data['xpts'], np.average(lengthrabiEgGf.data['counts_raw'][0, :, :, 1], axis=0) + np.average(lengthrabiEgGf.data['counts_raw'][0, :, :, 3], axis=0), label='q2 e counts')
# # plt.plot(lengthrabiEgGf.data['xpts'], np.average(lengthrabiEgGf.data['counts_raw'][0, :, :, 0], axis=0) + np.average(lengthrabiEgGf.data['counts_raw'][0, :, :, 2], axis=0), label='q2 g counts')
# # plt.ylabel('counts measurement 0')
# # plt.legend()
# # plt.show()

# # plt.figure()
# # # 00, 02, 10, 12
# # plt.plot(lengthrabiEgGf.data['xpts'], np.average(lengthrabiEgGf.data['counts_raw'][1, :, :, 2], axis=0) + np.average(lengthrabiEgGf.data['counts_raw'][1, :, :, 3], axis=0), label='q1 e counts')
# # plt.plot(lengthrabiEgGf.data['xpts'], np.average(lengthrabiEgGf.data['counts_raw'][1, :, :, 1], axis=0) + np.average(lengthrabiEgGf.data['counts_raw'][1, :, :, 3], axis=0), label='q2 f counts')
# # plt.plot(lengthrabiEgGf.data['xpts'], np.average(lengthrabiEgGf.data['counts_raw'][1, :, :, 0], axis=0) + np.average(lengthrabiEgGf.data['counts_raw'][1, :, :, 2], axis=0), label='q2 g counts')
# # plt.plot(lengthrabiEgGf.data['xpts'], np.average(lengthrabiEgGf.data['counts_raw'][1, :, :, 1], axis=0), label='gf counts')
# # plt.ylabel('counts measurement 1')
# # plt.legend()
# # plt.show()

lengthrabiEgGf.display(fit=True)

# # print(lengthrabiEgGf.data['counts_calib'])

### Length rabi with post selection

In [ ]:
lengthrabiEgGf = meas.LengthRabiEgGfExperiment(
    soccfg=soc,
    path=expt_path,
    prefix=f"length_rabi_EgGf_qubit{qubits[0]}{qubits[1]}",
    config_file=config_path,
)

# start_len = lengthrabiEgGf.cfg.device.qubit.pulses.pi_EgGf_Q.sigma[qDrive]
# span_len = 0
# npts_len = 10

start_len = 0
# span_len = 3.75*lengthrabiEgGf.cfg.device.qubit.pulses.pi_EgGf_Q.sigma[qDrive]
span_len = 1.0*lengthrabiEgGf.cfg.device.qubit.pulses.pi_EgGf_Q.sigma[qDrive]
# npts_len = 15
# start_len = 0.999*lengthrabiEgGf.cfg.device.qubit.pulses.pi_EgGf_Q.sigma[qDrive]
# span_len = 2*lengthrabiEgGf.cfg.device.qubit.pulses.pi_EgGf_Q.sigma[qDrive]
# npts_len = 50
npts_len = 1

lengthrabiEgGf.cfg.expt = dict(
    start=start_len, # pulse length (sigma) [us]
    # step=0,
    # expts=1,
    step=span_len/npts_len, # [us]
    expts=npts_len+1,
    reps=20000,
    # reps_f=20000,
    # reps=100,
    reps_f=10,
    loops=1,
    pulse_type='flat_top',
    post_process='threshold',
    # post_process=None,
  
    # singleshot_reps=500,
    singleshot_reps=20000,
    singleshot_reps_f=10,
    # singleshot_reps=100,
    # measure_f=None,
    measure_f=[qDrive],
    init_state=init_state,
    # init_state='test',
    # init_state='|1+0>|1>',
    # init_state='|1>|0+1>',
    # init_state='Q2Q1_|2>|0>',
    qDrive=qDrive,
    qubits=qubits,
    measure_qubits=measure_qubits,
    
    # setup_measure='qDrive_ef',

    test_pi_half=False,
    # cool_qubits=[0, 1],
    # cool_idle=[5, 5],
    
    readout_cool=True,
    n_init_readout=1,
    n_trig=1,
    init_read_wait_us=2,

    resonator_reset=[0, 2, 3],
    full_mux_expt=True,

    use_robust_pulses=True,

    n_cycles=3,
    skip_first_pi2=True,
    # plot_IQ=True,
    # init_read_wait_us=2,
    # use_gf_readout=True,
    
    # thresholds=[0, 403.4255711140312, 356.93670935384887, 0],
    # angles=[0, -15.164283291744868, 107.2178024864672, 0],
    # ge_avgs=[array([0., 0., 0., 0.]), [332.72639357096347, -312.0792793945312, 663.0205369628906, -231.3194316731771], [-55.472186757526444, -160.28748716436127, -169.05275673311635, -537.4039020748576], array([0., 0., 0., 0.])],
    # counts_calib=[[16091, 2293, 1360, 256], [2287, 16108, 208, 1397], [2399, 307, 14988, 2306], [414, 2510, 2206, 14870]]

)

lengthrabiEgGf.cfg.device.readout.gain[1] = 1e-4

print(lengthrabiEgGf.cfg)
# lengthrabiEgGf.im[lengthrabiEgGf.cfg.aliases.soc].reset()
try:
    lengthrabiEgGf.acquire(progress=True, debug=False)
except Exception:
    print("Pyro traceback:")
    print("".join(Pyro4.util.getPyroTraceback()))

In [ ]:
lengthrabiEgGf.save_data()

In [ ]:
# # TESTING
# lengthrabiEgGf = meas.LengthRabiEgGfExperiment(config_file=config_path)
# temp_data, attrs = prev_data(expt_path, '00000_length_rabi_qubit01.h5')
# lengthrabiEgGf.data = temp_data
# lengthrabiEgGf.cfg = AttrDict(attrs['config'])
# data = lengthrabiEgGf.analyze(data=temp_data, fit=True)
# lengthrabiEgGf.display(data=data, fit=False)

ps_qubits = [0, 2, 3]

ps_adjust = [-0.1, np.inf, -0.1, -0.1]

# ps_adjust = [-1.2, -1.0, -1.7, 0.0]
# ps_adjust_f = [-1.0, -1.0, -1.0, 0.0]
# ps_adjust_f = [-1, -1, np.inf, -1]
ps_adjust_f = [np.inf]*4

# ps_adjust = [np.inf]*4
# ps_adjust = None

apply_ps = True

# lengthrabiEgGf.analyze(fit=False, post_select_calib=apply_ps, post_select_experiment=apply_ps, separate_correction=False, ps_qubits=ps_qubits, ps_adjust=ps_adjust, ps_adjust_f=ps_adjust_f)

# lengthrabiEgGf.display(fit=False)

# plt.figure()
# print(lengthrabiEgGf.cfg.expt.measure_qubits)
# plt.plot(lengthrabiEgGf.data['xpts'], lengthrabiEgGf.data['amps'][0], label='q1')
# plt.plot(lengthrabiEgGf.data['xpts'], lengthrabiEgGf.data['amps'][1], label='q2')
# plt.ylabel('rotated amplitude')
# plt.legend()
# plt.show()

# plt.figure()
# # 00, 01, 10, 11
# plt.plot(lengthrabiEgGf.data['xpts'], np.average(lengthrabiEgGf.data['counts_raw'][0, :, :, 2], axis=0) + np.average(lengthrabiEgGf.data['counts_raw'][0, :, :, 3], axis=0), label='q1 e counts')
# plt.plot(lengthrabiEgGf.data['xpts'], np.average(lengthrabiEgGf.data['counts_raw'][0, :, :, 1], axis=0) + np.average(lengthrabiEgGf.data['counts_raw'][0, :, :, 3], axis=0), label='q2 e counts')
# plt.plot(lengthrabiEgGf.data['xpts'], np.average(lengthrabiEgGf.data['counts_raw'][0, :, :, 0], axis=0) + np.average(lengthrabiEgGf.data['counts_raw'][0, :, :, 2], axis=0), label='q2 g counts')
# plt.ylabel('counts measurement 0')
# plt.legend()
# plt.show()

# plt.figure()
# # 00, 02, 10, 12
# plt.plot(lengthrabiEgGf.data['xpts'], np.average(lengthrabiEgGf.data['counts_raw'][1, :, :, 2], axis=0) + np.average(lengthrabiEgGf.data['counts_raw'][1, :, :, 3], axis=0), label='q1 e counts')
# plt.plot(lengthrabiEgGf.data['xpts'], np.average(lengthrabiEgGf.data['counts_raw'][1, :, :, 1], axis=0) + np.average(lengthrabiEgGf.data['counts_raw'][1, :, :, 3], axis=0), label='q2 f counts')
# plt.plot(lengthrabiEgGf.data['xpts'], np.average(lengthrabiEgGf.data['counts_raw'][1, :, :, 0], axis=0) + np.average(lengthrabiEgGf.data['counts_raw'][1, :, :, 2], axis=0), label='q2 g counts')
# plt.plot(lengthrabiEgGf.data['xpts'], np.average(lengthrabiEgGf.data['counts_raw'][1, :, :, 1], axis=0), label='gf counts')
# plt.ylabel('counts measurement 1')
# plt.legend()
# plt.show()


data = lengthrabiEgGf.data
histdata = dict(iqshots=[])
check_states = []
# gg, ge, eg, ee, gf, ef q1/q2(q3), g(q0), e(q0), g(q2/3), e(q2/3)
loop = 0
imeasure = 0
ilen = -1

q = 2
if q in [2, 3]:
    if lengthrabiEgGf.measure_f:
        if q == lengthrabiEgGf.cfg.expt.measure_f[0]:
            iprep_g = 0
            iprep_e = 1
            iprep_f = 4
        else:
            iprep_g = 8
            iprep_e = 9
            iprep_f = None
    else:
        iprep_g = 0
        iprep_e = 2
        iprep_f = None
elif q == 1:
    iprep_g = 0
    iprep_e = 2
    iprep_f = None
elif q == 0:
    iprep_g = 6
    iprep_e = 7
    iprep_f = None


histdata['iqshots'].append((data[f'calib_ishots_raw_loops'][loop, imeasure, iprep_g, :, -1, :], data[f'calib_qshots_raw_loops'][loop, imeasure, iprep_g, :, -1, :]))
check_states.append('g')
histdata['iqshots'].append((data[f'calib_ishots_raw_loops'][loop, imeasure, iprep_e, :, -1, :], data[f'calib_qshots_raw_loops'][loop, imeasure, iprep_e, :, -1, :]))
check_states.append('e')
if q in [2, 3] and lengthrabiEgGf.measure_f and q == lengthrabiEgGf.cfg.expt.measure_f[0]:
    histdata['iqshots'].append((data[f'calib_ishots_raw_loops'][loop, imeasure, iprep_f, :, -1, :], data[f'calib_qshots_raw_loops'][loop, imeasure, iprep_f, :, -1, :]))
    check_states.append('f')
histdata['iqshots'].append((data[f'ishots_raw_{imeasure}'][loop, ilen, :, -1, :], data[f'qshots_raw_{imeasure}'][loop, ilen, :, -1, :]))
check_states.append('test')

if lengthrabiEgGf.cfg.expt.readout_cool:
    histdata['iqshots'].append((data[f'calib_ishots_raw_loops'][loop, imeasure, iprep_g, :, 0, :], data[f'calib_qshots_raw_loops'][loop, imeasure, iprep_g, :, 0, :]))
    check_states.append('g_0')

if imeasure == 0:
    thresholds = data['thresholds_loops'][loop]
    angles = data['angles_loops'][loop]
    ge_avgs = data['ge_avgs_loops'][loop]
    if ps_adjust is None: ps_thresholds = thresholds
    else: ps_thresholds = meas.clifford_averager_program.ps_threshold_adjust(ps_thresholds_init=thresholds, adjust=ps_adjust, ge_avgs=ge_avgs, angles=angles, amplitude_mode=lengthrabiEgGf.cfg.expt.full_mux_expt)
    print('thresholds', thresholds)
    print('ps thresholds', ps_thresholds)
else:
    thresholds = data['thresholds_f_loops'][loop]
    angles = data['angles_f_loops'][loop]
    ge_avgs = data['gf_avgs_loops'][loop]
    if ps_adjust_f is None: ps_thresholds = thresholds
    else: ps_thresholds = meas.clifford_averager_program.ps_threshold_adjust(ps_thresholds_init=thresholds, adjust=ps_adjust_f, ge_avgs=ge_avgs, angles=angles, amplitude_mode=lengthrabiEgGf.cfg.expt.full_mux_expt)

_, keep_indices = meas.clifford_averager_program.post_select_shots(
    final_qubit=q,
    all_ishots_raw_q=data[f'ishots_raw_{imeasure}'][loop, ilen, :, :, :],
    all_qshots_raw_q=data[f'qshots_raw_{imeasure}'][loop, ilen, :, :, :],
    angles=angles, 
    ps_thresholds=ps_thresholds, ps_qubits=ps_qubits,
    n_init_readout=lengthrabiEgGf.cfg.expt.n_init_readout,
    post_process=None, verbose=False, return_keep_indices=True,
    amplitude_mode=lengthrabiEgGf.cfg.expt.full_mux_expt)

ishots_unrot = data[f'ishots_raw_{imeasure}'][loop, ilen, :, -1, :]
qshots_unrot = data[f'qshots_raw_{imeasure}'][loop, ilen, :, -1, :]
ishots_ps_unrot = ishots_unrot[:, keep_indices]
qshots_ps_unrot = qshots_unrot[:, keep_indices]
print('keep', 100*np.sum(keep_indices)/len(ishots_unrot[q]), '%')
if apply_ps:
    histdata['iqshots'].append((ishots_ps_unrot, qshots_ps_unrot))
    check_states.append('test_ps')

    _, g_keep_indices = meas.clifford_averager_program.post_select_shots(
        final_qubit=q,
        all_ishots_raw_q=data[f'calib_ishots_raw_loops'][loop, imeasure, iprep_g, :, :, :],
        all_qshots_raw_q=data[f'calib_qshots_raw_loops'][loop, imeasure, iprep_g, :, :, :],
        angles=angles, 
        ps_thresholds=ps_thresholds, ps_qubits=ps_qubits,
        n_init_readout=lengthrabiEgGf.cfg.expt.n_init_readout,
        post_process=None, verbose=False, return_keep_indices=True,
        amplitude_mode=lengthrabiEgGf.cfg.expt.full_mux_expt)
    g_ishots_unrot = data[f'calib_ishots_raw_loops'][loop, imeasure, iprep_g, :, -1, :]
    g_qshots_unrot = data[f'calib_qshots_raw_loops'][loop, imeasure, iprep_g, :, -1, :]
    g_ishots_ps_unrot = g_ishots_unrot[:, g_keep_indices]
    g_qshots_ps_unrot = g_qshots_unrot[:, g_keep_indices]
    histdata['iqshots'].append((g_ishots_ps_unrot, g_qshots_ps_unrot))
    check_states.append('g_ps')

    _, e_keep_indices = meas.clifford_averager_program.post_select_shots(
        final_qubit=q,
        all_ishots_raw_q=data[f'calib_ishots_raw_loops'][loop, imeasure, iprep_e, :, :, :],
        all_qshots_raw_q=data[f'calib_qshots_raw_loops'][loop, imeasure, iprep_e, :, :, :],
        angles=angles, 
        ps_thresholds=ps_thresholds, ps_qubits=ps_qubits,
        n_init_readout=lengthrabiEgGf.cfg.expt.n_init_readout,
        post_process=None, verbose=False, return_keep_indices=True,
        amplitude_mode=lengthrabiEgGf.cfg.expt.full_mux_expt)
    e_ishots_unrot = data[f'calib_ishots_raw_loops'][loop, imeasure, iprep_e, :, -1, :]
    e_qshots_unrot = data[f'calib_qshots_raw_loops'][loop, imeasure, iprep_e, :, -1, :]
    e_ishots_ps_unrot = e_ishots_unrot[:, e_keep_indices]
    e_qshots_ps_unrot = e_qshots_unrot[:, e_keep_indices]
    histdata['iqshots'].append((e_ishots_ps_unrot, e_qshots_ps_unrot))
    check_states.append('e_ps')


    if q in [2, 3] and lengthrabiEgGf.measure_f and q == lengthrabiEgGf.cfg.expt.measure_f[0]:
        _, f_keep_indices = meas.clifford_averager_program.post_select_shots(
            final_qubit=q,
            all_ishots_raw_q=data[f'calib_ishots_raw_loops'][loop, imeasure, iprep_f, :, :, :],
            all_qshots_raw_q=data[f'calib_qshots_raw_loops'][loop, imeasure, iprep_f, :, :, :],
            angles=angles, 
            ps_thresholds=ps_thresholds, ps_qubits=ps_qubits,
            n_init_readout=lengthrabiEgGf.cfg.expt.n_init_readout,
            post_process=None, verbose=False, return_keep_indices=True,
            amplitude_mode=lengthrabiEgGf.cfg.expt.full_mux_expt)
        f_ishots_unrot = data[f'calib_ishots_raw_loops'][loop, imeasure, iprep_f, :, -1, :]
        f_qshots_unrot = data[f'calib_qshots_raw_loops'][loop, imeasure, iprep_f, :, -1, :]
        f_ishots_ps_unrot = f_ishots_unrot[:, f_keep_indices]
        f_qshots_ps_unrot = f_qshots_unrot[:, f_keep_indices]
        histdata['iqshots'].append((f_ishots_ps_unrot, f_qshots_ps_unrot))
        check_states.append('f_ps')

play_pulses_list = [] # this is just so we don't print play pulses in the multihist
for check_state in check_states:
    play_pulses_list.append([0])
g_states = [0]
e_states = [1]
if imeasure == 1 and q != 1: e_states = [2]

meas.single_qubit.single_shot.multihist(
    title=f'Single Shot Q{q} with Multi Readout',
    data=histdata, check_qubit=q, check_states=check_states, play_pulses_list=play_pulses_list, g_states=g_states, e_states=e_states,
    amplitude_mode=lengthrabiEgGf.cfg.expt.full_mux_expt, plot=True, export=False, log_mode=True)

# print(lengthrabiEgGf.data['counts_calib'])

In [ ]:
I2 = lengthrabiEgGf.data['avgi'][0]
Q2 = lengthrabiEgGf.data['avgq'][0]


print('I2 min:', min(I2))
print('I2 max:', max(I2))
print('Q2 min:', min(Q2))
print('Q2 max:', max(Q2))


F0 --> G1

In [ ]:
lengthrabiF0G1 = meas.LengthRabiF0G1Experiment(
    soccfg=soc,
    path=expt_path,
    prefix=f"length_rabi_F0G1_qubit{qubits[0]}{qubits[1]}",
    config_file=config_path,
)

start_len = 0
span_len = 1.0
npts_len = 100

lengthrabiF0G1.cfg.expt = dict(
    start=start_len, # pulse length (sigma) [us]
    step=span_len/npts_len, # [us]
    expts=npts_len,
    reps=1000,
    # gain=4250, # qubit gain [DAC units],
    pulse_type='flat_top',
    # post_process='threshold',
    post_process=None,
    # singleshot_reps=20000,
    measure_f=False,
    init_state=init_state,
    qDrive=qDrive,
    qubits=qubits,
    measure_qubits=measure_qubits,
)

# lengthrabiF0G1.cfg.device.readout.readout_length = [10,5] # [us]
print(lengthrabiF0G1.cfg)
# lengthrabiF0G1.im[lengthrabiF0G1.cfg.aliases.soc].reset()
try:
    lengthrabiF0G1.acquire(progress=True, debug=True)
except Exception:
    print("Pyro traceback:")
    print("".join(Pyro4.util.getPyroTraceback()))

In [ ]:
# # TESTING
# lengthrabiF0G1 = meas.LengthRabiF0G1Experiment(config_file=config_path)
# temp_data, attrs = prev_data(expt_path, '00000_length_rabi_qubit01.h5')
# data = lengthrabiF0G1.analyze(data=temp_data, fit=True)
# lengthrabiF0G1.display(data=data, fit=False)

lengthrabiF0G1.analyze(fit=True)
lengthrabiF0G1.display(fit=True)
# print(lengthrabiF0G1.data['counts_calib'])

In [ ]:
lengthrabiF0G1.save_data()

## Pi Minus Pi

In [18]:
test_pi_half = True
full_mux_expt = False
resonator_reset = [0, 2, 3] if full_mux_expt else None

In [19]:
piminuspi = meas.PiMinusPiEgGfExperiment(
    soccfg=soc,
    path=expt_path,
    prefix=f"PiMinusPiEgGfExptQ{qubits[0]}{qubits[1]}",
    config_file=config_path,
)

good_data = False

span_f = 2.0
# npts_f = 70
npts_f = 30 if not good_data else 75

start_f = piminuspi.cfg.device.qubit.f_EgGf_Q[qDrive]-span_f/2
if test_pi_half:
    start_f = piminuspi.cfg.device.qubit.f_EgGf_Q_half[qDrive]-span_f/2

piminuspi.cfg.expt = dict(
    start_N=0, # number gate repetitions
    # step_N=1 if not good_data else 1,
    # step_N=1,
    # expts_N=8 if not good_data else 20,
    step_N=1 if not good_data else 1,
    # step_N=1,
    expts_N=8 if not good_data else 20,
    start_f=start_f,
    # start_f=5136-span_f/2,
    step_f=span_f/npts_f,
    expts_f=npts_f+1,

    reps=1000 if not good_data else 6000,
    loops=1,

    pulse_type='flat_top',
    post_process='scale',
    # post_process=None,
    singleshot_reps=10000,
    # singleshot_reps=5000,
    measure_f=False,
    init_state=init_state,
    qDrive=qDrive,
    qubits=qubits,
    measure_qubits=measure_qubits,

    readout_cool=False,
    n_init_readout=1,
    n_trig=1,
    # init_read_wait_us=5,
    init_read_wait_us=2,

    test_pi_half=test_pi_half,
    
    full_mux_expt=full_mux_expt,
    resonator_reset=resonator_reset,
)
# piminuspi.cfg.device.readout.gain[1] = 1e-4

print(piminuspi.cfg)
try:
    piminuspi.acquire(progress=True, debug=False)
except Exception:
    print("Pyro traceback:")
    print("".join(Pyro4.util.getPyroTraceback()))

{'device': {'readout': {'trig_offset': 150, 'lo_sideband': [1, 1, 1, 1], 'frequency': [-389.975, 5795.761666663333, 502.368233333334, -228.9], 'gain': [0.5, 0.6, 0.4, 0.3], 'readout_length': [5, 20, 10, 8], 'frequency_ef': [-391.91408405, 5795.76166666, 500.44439227, -230.30732892], 'gain_ef': [0.05, 0.6, 0.4, 0.25], 'readout_length_ef': [1, 20, 10, 8], 'relax_delay': [450, 450, 450, 450]}, 'qubit': {'f_ge': [4111.103663892308, 4105.758870288914, 4110.961390890517, 4110.561006068113, 3442.9791477220547, 3448.3312965131795, 3447.2256002913127, 3447.022898974775, 4762.736987583428, 4761.772206046936, 4762.882027802732, 4762.340227746316, 4380.990256112639, 4380.2241341449835, 4380.999038792971, 4381.537145425829], 'f_ef': [3884.9668600899413, 0.0, 0.0, 0.0, 0.0, 3347.846225737386, 0.0, 0.0, 4571.9056482340775, 4571.009319367402, 4572.904716852475, 0.0, 4211.704645679207, 4206.869581627414, 0.0, 4208.123751627149], 'f_EgGf': [2686.53594683, 0, 2017.3, 2414.92850999], 'f_EgGf_Q_half': [0, 

  0%|          | 0/4 [00:00<?, ?it/s]

Qubit  (2)
ge fidelity (%): 81.83 	 angle (deg): 176.31576693690377 	 threshold ge: 44.657194585017955
Qubit  (1)
ge fidelity (%): 76.08000000000003 	 angle (deg): 20.26117449109091 	 threshold ge: 54.44249717312033


  0%|          | 0/8 [00:00<?, ?it/s]

 s:\Connie\experiments\qram_tprocv1_expts\experiments\clifford_averager_program.py: 1877

In [20]:
data = piminuspi.data
piminuspi.analyze(fit=True)
piminuspi.display(fit=True)
print(piminuspi.data['best_freq'])

Attempted to init fitparam 3 to 0.0, which is out of bounds 0 to 0.598299747698102. Instead init to 0.299149873849051
Fit best freq (Q2) 5885.577205906209 which is 5.072654922903894 away from old freq 5880.504550983305
Attempted to init fitparam 3 to 0.0, which is out of bounds 0 to 0.6758717444941975. Instead init to 0.33793587224709876
Fit best freq (Q1) 5885.636044906932 which is 5.131493923626294 away from old freq 5880.504550983305
best 1 0.05803664986720945
Best freq fitted 5885.636044906932
5885.636044906932


## Pi Minus Pi Gain vs Freq sweep

In [48]:
# opt_gain = 14500
# opt_len = 0.095
# opt_freq = 5138.0
# sweep2Q_q = 3

piminuspi_sweep_expt = Experiment(
    soccfg=soc,
    path=expt_path,
    prefix=f"eggf_piminuspi_gain_sweep_Q{qubits[0]}{qubits[1]}",
    config_file=config_path,
)
# piminuspi_sweep_expt.cfg = AttrDict(deepcopy(yaml_cfg))

opt_gain = piminuspi_sweep_expt.cfg.device.qubit.pulses.pi_EgGf_Q.half_gain[qDrive]
opt_len = piminuspi_sweep_expt.cfg.device.qubit.pulses.pi_EgGf_Q.half_sigma[qDrive]
opt_freq = piminuspi_sweep_expt.cfg.device.qubit.f_EgGf_Q_half[qDrive]
sweep2Q_q = qDrive


if not test_pi_half:
    piminuspi_sweep_expt.cfg.device.qubit.pulses.pi_EgGf_Q.sigma[sweep2Q_q] = opt_len
    piminuspi_sweep_expt.cfg.device.qubit.f_EgGf_Q[sweep2Q_q] = opt_freq
else:
    piminuspi_sweep_expt.cfg.device.qubit.pulses.pi_EgGf_Q.half_sigma[sweep2Q_q] = opt_len
    piminuspi_sweep_expt.cfg.device.qubit.f_EgGf_Q_half[sweep2Q_q] = opt_freq

span_gain = 1000
npts_gain = 10
start_gain = opt_gain - span_gain/2
step_gain = span_gain/npts_gain
expts_gain = npts_gain + 1
gain_sweep = start_gain + step_gain * np.arange(expts_gain)

start_N = 1
step_N = 2 if not good_data else 1
expts_N = 5 if not good_data else 10
cycle_sweep = start_N + step_N * np.arange(expts_N)

span_f = 2.0
npts_f = 20
start_f = opt_freq - span_f/2
step_f = span_f/npts_f
expts_f = npts_f + 1

freq_sweep = start_f + step_f * np.arange(expts_f)

print('gain_sweep', gain_sweep.tolist())
print('cycle_sweep', cycle_sweep.tolist())
print('freq_sweep', freq_sweep.tolist())

piminuspi_sweep_expt.data = dict(
    avgi = np.zeros((2, len(gain_sweep), len(cycle_sweep), len(freq_sweep))),
    avgq = np.zeros((2, len(gain_sweep), len(cycle_sweep), len(freq_sweep))),
    amps = np.zeros((2, len(gain_sweep), len(cycle_sweep), len(freq_sweep))),
    products = np.zeros((2, len(gain_sweep), len(freq_sweep))),
    cycle_sweep = cycle_sweep,
    freq_sweep = freq_sweep,
    gain_sweep = gain_sweep,
)

angles = thresholds = ge_avgs = counts_calib = None
    

for i_gain, gain in enumerate(gain_sweep):
    piminuspi = meas.PiMinusPiEgGfExperiment(
        soccfg=soc,
        path=expt_path,
        prefix=f"PiMinusPiEgGfExptQ{qubits[0]}{qubits[1]}",
        config_file=config_path,
    )

    piminuspi.cfg = AttrDict(deepcopy(piminuspi_sweep_expt.cfg))

    if not test_pi_half:
        piminuspi.cfg.device.qubit.pulses.pi_EgGf_Q.gain[sweep2Q_q] = int(gain)
    else:
        piminuspi.cfg.device.qubit.pulses.pi_EgGf_Q.half_gain[sweep2Q_q] = int(gain)

    piminuspi.cfg.expt = dict(
        start_N=start_N, # number gate repetitions
        step_N=step_N,
        expts_N=expts_N,
        start_f=start_f,
        step_f=step_f,
        expts_f=expts_f,

        reps=300 if not good_data else 3000,
        loops=1,

        pulse_type='flat_top',
        post_process='scale',
        singleshot_reps=10000,
        angles=angles,
        thresholds=thresholds,
        ge_avgs=ge_avgs,
        counts_calib=counts_calib,

        measure_f=False,
        init_state=init_state,
        qDrive=qDrive,
        qubits=qubits,
        measure_qubits=measure_qubits,

        test_pi_half=test_pi_half,

        readout_cool=False,
        n_init_readout=1,
        n_trig=1,
        init_read_wait_us=2,
        
        full_mux_expt=full_mux_expt,
        resonator_reset=resonator_reset,
    )

    try:
        piminuspi.acquire(progress=True, debug=False)
    except Exception:
        print("Pyro traceback:")
        print("".join(Pyro4.util.getPyroTraceback()))

    data = piminuspi.data
    piminuspi.analyze(fit=False)
    piminuspi.display(fit=False)

    for data_name in ['avgi', 'avgq', 'amps']:
        piminuspi_sweep_expt.data[data_name][:, i_gain, :, :] = piminuspi.data[data_name]
    piminuspi_sweep_expt.data["products"][:, i_gain, :] = piminuspi.data["products"]

    angles = data["angles"]
    thresholds = data["thresholds"]
    ge_avgs = data["ge_avgs"]
    counts_calib = data["counts_calib"]

gain_sweep [11400.0, 11500.0, 11600.0, 11700.0, 11800.0, 11900.0, 12000.0, 12100.0, 12200.0, 12300.0, 12400.0]
cycle_sweep [1, 3, 5, 7, 9]
freq_sweep [5884.5, 5884.6, 5884.7, 5884.8, 5884.9, 5885.0, 5885.1, 5885.2, 5885.3, 5885.4, 5885.5, 5885.6, 5885.7, 5885.8, 5885.9, 5886.0, 5886.1, 5886.2, 5886.3, 5886.4, 5886.5]


  0%|          | 0/4 [00:00<?, ?it/s]

Qubit  (2)
ge fidelity (%): 82.27942694269424 	 angle (deg): -27.066401717620096 	 threshold ge: 44.90051398003603
Qubit  (1)
ge fidelity (%): 74.24934593459346 	 angle (deg): 20.815068976872915 	 threshold ge: 55.851158890591336


  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

...

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

 s:\Connie\experiments\qram_tprocv1_expts\experiments\two_qubit\length_rabi_EgGf.py: 2971

  0%|          | 0/5 [00:00<?, ?it/s]

In [50]:
data = piminuspi_sweep_expt.data
qA, qB = qubits
label = "($X_{\pi}, X_{-\pi})^N$"
if test_pi_half:
    # label = "($X_{\pi/2}, X_{-\pi/2})^N$"
    label = "($X_{\pi/2}, X_{\pi/2}, X_{-\pi/2}, X_{-\pi/2})^N$"
title = (
    f"Q{qA}/Q{qB} Product of Ns for {label}\n Len {opt_len:.3f}, Ns={cycle_sweep.tolist()}"
)

inner_sweep = data["freq_sweep"]
outer_sweep = data["gain_sweep"]

y_sweep = outer_sweep
x_sweep = inner_sweep

plt.figure(figsize=(8, 9))
plt.suptitle(title, fontsize=20)
data_name = "products"

ax_qA = plt.subplot(211, title=f"QA ({qA})")
ax_qA.set_ylabel(f"Gain", fontsize=18)
ax_qA.tick_params(axis="both", which="major", labelsize=16)
plot_data = data[data_name][0]
plt.pcolormesh(x_sweep, y_sweep, plot_data, cmap="viridis", shading="auto")
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=18)

ax_qB = plt.subplot(212, title=f"QB ({qB})")
ax_qB.set_ylabel(f"Gain", fontsize=18)
ax_qB.set_xlabel("Frequency [MHz]", fontsize=18)
ax_qB.tick_params(axis="both", which="major", labelsize=16)
plot_data = data[data_name][1]
plt.pcolormesh(x_sweep, y_sweep, plot_data, cmap="viridis", shading="auto")
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=18)

plt.tight_layout()
plt.show()
    

In [36]:
print(piminuspi_sweep_expt.save_data())

S:\QRAM\qram_4QR2\data\data_241025\00001_eggf_piminuspi_gain_sweep_Q21.h5


## Pi Pi Gain vs Freq sweep

In [96]:
# opt_gain = 14500
# opt_len = 0.095
# opt_freq = 5138.0
# sweep2Q_q = 3

pipi_sweep_expt = Experiment(
    soccfg=soc,
    path=expt_path,
    prefix=f"eggf_pipi_gain_sweep_Q{qubits[0]}{qubits[1]}",
    config_file=config_path,
)
# pipi_sweep_expt.cfg = AttrDict(deepcopy(yaml_cfg))

if not test_pi_half:
    pipi_sweep_expt.cfg.device.qubit.pulses.pi_EgGf_Q.sigma[sweep2Q_q] = opt_len
    pipi_sweep_expt.cfg.device.qubit.f_EgGf_Q[sweep2Q_q] = opt_freq
else:
    pipi_sweep_expt.cfg.device.qubit.pulses.pi_EgGf_Q.half_sigma[sweep2Q_q] = opt_len
    pipi_sweep_expt.cfg.device.qubit.f_EgGf_Q_half[sweep2Q_q] = opt_freq


opt_gain = pipi_sweep_expt.cfg.device.qubit.pulses.pi_EgGf_Q.half_gain[qDrive]
opt_len = pipi_sweep_expt.cfg.device.qubit.pulses.pi_EgGf_Q.half_sigma[qDrive]
opt_freq = pipi_sweep_expt.cfg.device.qubit.f_EgGf_Q_half[qDrive]
sweep2Q_q = qDrive


span_gain = 500
npts_gain = 10
start_gain = opt_gain - span_gain/2
step_gain = span_gain/npts_gain
expts_gain = npts_gain + 1
gain_sweep = start_gain + step_gain * np.arange(expts_gain)

# start_N = 1
# step_N = 2 if not good_data else 1
# expts_N = 5 if not good_data else 10
start_N = 5
step_N = 5 if not good_data else 1
expts_N = 4 if not good_data else 10
# start_N = 5
# step_N = 0
# expts_N = 1
cycle_sweep = start_N + step_N * np.arange(expts_N)

span_f = 1.0
# npts_f = 30
npts_f = 10
start_f = opt_freq - span_f/2
step_f = span_f/npts_f
expts_f = npts_f + 1

freq_sweep = start_f + step_f * np.arange(expts_f)

print('gain_sweep', gain_sweep.tolist())
print('cycle_sweep', cycle_sweep.tolist())
print('freq_sweep', freq_sweep.tolist())

pipi_sweep_expt.data = dict(
    avgi = np.zeros((2, len(gain_sweep), len(cycle_sweep), len(freq_sweep))),
    avgq = np.zeros((2, len(gain_sweep), len(cycle_sweep), len(freq_sweep))),
    amps = np.zeros((2, len(gain_sweep), len(cycle_sweep), len(freq_sweep))),
    products = np.zeros((2, len(gain_sweep), len(freq_sweep))),
    cycle_sweep = cycle_sweep,
    freq_sweep = freq_sweep,
    gain_sweep = gain_sweep,
)

angles = thresholds = ge_avgs = counts_calib = None
    

for i_freq, freq in enumerate(tqdm(freq_sweep)):
    pipi = meas.PiTrainEgGfGainSweepExperiment(
        soccfg=soc,
        path=expt_path,
        prefix=f"PiTrainEgGfGainSweepExptQ{qubits[0]}{qubits[1]}",
        config_file=config_path,
    )

    pipi.cfg = AttrDict(deepcopy(pipi_sweep_expt.cfg))

    if not test_pi_half:
        pipi.cfg.device.qubit.f_EgGf_Q[sweep2Q_q] = freq
    else:
        pipi.cfg.device.qubit.f_EgGf_Q_half[sweep2Q_q] = freq

    pipi.cfg.expt = dict(
        start_gain=start_gain,
        step_gain=step_gain,
        expts_gain=expts_gain,
        start_N=start_N,
        step_N=step_N,
        expts_N=expts_N,

        reps=500 if not good_data else 1000,
        loops=1,

        pulse_type='flat_top',
        post_process='scale',
        singleshot_reps=10000,
        angles=angles,
        thresholds=thresholds,
        ge_avgs=ge_avgs,
        counts_calib=counts_calib,

        measure_f=False,
        init_state=init_state,
        qDrive=qDrive,
        qubits=qubits,
        measure_qubits=measure_qubits,

        test_pi_half=test_pi_half,

        readout_cool=False,
        n_init_readout=1,
        n_trig=1,
        init_read_wait_us=2,
        
        full_mux_expt=full_mux_expt,
        resonator_reset=resonator_reset,
    )

    try:
        pipi.acquire(progress=True, debug=False)
    except Exception:
        print("Pyro traceback:")
        print("".join(Pyro4.util.getPyroTraceback()))

    data = pipi.data
    pipi.analyze(fit=False)
    pipi.display(fit=False)

    for data_name in ['avgi', 'avgq', 'amps']:
        pipi_sweep_expt.data[data_name][:, :, :, i_freq] = np.swapaxes(pipi.data[data_name], 1, 2)
    pipi_sweep_expt.data["products"][:, :, i_freq] = pipi.data["products"]

    angles = data["angles"]
    thresholds = data["thresholds"]
    ge_avgs = data["ge_avgs"]
    counts_calib = data["counts_calib"]

gain_sweep [11897.0, 11947.0, 11997.0, 12047.0, 12097.0, 12147.0, 12197.0, 12247.0, 12297.0, 12347.0, 12397.0]
cycle_sweep [5, 10, 15, 20]
freq_sweep [5885.0, 5885.1, 5885.2, 5885.3, 5885.4, 5885.5, 5885.6, 5885.7, 5885.8, 5885.9, 5886.0]


  0%|          | 0/11 [00:00<?, ?it/s]

  0%|          | 0/11 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

Qubit  (2)
ge fidelity (%): 83.05999999999999 	 angle (deg): 86.8063311984809 	 threshold ge: 44.98106707895821
Qubit  (1)
ge fidelity (%): 72.27929192919292 	 angle (deg): 19.569739100149643 	 threshold ge: 56.62996251761378


...

 82%|████████▏ | 9/11 [02:29<00:29, 14.87s/it]

  0%|          | 0/11 [00:00<?, ?it/s]

 91%|█████████ | 10/11 [02:44<00:14, 14.78s/it]iments\two_qubit\length_rabi_EgGf.py: 3263

  0%|          | 0/11 [00:00<?, ?it/s]

100%|██████████| 11/11 [02:59<00:00, 16.28s/it]


In [104]:
data = pipi_sweep_expt.data
qA, qB = qubits
label = "($X_{\pi}, X_{-\pi})^N$"
if test_pi_half:
    label = "($X_{\pi/2}, X_{\pi/2})^{2N}$"
title = (
    f"Q{qA}/Q{qB} Product of Ns for {label}\n Len {opt_len:.3f}, Ns={cycle_sweep.tolist()}"
)

inner_sweep = data["freq_sweep"]
outer_sweep = data["gain_sweep"]

y_sweep = outer_sweep
x_sweep = inner_sweep

plt.figure(figsize=(8, 9))
plt.suptitle(title, fontsize=20)
data_name = "products"

ax_qA = plt.subplot(211, title=f"QA ({qA})")
ax_qA.set_ylabel(f"Gain", fontsize=18)
ax_qA.tick_params(axis="both", which="major", labelsize=16)
plot_data = data[data_name][0]
plt.pcolormesh(x_sweep, y_sweep, plot_data, cmap="viridis", shading="auto")
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=18)

ax_qB = plt.subplot(212, title=f"QB ({qB})")
ax_qB.set_ylabel(f"Gain", fontsize=18)
ax_qB.set_xlabel("Frequency [MHz]", fontsize=18)
ax_qB.tick_params(axis="both", which="major", labelsize=16)
plot_data = data[data_name][1]
plt.pcolormesh(x_sweep, y_sweep, plot_data, cmap="viridis", shading="auto")
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=18)

plt.tight_layout()
plt.show()
    

In [103]:
print(pipi_sweep_expt.save_data())

S:\QRAM\qram_4QR2\data\data_241025\00002_eggf_pipi_gain_sweep_Q21.h5


In [ ]:
print(pipi_sweep_expt.data['amps'].shape)
plt.figure()
plt.plot(pipi_sweep_expt.data['gain_sweep'], pipi_sweep_expt.data['amps'][0, :, 0, 4], label='q1')
plt.show()

(2, 11, 5, 21)


## Multiply the pi/pi and pi/-pi

In [158]:
pipi_freq_sweep, pipi_gain_sweep = pipi_sweep_expt.data["freq_sweep"], pipi_sweep_expt.data["gain_sweep"]
piminuspi_freq_sweep, piminuspi_gain_sweep = piminuspi_sweep_expt.data["freq_sweep"], piminuspi_sweep_expt.data["gain_sweep"]
reduced_freq_sweep = np.linspace(np.max([pipi_freq_sweep[0], piminuspi_freq_sweep[0]]), np.min([pipi_freq_sweep[-1], piminuspi_freq_sweep[-1]]), 20)
reduced_gain_sweep = np.linspace(np.max([pipi_gain_sweep[0], piminuspi_gain_sweep[0]]), np.min([pipi_gain_sweep[-1], piminuspi_gain_sweep[-1]]), 25)
reduced_y = reduced_gain_sweep
reduced_x = reduced_freq_sweep

print("reduced gain range", reduced_gain_sweep[0], "to", reduced_gain_sweep[-1])
print("reduced freq range", reduced_freq_sweep[0], "to", reduced_freq_sweep[-1])

fine_pipi_data = np.zeros((2, len(reduced_y), len(reduced_x)))
fine_piminuspi_data = np.zeros((2, len(reduced_y), len(reduced_x)))
for iq, q in enumerate(qubits):
    y, x = pipi_gain_sweep, pipi_freq_sweep
    pipi_interp = sp.interpolate.RegularGridInterpolator((y, x), pipi_sweep_expt.data["products"][iq], bounds_error=False, fill_value=0)

    y, x = piminuspi_gain_sweep, piminuspi_freq_sweep
    piminuspi_interp = sp.interpolate.RegularGridInterpolator((y, x), piminuspi_sweep_expt.data["products"][iq], bounds_error=False, fill_value=0)

    yg, xg = np.meshgrid(reduced_y, reduced_x, indexing='ij')
    fine_pipi_data[iq] = pipi_interp((yg, xg))
    fine_piminuspi_data[iq] = piminuspi_interp((yg, xg))



# fine_pipi_data = pipi_sweep_expt.data["products"]
# fine_piminuspi_data = piminuspi_sweep_expt.data["products"]
# reduced_x = pipi_sweep_expt.data["freq_sweep"]
# reduced_y = pipi_sweep_expt.data["gain_sweep"]


x_sweep = reduced_x
y_sweep = reduced_y

prod_of_prods_data_q = np.array([fine_piminuspi_data[i] * fine_pipi_data[i] for i in range(2)])

# Interpolated pi pi data
label_pipi = "($X_{\pi}, X_{\pi})^N$"
if test_pi_half:
    label_pipi = "($X_{\pi/2}, X_{\pi/2})^{2N}$"
title = (
    f"Interpolated Q{qA}/Q{qB} Product of Ns for {label_pipi}\n Len {opt_len:.3f}, Ns={pipi_sweep_expt.data['cycle_sweep'].tolist()}"
)

plt.figure(figsize=(8, 9))
plt.suptitle(title, fontsize=20)

ax_qA = plt.subplot(211, title=f"QA ({qA})")
ax_qA.set_ylabel(f"Gain", fontsize=18)
ax_qA.tick_params(axis="both", which="major", labelsize=16)
plot_data = fine_pipi_data[0]
plt.pcolormesh(x_sweep, y_sweep, plot_data, cmap="viridis", shading="auto")
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=18)

ax_qB = plt.subplot(212, title=f"QB ({qB})")
ax_qB.set_ylabel(f"Gain", fontsize=18)
ax_qB.set_xlabel("Frequency [MHz]", fontsize=18)
ax_qB.tick_params(axis="both", which="major", labelsize=16)
plot_data = fine_pipi_data[1]
plt.pcolormesh(x_sweep, y_sweep, plot_data, cmap="viridis", shading="auto")
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=18)

plt.tight_layout()
plt.show()

# Interpolated pi -pi data
label_piminuspi = "($X_{\pi}, X_{-\pi})^N$"
if test_pi_half:
    # label_piminuspi = "($X_{\pi/2}, X_{-\pi/2})^{N}$"
    label_piminuspi = "($X_{\pi/2}, X_{\pi/2}, X_{-\pi/2}, X_{-\pi/2})^N$"
title = (
    f"Interpolated Q{qA}/Q{qB} Product of Ns for {label_piminuspi}\n Len {opt_len:.3f}, Ns={piminuspi_sweep_expt.data['cycle_sweep'].tolist()}"
)

plt.figure(figsize=(8, 9))
plt.suptitle(title, fontsize=20)

ax_qA = plt.subplot(211, title=f"QA ({qA})")
ax_qA.set_ylabel(f"Gain", fontsize=18)
ax_qA.tick_params(axis="both", which="major", labelsize=16)
plot_data = fine_piminuspi_data[0]
plt.pcolormesh(x_sweep, y_sweep, plot_data, cmap="viridis", shading="auto")
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=18)

ax_qB = plt.subplot(212, title=f"QB ({qB})")
ax_qB.set_ylabel(f"Gain", fontsize=18)
ax_qB.set_xlabel("Frequency [MHz]", fontsize=18)
ax_qB.tick_params(axis="both", which="major", labelsize=16)
plot_data = fine_piminuspi_data[1]
plt.pcolormesh(x_sweep, y_sweep, plot_data, cmap="viridis", shading="auto")
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=18)

plt.tight_layout()
plt.show()

# Product of the two inteprolated data sets
title = (
    f"Interpolated Q{qA}/Q{qB} Product {label_piminuspi} $\\times$ {label_pipi}\n Len {opt_len:.3f}"
)

plt.figure(figsize=(8, 9))
plt.suptitle(title, fontsize=20)

ax_qA = plt.subplot(211, title=f"QA ({qA})")
ax_qA.set_ylabel(f"Gain", fontsize=18)
ax_qA.tick_params(axis="both", which="major", labelsize=16)
plot_data = prod_of_prods_data_q[0]
plt.pcolormesh(x_sweep, y_sweep, plot_data, cmap="viridis", shading="auto")
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=18)

ax_qB = plt.subplot(212, title=f"QB ({qB})")
ax_qB.set_ylabel(f"Gain", fontsize=18)
ax_qB.set_xlabel("Frequency [MHz]", fontsize=18)
ax_qB.tick_params(axis="both", which="major", labelsize=16)
plot_data = prod_of_prods_data_q[1]
plt.pcolormesh(x_sweep, y_sweep, plot_data, cmap="viridis", shading="auto")
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=18)

plt.tight_layout()
plt.show()

reduced gain range 11897.0 to 12397.0
reduced freq range 5885.0 to 5886.0


In [180]:
prod_of_prods_data = np.prod(prod_of_prods_data_q, axis=0)
blobs_list = find_blobs(prod_of_prods_data, x_sweep=x_sweep, y_sweep=y_sweep)

idxy = np.argmax(prod_of_prods_data)
ymax, xmax = np.unravel_index(idxy, prod_of_prods_data.shape)
print("best freq from max", x_sweep[xmax])
print("best gain from max", y_sweep[ymax])

for blob in blobs_list:
    r, c, sigma = blob
    opt_freq = x_sweep[int(c)]
    opt_gain = y_sweep[int(r)]
    sigma_x = sigma * (x_sweep[1] - x_sweep[0])
    sigma_y = sigma * (y_sweep[1] - y_sweep[0])
    print(f"Blob found at: freq {opt_freq}, gain {opt_gain}) with sigma_freq {sigma_x:.3f}, sigma_gain {sigma_y:.3f}")

best freq from max 5885.631578947368
best gain from max 12147.0
Blob found at: freq 5885.631578947368, gain 12167.833333333334) with sigma_freq 0.223, sigma_gain 88.388


## Error amplification

In [ ]:
npulsecalib = meas.NPulseEgGfExperiment(
    soccfg=soc,
    path=expt_path,
    prefix=f"NPulseExptEgGfQ{qubits[0]}{qubits[1]}",
    config_file=config_path,
)

npulsecalib.cfg.expt = dict(
    start=1, # number gate repetitions
    step=1,
    expts=20,
    reps=2000,
    loops=5,

    pulse_type='flat_top',
    post_process='threshold',
    # post_process='scale',
    singleshot_reps=10000,
    measure_f=False,
    init_state=init_state,
    qDrive=qDrive,
    qubits=qubits,
    measure_qubits=measure_qubits,

    full_mux_expt=full_mux_expt,
    resonator_reset=resonator_reset,

    test_pi_half=True,

    # thresholds=[0, 43.41811611277916, 10.392971620466497, 0],
    # angles=[0, -101.73981388813908, 141.40196686195807, 0],
    # ge_avgs=[array([0., 0., 0., 0.]), [-17.911846226415093, 35.64949229017567, -20.07804817826936, 45.928960572543915], [-4.966982513216755, -4.092012810085401, -10.202727938186255, -8.330811813745425], array([0., 0., 0., 0.])],
    # counts_calib=[[11285, 4713, 2839, 1163], [4149, 11780, 1119, 2952], [4335, 1723, 9750, 4192], [1394, 3570, 3963, 11073]]
)

print(npulsecalib.cfg)
try:
    npulsecalib.go(analyze=False, display=False, progress=True, save=False)
except Exception:
    print("Pyro traceback:")
    print("".join(Pyro4.util.getPyroTraceback()))

In [ ]:
# # TESTING
# temp_data, attrs = prev_data(expt_path, '00000_voltage_sweep_qubit_spectroscopy_qubit3.h5')
# data = lengthrabi.analyze(data=temp_data, fit=True)

data = npulsecalib.data
npulsecalib.analyze(fit=True)
npulsecalib.display(fit=True)

npulsecalib_fit, npulsecalib_fit_err = meas.fitting.get_best_fit(npulsecalib.data, prefixes=['fitA', 'fitB'], check_measures=['avgi'])

if np.all(npulsecalib_fit == data['fitA_avgi']): use_q =  npulsecalib.cfg.expt.measure_qubits[0]
else: use_q = npulsecalib.cfg.expt.measure_qubits[1]
if use_q == qDrive: sign = 1
else: sign = -1
print(use_q)
angle_err = npulsecalib_fit[1]
amp_ratio = (180 - sign*angle_err) / 180
new_opt_len = npulsecalib.cfg.device.qubit.pulses.pi_EgGf_Q.sigma[qDrive] / amp_ratio
print(new_opt_len)

In [ ]:
npulsecalib.save_data()

## Pi pulse train

https://arxiv.org/pdf/2304.06087

In [56]:
full_mux_expt = False
resonator_reset = [0, 2, 3] if full_mux_expt else None
use_robust_pulses = True

test_pi_half = True

In [80]:
train_expt = meas.PiTrainEgGfGainSweepExperiment(
    soccfg=soc,
    path=expt_path,
    prefix=f"eggf_train_Q{qubits[0]}{qubits[1]}",
    config_file=config_path,
)

span_gain = 4000
npts_gain = 50
start_gain = train_expt.cfg.device.qubit.pulses.pi_EgGf_Q.gain[qDrive]-span_gain/2
if test_pi_half:
    start_gain = train_expt.cfg.device.qubit.pulses.pi_EgGf_Q.half_gain[qDrive]-span_gain/2
step_gain = span_gain/npts_gain
expts_gain = npts_gain + 1

start_N = 1 # this is the number of pi swap cycles
step_N = 2
expts_N = 3
# start_N = 30 # this is the number of pi swap cycles
# step_N = 30
# expts_N = 2

train_expt.cfg.expt = dict(
    start_gain=start_gain,
    step_gain=step_gain,
    expts_gain=expts_gain,
    start_N=start_N,
    step_N=step_N,
    expts_N=expts_N,

    reps=1500,
    loops=1,

    pulse_type='flat_top',
    post_process='threshold',
    singleshot_reps=10000,
    measure_f=False,
    init_state=init_state,
    qDrive=qDrive,
    qubits=qubits,
    measure_qubits=measure_qubits,

    test_pi_half=test_pi_half,
    # test_pi_half=False,

    full_mux_expt=full_mux_expt,
    resonator_reset=resonator_reset,

    use_robust_pulses=use_robust_pulses,
)

print(train_expt.cfg)
try:
    train_expt.acquire(progress=True, debug=False)
except Exception:
    print("Pyro traceback:")
    print("".join(Pyro4.util.getPyroTraceback()))


{'device': {'readout': {'trig_offset': 150, 'lo_sideband': [1, 1, 1, 1], 'frequency': [-389.975, 5795.761666663333, 502.368233333334, -228.9], 'gain': [0.5, 0.6, 0.4, 0.3], 'readout_length': [5, 20, 10, 8], 'frequency_ef': [-391.91408405, 5795.76166666, 500.44439227, -230.30732892], 'gain_ef': [0.05, 0.6, 0.4, 0.25], 'readout_length_ef': [1, 20, 10, 8], 'relax_delay': [450, 450, 450, 450]}, 'qubit': {'f_ge': [4111.103663892308, 4105.758870288914, 4110.961390890517, 4110.561006068113, 3442.9791477220547, 3448.3312965131795, 3447.2256002913127, 3447.022898974775, 4762.736987583428, 4761.772206046936, 4762.882027802732, 4762.340227746316, 4380.990256112639, 4380.2241341449835, 4380.999038792971, 4381.537145425829], 'f_ef': [3884.9668600899413, 0.0, 0.0, 0.0, 0.0, 3347.846225737386, 0.0, 0.0, 4571.9056482340775, 4571.009319367402, 4572.904716852475, 0.0, 4211.704645679207, 4206.869581627414, 0.0, 4205.123751627149], 'f_EgGf': [2686.53594683, 0, 2017.3, 2414.92850999], 'f_EgGf_Q_half': [0, 

  0%|          | 0/51 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [79]:
train_expt.analyze(fit=True)
train_expt.display(fit=True)

plt.figure()
for i in range(len(train_expt.data["cycle_sweep"])):
    plt.plot(train_expt.data["gain_sweep"], train_expt.data["amps"][0, i, :], label=f'N={train_expt.data["cycle_sweep"][i]}')
plt.legend()
plt.title(f'Q{qubits[0]}')
plt.tight_layout()
plt.show()

plt.figure()
for i in range(len(train_expt.data["cycle_sweep"])):
    plt.plot(train_expt.data["gain_sweep"], train_expt.data["amps"][1, i, :], label=f'N={train_expt.data["cycle_sweep"][i]}')
plt.legend()
plt.title(f'Q{qubits[1]}')
plt.tight_layout()
plt.show()


print(train_expt.data['best_gain'])

Attempted to init fitparam 3 to 0.0, which is out of bounds 0 to 0.7894789152785344. Instead init to 0.3947394576392672
Fit best gain (Q2) 12608.320955198844
Attempted to init fitparam 3 to 0.0, which is out of bounds 0 to 0.9115392422092774. Instead init to 0.4557696211046387
Fit best gain (Q1) 12618.845814129929
best 0 0.1441459723746166
Best gain fitted 12608.320955198844
12608.320955198844


In [ ]:
print(train_expt.save_data())

## Amplitude Rabi eg-gf single sweep

In [ ]:
amprabiEgGf = meas.AmplitudeRabiEgGfExperiment(
    soccfg=soc,
    path=expt_path,
    prefix=f"amp_rabi_EgGf_qubit{qubits[0]}{qubits[1]}",
    config_file=config_path,
)

span = 1000
npts = 101
   
amprabiEgGf.cfg.expt = dict(
    start=0, # qubit gain [dac level]
    step=int(span/npts), # [dac level]
    expts=npts,
    # step=0.001,
    rounds=10,
    reps=500,
    # pi_EgGf_sigma=0.600, # gaussian sigma/const pulse length, overrides config [us]
    pulse_type='const',
    qubits=qubits,
    singleshot=False,
)

# amprabiEgGf.cfg.device.readout.readout_length = [15, 15] # [us]
# amprabiEgGf.cfg.device.readout.relax_delay = [600, 50] # Wait time between experiments [us]

print(amprabiEgGf.cfg)

try:
    amprabiEgGf.go(analyze=False, display=False, progress=True, save=False)
except Exception:
    print("Pyro traceback:")
    print("".join(Pyro4.util.getPyroTraceback()))

In [ ]:
# # TESTING
# amprabiEgGf = meas.AmplitudeRabiEgGfExperiment(config_file=config_path)
# amprabiEgGf.cfg.expt = dict(sigma_test=0.400, singleshot=False,)
# temp_data, attrs = prev_data(expt_path, '00000_amp_rabi_EgGf_qubit01.h5')
# data = amprabiEgGf.analyze(data=temp_data, fit=True)
# amprabiEgGf.display(data=data, fit=True)

# # apply mask onto data
# temp_data = dict()
# for key, value in amprabiEgGf.data.items():
#     if len(np.shape(value)) > 1:
#         temp_data.update({key:np.array([value[0][:50], value[1][:50]])})
#     else: temp_data.update({key:value[:50]})

amprabiEgGf.analyze(data=None, fit=True)
amprabiEgGf.display(data=None, fit=False)

In [ ]:
amprabiEgGf.save_data()

In [ ]:
prog = amprabiEgGf.prog
shot_avg = 10
threshold = amprabiEgGf.cfg.device.readout.threshold
bufi = np.array([prog.di_buf[i] for i, ch in enumerate(prog.ro_chs)])
avgi = []
for bufi_ch in bufi:
    # drop extra shots that aren't divisible into averages
    new_bufi_ch = bufi_ch[:len(bufi_ch) - (len(bufi_ch) % shot_avg)]
    # average over shots_avg number of consecutive shots
    new_bufi_ch = np.reshape(new_bufi_ch, (len(new_bufi_ch)//shot_avg, shot_avg))
    new_bufi_ch = np.average(new_bufi_ch, axis=1)
    avgi.append(new_bufi_ch)
avgi = np.array(avgi)
shots = np.array([np.heaviside(avgi[i]/prog.ro_chs[ch].length-threshold[i], 0) for i, ch in enumerate(prog.ro_chs)])

qubits = prog.cfg.expt.qubits
# get the shots for the qubits we care about
shots = np.array([shots[prog.adc_chs[q]] for q in qubits])

# data is returned as n00, n01, n10, n11 measured for the two qubits
n00 = np.sum(np.logical_and(np.logical_not(shots[0]), np.logical_not(shots[1])))
n01 = np.sum(np.logical_and(np.logical_not(shots[0]), shots[1]))
n10 = np.sum(np.logical_and(shots[0], np.logical_not(shots[1])))
n11 = np.sum(np.logical_and(shots[0], shots[1]))
print(n00, n01, n10, n11)

## [Auto2] Optimal amplitude sweep with freq + len calibration

In [ ]:
npts = 12

gain_sweep = np.linspace(12000, 18000, npts) # Q3/Q1
# gain_sweep = np.linspace(5000, 30000, npts) # Q2/Q1
print('gain_sweep\n', gain_sweep)

len_approx_gains = 1.5*np.linspace(0.100, 0.070, npts) # Q3/Q1
# len_approx_gains = 1.5*np.linspace(1.5, 0.300, npts) # Q2/Q1
print('max len for each gain (us)\n', len_approx_gains)

freq_approx_gains = meas.fitting.quadraticfunc(gain_sweep, *p)

print('approx freq for each gain\n', freq_approx_gains)
print('qubits', qubits)

opt_lens = []
opt_freqs = []
saved_files = []
save_files = True

# opt_fit_index = 1 # index of [QA I, QA Q, QB I, QB Q] to use as saved optimal
signs = [1, 1, 1, 1] # whether the period occurs at the max (1) or min (-1)

opt_fit_index = 0 # index of [QA amps, QB amps] to use as saved optimal

In [ ]:
for gain, stop_len, approx_freq in zip(gain_sweep, len_approx_gains, freq_approx_gains):
    # if gain < 27900: continue
    print('gain', gain, 'stop_len', stop_len, 'approx freq', approx_freq)
    rabiEgGfFreqLenChevron = meas.EgGfFreqLenChevronExperiment(
        soccfg=soc,
        path=expt_path,
        prefix=f"rabi_EgGf_freqlen_chevron_qubit{qubits[0]}{qubits[1]}",
        config_file=config_path,
    )

    span_f = 2.5 # MHz
    npts_f = 20
    # npts_f = 60

    start_len = 0
    span_len = stop_len - start_len
    npts_len = 15
    
    rabiEgGfFreqLenChevron.cfg.expt = dict(
        start_f=approx_freq - 0.5*span_f, # start freq [MHz],
        step_f=span_f/npts_f, # freq step, 
        expts_f=npts_f, # number of different freq experiments, 
        start_len=start_len, # pulse length (sigma) [us]
        step_len=span_len/npts_len, # [us]
        expts_len=npts_len+1,
        gain=int(gain),
        reps=800,
        # reps=2000,
        # reps=10,
        pulse_type='flat_top',
        measure_f=None,
        post_process=None,
        init_state=init_state,
        qDrive=qDrive,
        qubits=qubits,
        measure_qubits=measure_qubits,
    )

    success = False
    while not success:
        try:
            rabiEgGfFreqLenChevron.go(analyze=False, display=False, progress=True, save=False)
            success = True
        except Exception:
            print("Pyro traceback:")
            print("".join(Pyro4.util.getPyroTraceback()))

    if save_files: saved_files.append(rabiEgGfFreqLenChevron.save_data())
    data = rabiEgGfFreqLenChevron.analyze(data=None)
    plot_freqs, plot_lens = rabiEgGfFreqLenChevron.display(data=data, fit=True, signs=signs, verbose=False)

    opt_freqs.append(plot_freqs[opt_fit_index])
    opt_lens.append(plot_lens[opt_fit_index])
    print('opt freq', opt_freqs[-1], 'opt len', opt_lens[-1])

In [ ]:
print(saved_files)

In [ ]:
print('gain sweep\n', gain_sweep.tolist())
print('optimal lens\n', opt_lens)
print('optimal freqs\n', opt_freqs)
print('saved files\n', saved_files)

plt.plot(gain_sweep, opt_lens, '.-')
plt.xlabel('Gain')
plt.ylabel('Length (us)')
plt.show()

plt.plot(gain_sweep, opt_freqs, '.-')
plt.xlabel('Gain')
plt.ylabel('Frequecy (MHz)')
plt.show()


Replot data above

In [16]:
signs = [-1, -1, 1, 1] # whether the period occurs at the max (1) or min (-1)
opt_fit_index = 0

In [17]:
# saved_files = [
#    's:\\Connie\\experiments\\qramLL_4QR2\\data\\data_240617\\00165_rabi_EgGf_freqlen_chevron_qubit21.h5',
#    's:\\Connie\\experiments\\qramLL_4QR2\\data\\data_240617\\00166_rabi_EgGf_freqlen_chevron_qubit21.h5',
#    's:\\Connie\\experiments\\qramLL_4QR2\\data\\data_240617\\00167_rabi_EgGf_freqlen_chevron_qubit21.h5',
#    's:\\Connie\\experiments\\qramLL_4QR2\\data\\data_240617\\00168_rabi_EgGf_freqlen_chevron_qubit21.h5',
#    's:\\Connie\\experiments\\qramLL_4QR2\\data\\data_240617\\00169_rabi_EgGf_freqlen_chevron_qubit21.h5',
#    's:\\Connie\\experiments\\qramLL_4QR2\\data\\data_240617\\00170_rabi_EgGf_freqlen_chevron_qubit21.h5',
#    's:\\Connie\\experiments\\qramLL_4QR2\\data\\data_240617\\00171_rabi_EgGf_freqlen_chevron_qubit21.h5',
#    's:\\Connie\\experiments\\qramLL_4QR2\\data\\data_240617\\00172_rabi_EgGf_freqlen_chevron_qubit21.h5',
#    's:\\Connie\\experiments\\qramLL_4QR2\\data\\data_240617\\00173_rabi_EgGf_freqlen_chevron_qubit21.h5',
#    's:\\Connie\\experiments\\qramLL_4QR2\\data\\data_240617\\00174_rabi_EgGf_freqlen_chevron_qubit21.h5',
#    's:\\Connie\\experiments\\qramLL_4QR2\\data\\data_240617\\00175_rabi_EgGf_freqlen_chevron_qubit21.h5',
#    's:\\Connie\\experiments\\qramLL_4QR2\\data\\data_240617\\00176_rabi_EgGf_freqlen_chevron_qubit21.h5',
#    's:\\Connie\\experiments\\qramLL_4QR2\\data\\data_240617\\00186_rabi_EgGf_freqlen_chevron_qubit21.h5',
#    's:\\Connie\\experiments\\qramLL_4QR2\\data\\data_240617\\00187_rabi_EgGf_freqlen_chevron_qubit21.h5',
#    's:\\Connie\\experiments\\qramLL_4QR2\\data\\data_240617\\00188_rabi_EgGf_freqlen_chevron_qubit21.h5',
#    's:\\Connie\\experiments\\qramLL_4QR2\\data\\data_240617\\00189_rabi_EgGf_freqlen_chevron_qubit21.h5',
#    's:\\Connie\\experiments\\qramLL_4QR2\\data\\data_240617\\00190_rabi_EgGf_freqlen_chevron_qubit21.h5',
#    's:\\Connie\\experiments\\qramLL_4QR2\\data\\data_240617\\00191_rabi_EgGf_freqlen_chevron_qubit21.h5',
#    's:\\Connie\\experiments\\qramLL_4QR2\\data\\data_240617\\00192_rabi_EgGf_freqlen_chevron_qubit21.h5',
#    's:\\Connie\\experiments\\qramLL_4QR2\\data\\data_240617\\00193_rabi_EgGf_freqlen_chevron_qubit21.h5',
#    's:\\Connie\\experiments\\qramLL_4QR2\\data\\data_240617\\00194_rabi_EgGf_freqlen_chevron_qubit21.h5',
#    's:\\Connie\\experiments\\qramLL_4QR2\\data\\data_240617\\00195_rabi_EgGf_freqlen_chevron_qubit21.h5',
#    's:\\Connie\\experiments\\qramLL_4QR2\\data\\data_240617\\00196_rabi_EgGf_freqlen_chevron_qubit21.h5',
#    's:\\Connie\\experiments\\qramLL_4QR2\\data\\data_240617\\00197_rabi_EgGf_freqlen_chevron_qubit21.h5',
#    's:\\Connie\\experiments\\qramLL_4QR2\\data\\data_240617\\00198_rabi_EgGf_freqlen_chevron_qubit21.h5',
# ]

gain_sweep = [0]*len(saved_files)
opt_freqs = [0]*len(saved_files)
opt_lens = [0]*len(saved_files)

In [ ]:
# going through files again to get opt lens, freqs
for i_file, saved_file in enumerate(saved_files):
    file_name = saved_file.split('\\')[-1]
    print(file_name)

    rabiEgGfLenChevron = meas.EgGfFreqLenChevronExperiment(config_file=config_path)
    temp_data, attrs = prev_data(expt_path, file_name)
    rabiEgGfLenChevron.cfg.expt = AttrDict(attrs['config']['expt'])
    # print(temp_data.keys())

    # if file_name != '00013_rabi_EgGf_freqlen_chevron_qubit21.h5': continue
    fitparams=[None, 1/temp_data['lenpts'][-1], None, None]

    data = rabiEgGfFreqLenChevron.analyze(data=temp_data, fitparams=fitparams)
    plot_freqs, plot_lens = rabiEgGfFreqLenChevron.display(data=data, fit=True, signs=signs)

    print('gain', rabiEgGfLenChevron.cfg.expt.gain)
    print('plot freq', plot_freqs[opt_fit_index])
    print('plot len', plot_lens[opt_fit_index])
    gain_sweep[i_file] = rabiEgGfLenChevron.cfg.expt.gain
    opt_freqs[i_file] = plot_freqs[opt_fit_index]
    opt_lens[i_file] = plot_lens[opt_fit_index]

In [ ]:
print('gain sweep\n', gain_sweep)
print('optimal lens\n', opt_lens)
print('optimal freqs\n', opt_freqs)
print('saved files\n', saved_files)

print(len(gain_sweep))
print(len(opt_lens))
print(len(opt_freqs))

plt.plot(gain_sweep, opt_lens, '.-')
plt.xlabel('Gain')
plt.ylabel('Length (us)')
plt.show()

plt.plot(gain_sweep, opt_freqs, '.-')
plt.xlabel('Gain')
plt.ylabel('Frequecy (MHz)')
plt.show()


### Alternative: fine gain vs freq sweep setup

In [ ]:
npts = 15

gain_sweep = np.linspace(14000, 18500, npts) # Q3/Q1
# gain_sweep = np.linspace(5000, 30000, npts) # Q2/Q1
print('gain_sweep\n', gain_sweep)

len_approx_gains = 1.5*np.linspace(0.350, 0.250, npts) # Q3/Q1
# len_approx_gains = 1.5*np.linspace(1.00, 0.300, npts) # Q2/Q1
print('max len for each gain (us)\n', len_approx_gains)

freq_approx_gains = np.linspace(5148.0, 5145.0, npts) # approx freq for each gain # Q3/Q1
# freq_approx_gains = np.linspace(5887.0, 5883, npts) # approx freq for each gain # Q2/Q1

print('approx freq for each gain\n', freq_approx_gains)
print('qubits', qubits)

opt_lens = len_approx_gains/1.5
opt_freqs = []
saved_files = []
save_files = True

In [128]:
len_approx_gains = 1.5*np.array(opt_lens)
freq_approx_gains = np.array(opt_freqs)
saved_files = []

In [ ]:
for gain, stop_len, approx_freq in zip(gain_sweep, len_approx_gains, freq_approx_gains):
    # if gain < 27900: continue
    print('gain', int(gain), 'stop_len', stop_len, 'approx freq', approx_freq)
    rabiEgGfFreqLenChevron = meas.EgGfFreqLenChevronExperiment(
        soccfg=soc,
        path=expt_path,
        prefix=f"rabi_EgGf_freqlen_chevron_qubit{qubits[0]}{qubits[1]}",
        config_file=config_path,
    )

    span_f = 2.5 # MHz
    npts_f = 20
    # npts_f = 60

    # start_len = stop_len/1.5
    # span_len = 0
    # expts_len = 1

    start_len = 0
    span_len = stop_len
    expts_len = 15

    rabiEgGfFreqLenChevron.cfg.expt = dict(
        start_f=approx_freq - 0.5*span_f, # start freq [MHz],
        step_f=span_f/npts_f, # freq step, 
        expts_f=npts_f+1, # number of different freq experiments, 
        start_len=start_len, # pulse length (sigma) [us]
        step_len=span_len/npts_len, # [us]
        expts_len=expts_len,
        gain=int(gain),
        reps=1000,
        # reps=2000,
        # reps=10,
        pulse_type='flat_top',
        measure_f=None,
        post_process=None,
        init_state=init_state,
        qDrive=qDrive,
        qubits=qubits,
        measure_qubits=measure_qubits,

        full_mux_expt=full_mux_expt,
        resonator_reset=[0, 2, 3],
    )

    success = False
    while not success:
        try:
            rabiEgGfFreqLenChevron.go(analyze=False, display=False, progress=True, save=False)
            success = True
        except Exception:
            print("Pyro traceback:")
            print("".join(Pyro4.util.getPyroTraceback()))

    if save_files: saved_files.append(rabiEgGfFreqLenChevron.save_data())
    
    plt.figure(figsize=(9, 8))
    data = rabiEgGfFreqLenChevron.data
    plot_len = rabiEgGfFreqLenChevron.cfg.device.qubit.pulses.pi_EgGf_Q.sigma[qubits[0]]
    # plot_len = rabiEgGfFreqLenChevron.cfg.device.qubit.pulses.pi_EgGf.sigma[qubits[0]]
    len_ind = np.argmin(np.abs(data['lenpts']-plot_len))
    
    amps_freq_cut_qA = data['amps'][0][:, len_ind]
    amps_freq_cut_qB = data['amps'][1][:, len_ind]
    p_ampsA, pCov_ampsA = meas.fitting.fitquadratic(data['freqpts'], amps_freq_cut_qA)
    p_ampsB, pCov_ampsB = meas.fitting.fitquadratic(data['freqpts'], amps_freq_cut_qB)
    fit_amps_freq_cut_qA = meas.fitting.quadraticfunc(data['freqpts'], *p_ampsA)
    fit_amps_freq_cut_qB = meas.fitting.quadraticfunc(data['freqpts'], *p_ampsB)
    freq_max_amp_qA = data['freqpts'][np.argmax(fit_amps_freq_cut_qA)]
    freq_min_amp_qB = data['freqpts'][np.argmin(fit_amps_freq_cut_qB)]
    print('QA freq with max amp', freq_max_amp_qA)
    print('QB freq with min amp', freq_min_amp_qB)
    plt.suptitle(f"Cut through length {data['lenpts'][len_ind]:.3} us")
    plt.subplot(211, title=f'QA ({qubits[0]})')
    plt.plot(data['freqpts'], amps_freq_cut_qA, '.-', label='qA', color=default_colors[0])
    plt.plot(data['freqpts'], fit_amps_freq_cut_qA, color=default_colors[0])
    plt.axvline(freq_max_amp_qA, color=default_colors[0], linestyle='--')
    plt.ylabel('Amplitude [ADC units]')
    
    plt.subplot(212, title=f'QB ({qubits[1]})')
    plt.plot(data['freqpts'], amps_freq_cut_qB, '.-', label='qB', color=default_colors[1])
    plt.plot(data['freqpts'], meas.fitting.quadraticfunc(data['freqpts'], *p_ampsB), color=default_colors[1])
    plt.axvline(freq_min_amp_qB, color=default_colors[1], linestyle='--')
    # plt.axvline(5890.45, color='r', linestyle='--')
    # plt.axvline(bad_ZZ_freq, color='r', linestyle='--')
    plt.xlabel('Frequency [MHz]')
    plt.ylabel('Amplitude [ADC units]')
    plt.tight_layout()
    plt.show()

    opt_freqs.append((freq_max_amp_qA + freq_max_amp_qB)/2)
    print('opt freq', opt_freqs[-1])

In [ ]:
print(saved_files)

In [ ]:
print('gain sweep\n', gain_sweep.tolist())
print('optimal freqs\n', opt_freqs)
print('saved files\n', saved_files)

plt.plot(gain_sweep, opt_freqs, '.-')
plt.xlabel('Gain')
plt.ylabel('Frequecy (MHz)')
plt.show()


In [65]:
['S:\\Connie\\experiments\\qramLL_4QR2\\data\\data_240617\\00086_rabi_EgGf_freqlen_chevron_qubit31.h5', 'S:\\Connie\\experiments\\qramLL_4QR2\\data\\data_240617\\00087_rabi_EgGf_freqlen_chevron_qubit31.h5', 'S:\\Connie\\experiments\\qramLL_4QR2\\data\\data_240617\\00088_rabi_EgGf_freqlen_chevron_qubit31.h5', 'S:\\Connie\\experiments\\qramLL_4QR2\\data\\data_240617\\00089_rabi_EgGf_freqlen_chevron_qubit31.h5', 'S:\\Connie\\experiments\\qramLL_4QR2\\data\\data_240617\\00090_rabi_EgGf_freqlen_chevron_qubit31.h5', 'S:\\Connie\\experiments\\qramLL_4QR2\\data\\data_240617\\00091_rabi_EgGf_freqlen_chevron_qubit31.h5', 'S:\\Connie\\experiments\\qramLL_4QR2\\data\\data_240617\\00092_rabi_EgGf_freqlen_chevron_qubit31.h5', 'S:\\Connie\\experiments\\qramLL_4QR2\\data\\data_240617\\00093_rabi_EgGf_freqlen_chevron_qubit31.h5', 'S:\\Connie\\experiments\\qramLL_4QR2\\data\\data_240617\\00094_rabi_EgGf_freqlen_chevron_qubit31.h5', 'S:\\Connie\\experiments\\qramLL_4QR2\\data\\data_240617\\00095_rabi_EgGf_freqlen_chevron_qubit31.h5', 'S:\\Connie\\experiments\\qramLL_4QR2\\data\\data_240617\\00096_rabi_EgGf_freqlen_chevron_qubit31.h5', 'S:\\Connie\\experiments\\qramLL_4QR2\\data\\data_240617\\00097_rabi_EgGf_freqlen_chevron_qubit31.h5', 'S:\\Connie\\experiments\\qramLL_4QR2\\data\\data_240617\\00098_rabi_EgGf_freqlen_chevron_qubit31.h5']
gain_sweep = [0]*len(saved_files)
opt_freqs = [0]*len(saved_files)

In [ ]:
# going through files again to get opt lens, freqs
for i_file, saved_file in enumerate(saved_files):
    file_name = saved_file.split('\\')[-1]
    print(file_name)

    rabiEgGfLenChevron = meas.EgGfFreqLenChevronExperiment(config_file=config_path)
    temp_data, attrs = prev_data(expt_path, file_name)
    rabiEgGfLenChevron.cfg.expt = AttrDict(attrs['config']['expt'])
    # print(temp_data.keys())
    
    plt.figure(figsize=(9, 8))
    data = temp_data
    plot_len = rabiEgGfFreqLenChevron.cfg.device.qubit.pulses.pi_EgGf_Q.sigma[qubits[0]]
    # plot_len = rabiEgGfFreqLenChevron.cfg.device.qubit.pulses.pi_EgGf.sigma[qubits[0]]
    len_ind = np.argmin(np.abs(data['lenpts']-plot_len))
    
    amps_freq_cut_qA = data['amps'][0][:, len_ind]
    amps_freq_cut_qB = data['amps'][1][:, len_ind]
    p_ampsA, pCov_ampsA = meas.fitting.fitquadratic(data['freqpts'], amps_freq_cut_qA)
    p_ampsB, pCov_ampsB = meas.fitting.fitquadratic(data['freqpts'], amps_freq_cut_qB)
    fit_amps_freq_cut_qA = meas.fitting.quadraticfunc(data['freqpts'], *p_ampsA)
    fit_amps_freq_cut_qB = meas.fitting.quadraticfunc(data['freqpts'], *p_ampsB)
    freq_max_amp_qA = data['freqpts'][np.argmax(fit_amps_freq_cut_qA)]
    freq_min_amp_qB = data['freqpts'][np.argmin(fit_amps_freq_cut_qB)]
    print('QA freq with max amp', freq_max_amp_qA)
    print('QB freq with min amp', freq_min_amp_qB)
    plt.suptitle(f"Cut through length {data['lenpts'][len_ind]:.3} us")
    plt.subplot(211, title=f'QA ({qubits[0]})')
    plt.plot(data['freqpts'], amps_freq_cut_qA, '.-', label='qA', color=default_colors[0])
    plt.plot(data['freqpts'], fit_amps_freq_cut_qA, color=default_colors[0])
    plt.axvline(freq_max_amp_qA, color=default_colors[0], linestyle='--')
    plt.ylabel('Amplitude [ADC units]')
    
    plt.subplot(212, title=f'QB ({qubits[1]})')
    plt.plot(data['freqpts'], amps_freq_cut_qB, '.-', label='qB', color=default_colors[1])
    plt.plot(data['freqpts'], meas.fitting.quadraticfunc(data['freqpts'], *p_ampsB), color=default_colors[1])
    plt.axvline(freq_min_amp_qB, color=default_colors[1], linestyle='--')
    # plt.axvline(5890.45, color='r', linestyle='--')
    # plt.axvline(bad_ZZ_freq, color='r', linestyle='--')
    plt.xlabel('Frequency [MHz]')
    plt.ylabel('Amplitude [ADC units]')
    plt.tight_layout()
    plt.show()

    print('gain', rabiEgGfLenChevron.cfg.expt.gain)
    print('plot freq', plot_freqs[opt_fit_index])
    
    gain_sweep[i_file] = rabiEgGfLenChevron.cfg.expt.gain
    opt_freqs[i_file] = (freq_max_amp_qA + freq_min_amp_qB)/2

In [ ]:
print('gain sweep\n', gain_sweep)
print('optimal freqs\n', opt_freqs)
print('saved files\n', saved_files)

print(len(gain_sweep))
print(len(opt_freqs))

plt.plot(gain_sweep, opt_freqs, '.-')
plt.xlabel('Gain')
plt.ylabel('Frequecy (MHz)')
plt.show()


## [Auto3] Run again with length rabi and error amplification on selected frequencies

### Functions for fine calibration experiments

In [11]:
def set_yaml_params(sweep2Q_q, yaml_cfg, opt_gain, opt_len, opt_freq, test_pi_half):
    new_cfg = deepcopy(yaml_cfg)
    if not test_pi_half:
        new_cfg.device.qubit.pulses.pi_EgGf_Q.sigma[sweep2Q_q] = opt_len
        new_cfg.device.qubit.f_EgGf_Q[sweep2Q_q] = opt_freq
        new_cfg.device.qubit.pulses.pi_EgGf_Q.gain[sweep2Q_q] = opt_gain
    else:
        new_cfg.device.qubit.pulses.pi_EgGf_Q.half_sigma[sweep2Q_q] = opt_len
        new_cfg.device.qubit.f_EgGf_Q_half[sweep2Q_q] = opt_freq
        new_cfg.device.qubit.pulses.pi_EgGf_Q.half_gain[sweep2Q_q] = opt_gain
    return new_cfg

In [12]:
def run_swap_length_rabi(sweep2Q_q, yaml_cfg, opt_gain, opt_len, opt_freq, test_pi_half=False, save_data=True, saved_files=None, good_data=False):
    print(f"-------- swap length rabi --------")
    lengthrabiEgGf = meas.LengthRabiEgGfExperiment(
        soccfg=soc,
        path=expt_path,
        prefix=f"length_rabi_EgGf_qubit{qubits[0]}{qubits[1]}",
        config_file=config_path,
    )
    
    lengthrabiEgGf.cfg = AttrDict(set_yaml_params(sweep2Q_q, yaml_cfg, opt_gain, opt_len, opt_freq, test_pi_half))
    
    stop_len = opt_len*2
    start_len = 0
    span_len = stop_len - start_len
    npts_len = 101
    # npts_len = 50

    sweep2Q_q = qubits[0]

    lengthrabiEgGf.cfg.expt = dict(
        start=start_len, # pulse length (sigma) [us]
        step=span_len/npts_len, # [us]
        expts=npts_len,
        # reps=60,
        # loops=10,
        reps=1000 if not good_data else 3000,
        # loops=3,
        # post_process='threshold',
        post_process=None,
        singleshot_reps=20000,
        measure_f=None,
        init_state=init_state,
        qDrive=qDrive,
        qubits=qubits,
        measure_qubits=measure_qubits,

        test_pi_half=test_pi_half,

        full_mux_expt=full_mux_expt,
        resonator_reset=resonator_reset,
    )

    try:
        lengthrabiEgGf.acquire(progress=True, debug=True)
        success = True
    except Exception:
        print("Pyro traceback:")
        print("".join(Pyro4.util.getPyroTraceback()))

    if save_data:
        assert saved_files is not None
        saved_files.append(lengthrabiEgGf.save_data())
        print('Saved', saved_files[-1])
    data = lengthrabiEgGf.analyze(data=None, fit=True)
    p, pcov = meas.fitting.get_best_fit(data, prefixes=['fitA', 'fitB'], check_measures=['amps'])
    if p[2] > 180:
        p[2] = p[2] - 360
    elif p[2] < -180:
        p[2] = p[2] + 360
    if p[2] < 0:
        pi_len = (1 / 2 - p[2] / 180) / 2 / p[1]
    else:
        pi_len = (3 / 2 - p[2] / 180) / 2 / p[1]

    lengthrabiEgGf.display(data=data, fit=True)
    # new_opt_len = pi_lens[opt_fit_index]
    new_opt_len = pi_len

    return new_opt_len

In [13]:
def run_swap_piminuspi(sweep2Q_q, yaml_cfg, opt_gain, opt_len, opt_freq, span_f=2.0, test_pi_half=False, save_data=True, saved_files=None, good_data=False):
    print(f"-------- swap pi minus pi --------")
    piminuspi = meas.PiMinusPiEgGfExperiment(
        soccfg=soc,
        path=expt_path,
        prefix=f"PiMinusPiEgGfExptQ{qubits[0]}{qubits[1]}",
        config_file=config_path,
    )

    piminuspi.cfg = AttrDict(set_yaml_params(sweep2Q_q, yaml_cfg, opt_gain, opt_len, opt_freq, test_pi_half))

    # npts_f = 70
    npts_f = 30 if not good_data else 75

    piminuspi.cfg.expt = dict(
        start_N=0, # number gate repetitions
        # step_N=2,
        step_N=1,
        expts_N=10 if not good_data else 20,
        start_f=opt_freq-span_f/2,
        step_f=span_f/npts_f,
        expts_f=npts_f+1,

        reps=700 if not good_data else 6000,
        loops=1,

        pulse_type='flat_top',
        post_process='scale',
        singleshot_reps=15000,
        # singleshot_reps=5000,
        measure_f=False,
        init_state=init_state,
        qDrive=qDrive,
        qubits=qubits,
        measure_qubits=measure_qubits,

        test_pi_half=test_pi_half,

        readout_cool=False,
        n_init_readout=1,
        n_trig=1,
        # init_read_wait_us=5,
        init_read_wait_us=2,
        
        full_mux_expt=full_mux_expt,
        resonator_reset=resonator_reset,
    )
    # piminuspi.cfg.device.readout.gain[1] = 1e-4

    print(piminuspi.cfg)
    try:
        piminuspi.acquire(progress=True, debug=False)
    except Exception:
        print("Pyro traceback:")
        print("".join(Pyro4.util.getPyroTraceback()))

    data = piminuspi.data
    piminuspi.analyze(fit=True)
    piminuspi.display(fit=True)

    if save_data:
        assert saved_files is not None
        saved_files.append(piminuspi.save_data())
        print('Saved', saved_files[-1])

    return data['best_freq']

In [14]:
def run_swap_error_amp(sweep2Q_q, yaml_cfg, opt_gain, opt_len, opt_freq, test_pi_half=False, save_data=True, saved_files=None, good_data=False):
    print(f"-------- swap error amplification --------")
    npulsecalib = meas.NPulseEgGfExperiment(
        soccfg=soc,
        path=expt_path,
        prefix=f"NPulseExptEgGfQ{qubits[0]}{qubits[1]}",
        config_file=config_path,
    )

    npulsecalib.cfg = AttrDict(set_yaml_params(sweep2Q_q, yaml_cfg, opt_gain, opt_len, opt_freq, test_pi_half))

    npulsecalib.cfg.expt = dict(
        start=0, # number gate repetitions
        step=1,
        expts=10 if not good_data else 15,
        reps=2000,
        loops=10,
        # reps=1000,
        # loops=2,

        pulse_type='flat_top',
        # post_process='threshold',
        post_process='scale',
        singleshot_reps=15000,
        measure_f=False,
        init_state=init_state,
        qDrive=qDrive,
        qubits=qubits,
        measure_qubits=measure_qubits,

        test_pi_half=test_pi_half,

        full_mux_expt=full_mux_expt,
        resonator_reset=resonator_reset,
    )

    print(npulsecalib.cfg)
    try:
        npulsecalib.acquire(progress=True, debug=False)
    except Exception:
        print("Pyro traceback:")
        print("".join(Pyro4.util.getPyroTraceback()))
    if save_data:
        assert saved_files is not None
        saved_files.append(npulsecalib.save_data())
        print('Saved', saved_files[-1])

    data = npulsecalib.data
    npulsecalib.analyze(fit=True)
    npulsecalib.display(fit=True)

    npulsecalib_fit, npulsecalib_fit_err = meas.fitting.get_best_fit(npulsecalib.data, prefixes=['fitA', 'fitB'], check_measures=['amps'])

    if np.all(npulsecalib_fit == data['fitA_avgi']): use_q =  npulsecalib.cfg.expt.measure_qubits[0]
    else: use_q = npulsecalib.cfg.expt.measure_qubits[1]
    if use_q == qDrive: sign = 1
    else: sign = -1
    angle_err = npulsecalib_fit[1]
    amp_ratio = (180 - sign*angle_err)/180
    new_opt_len = npulsecalib.cfg.device.qubit.pulses.pi_EgGf_Q.sigma[sweep2Q_q] / amp_ratio

    print('pi len precise', new_opt_len, f'adjustment ratio {amp_ratio} from Q{use_q}')
    return new_opt_len

In [15]:
def run_swap_pi_train(sweep2Q_q, yaml_cfg, opt_gain, opt_len, opt_freq, test_pi_half=False, save_data=True, saved_files=None, good_data=False):
    print(f"-------- swap pi train --------")
    train_expt = meas.PiTrainEgGfGainSweepExperiment(
        soccfg=soc,
        path=expt_path,
        prefix=f"eggf_train_Q{qubits[0]}{qubits[1]}",
        config_file=config_path,
    )
    train_expt.cfg = AttrDict(set_yaml_params(sweep2Q_q, yaml_cfg, opt_gain, opt_len, opt_freq, test_pi_half))

    span_gain = 1000
    npts_gain = 25
    start_gain = opt_gain - span_gain/2
    step_gain = span_gain/npts_gain
    expts_gain = npts_gain + 1

    start_N = 4 # this is the number of pi swap cycles
    step_N = 2
    expts_N = 5

    train_expt.cfg.expt = dict(
        start_gain=start_gain,
        step_gain=step_gain,
        expts_gain=expts_gain,
        start_N=start_N,
        step_N=step_N,
        expts_N=expts_N,
    
        reps=400,
        loops=1,
    
        pulse_type='flat_top',
        post_process='threshold',
        singleshot_reps=10000,
        measure_f=False,
        init_state=init_state,
        qDrive=qDrive,
        qubits=qubits,
        measure_qubits=measure_qubits,
    
        test_pi_half=test_pi_half,
    
        full_mux_expt=full_mux_expt,
        resonator_reset=resonator_reset,
    
        use_robust_pulses=use_robust_pulses,
    )

    train_expt.acquire(progress=True, debug=False)
    train_expt.analyze(fit=True)
    train_expt.display(fit=True)
    
    if save_data:
        assert saved_files is not None
        saved_files.append(train_expt.save_data())
        print('Saved', saved_files[-1])

    new_opt_gain = train_expt.data["best_gain"]
    print('pi gain precise', new_opt_gain)
    return new_opt_gain
    

#### V2 fine calibration experiments

In [46]:
def run_amp_rabi_chevron(sweep2Q_q, yaml_cfg, opt_gain, opt_len, opt_freq, test_pi_half=False, span_gain=7000, span_f=20.0, save_data=True, saved_files=None, good_data=False):
    print(f"-------- amp rabi chevron --------")
    chevron = meas.EgGfFreqGainChevronExperiment(
        soccfg=soc,
        path=expt_path,
        prefix=f"rabi_EgGf_freqgain_chevron_qubit{qubits[0]}{qubits[1]}",
        config_file=config_path,
    )
    chevron.cfg = AttrDict(set_yaml_params(sweep2Q_q, yaml_cfg, opt_gain, opt_len, opt_freq, test_pi_half))
    print("sweeping near freq", opt_freq, "gain", opt_gain, "at fixed len", opt_len)

    start_gain = int(np.max((0, opt_gain - span_gain / 2)))
    npts_gain = 15

    start_f = opt_freq - span_f / 2
    npts_f = 40

    chevron.cfg.expt = dict(
        start_gain=start_gain, # qubit gain [dac level]
        step_gain=int(span_gain/npts_gain), # [dac level]
        expts_gain=npts_gain,
        start_f=start_f,
        step_f=span_f/npts_f, # freq step, 
        expts_f=npts_f, # number of different freq experiments, 
        reps=500,
        rounds=1,
        pi_EgGf_sigma=opt_len, # gaussian sigma/const pulse length, overrides config [us]
        pulse_type='flat_top',
        setup_ZZ=setup_ZZ,
        qDrive=qDrive,
        qubits=qubits,
        measure_qubits=measure_qubits,

        test_pi_half=test_pi_half,

        full_mux_expt=full_mux_expt,
        resonator_reset=resonator_reset,
        use_robust_pulses=use_robust_pulses,
    )
    chevron.acquire(progress=True)

    if save_data:
        assert saved_files is not None
        saved_files.append(chevron.save_data())
        print('Saved', saved_files[-1])

    data = chevron.analyze(data=None, fit=True)
    chevron.display(fit=True, data=data)

    best_freq = data["best_freq"]
    best_gain = data["best_gain"]
    
    return best_freq, best_gain

In [53]:
def run_swap_piminuspi_gain_sweep(sweep2Q_q, yaml_cfg, opt_gain, opt_len, opt_freq, span_f=2.5, test_pi_half=False, save_data=True, saved_files=None, good_data=False):
    print(f"-------- pi minus pi gain vs freq sweep --------")
    """
    Repeat pi/-pi sweeping both gain and freq, collapse the different N into 1 curve per gain/freq combo
    """
    sweep_expt = Experiment(
        soccfg=soc,
        path=expt_path,
        prefix=f"eggf_piminuspi_gain_sweep_Q{qubits[0]}{qubits[1]}",
        config_file=config_path,
    )
    sweep_expt.cfg = AttrDict(set_yaml_params(sweep2Q_q, yaml_cfg, opt_gain, opt_len, opt_freq, test_pi_half))
    print("sweeping near freq", opt_freq, "gain", opt_gain, "at fixed len", opt_len)

    span_gain = 0.2*opt_gain
    npts_gain = 15
    start_gain = opt_gain - span_gain/2
    step_gain = span_gain/npts_gain
    expts_gain = npts_gain + 1
    gain_sweep = start_gain + step_gain * np.arange(expts_gain)

    start_N = 1
    step_N = 2 if not good_data else 1
    expts_N = 5 if not good_data else 10
    cycle_sweep = start_N + step_N * np.arange(expts_N)

    npts_f = 30
    start_f = opt_freq - span_f/2
    step_f = span_f/npts_f
    expts_f = npts_f + 1

    freq_sweep = start_f + step_f * np.arange(expts_f)

    print('gain_sweep', gain_sweep.tolist())
    print('cycle_sweep', cycle_sweep.tolist())
    print('freq_sweep', freq_sweep.tolist())

    sweep_expt.data = dict(
        avgi = np.zeros((2, len(gain_sweep), len(cycle_sweep), len(freq_sweep))),
        avgq = np.zeros((2, len(gain_sweep), len(cycle_sweep), len(freq_sweep))),
        amps = np.zeros((2, len(gain_sweep), len(cycle_sweep), len(freq_sweep))),
        products = np.zeros((2, len(gain_sweep), len(freq_sweep))),
        cycle_sweep = cycle_sweep,
        freq_sweep = freq_sweep,
        gain_sweep = gain_sweep,
    )

    angles = thresholds = ge_avgs = counts_calib = None
    

    for i_gain, gain in enumerate(gain_sweep):
        piminuspi = meas.PiMinusPiEgGfExperiment(
            soccfg=soc,
            path=expt_path,
            prefix=f"PiMinusPiEgGfExptQ{qubits[0]}{qubits[1]}",
            config_file=config_path,
        )

        piminuspi.cfg = AttrDict(deepcopy(sweep_expt.cfg))

        if not test_pi_half:
            piminuspi.cfg.device.qubit.pulses.pi_EgGf_Q.gain[sweep2Q_q] = int(gain)
        else:
            piminuspi.cfg.device.qubit.pulses.pi_EgGf_Q.half_gain[sweep2Q_q] = int(gain)

        piminuspi.cfg.expt = dict(
            start_N=start_N, # number gate repetitions
            step_N=step_N,
            expts_N=expts_N,
            start_f=start_f,
            step_f=step_f,
            expts_f=expts_f,

            reps=800 if not good_data else 3000,
            loops=1,

            pulse_type='flat_top',
            post_process='scale',
            singleshot_reps=10000,
            angles=angles,
            thresholds=thresholds,
            ge_avgs=ge_avgs,
            counts_calib=counts_calib,

            measure_f=False,
            init_state=init_state,
            qDrive=qDrive,
            qubits=qubits,
            measure_qubits=measure_qubits,

            test_pi_half=test_pi_half,

            readout_cool=False,
            n_init_readout=1,
            n_trig=1,
            init_read_wait_us=2,
            
            full_mux_expt=full_mux_expt,
            resonator_reset=resonator_reset,
        )

        try:
            piminuspi.acquire(progress=True, debug=False)
        except Exception:
            print("Pyro traceback:")
            print("".join(Pyro4.util.getPyroTraceback()))

        data = piminuspi.data
        piminuspi.analyze(fit=True, verbose=False)
        # piminuspi.display(fit=True)

        for data_name in ['avgi', 'avgq', 'amps']:
            sweep_expt.data[data_name][:, i_gain, :, :] = piminuspi.data[data_name]
        sweep_expt.data["products"][:, i_gain, :] = piminuspi.data["products"]

        angles = data["angles"]
        thresholds = data["thresholds"]
        ge_avgs = data["ge_avgs"]
        counts_calib = data["counts_calib"]

    if save_data:
        assert saved_files is not None
        saved_files.append(sweep_expt.save_data())
        print('Saved', saved_files[-1])


    data = sweep_expt.data
    qA, qB = qubits
    label = "($X_{\pi}, X_{-\pi})^N$"
    if test_pi_half:
        # label = "($X_{\pi/2}, X_{-\pi/2})^N$"
        label = "($X_{\pi/2}, X_{\pi/2}, X_{-\pi/2}, X_{-\pi/2})^N$"
    title = (
        f"Q{qA}/Q{qB} Product of Ns for {label}\n Len {opt_len:.3f}, Ns={cycle_sweep.tolist()}"
    )

    inner_sweep = data["freq_sweep"]
    outer_sweep = data["gain_sweep"]

    y_sweep = outer_sweep
    x_sweep = inner_sweep

    plt.figure(figsize=(8, 9))
    plt.suptitle(title, fontsize=20)
    data_name = "products"

    ax_qA = plt.subplot(211, title=f"QA ({qA})")
    ax_qA.set_ylabel(f"Gain [DAC units]", fontsize=18)
    ax_qA.tick_params(axis="both", which="major", labelsize=16)
    plot_data = data[data_name][0]
    plt.pcolormesh(x_sweep, y_sweep, plot_data, cmap="viridis", shading="auto")
    cbar = plt.colorbar()
    cbar.ax.tick_params(labelsize=18)

    ax_qB = plt.subplot(212, title=f"QB ({qB})")
    ax_qB.set_ylabel(f"Gain [DAC units]", fontsize=18)
    ax_qB.set_xlabel("Frequency [MHz]", fontsize=18)
    ax_qB.tick_params(axis="both", which="major", labelsize=16)
    plot_data = data[data_name][1]
    plt.pcolormesh(x_sweep, y_sweep, plot_data, cmap="viridis", shading="auto")
    cbar = plt.colorbar()
    cbar.ax.tick_params(labelsize=18)

    plt.tight_layout()
    plt.show()

    return sweep_expt.data
    

In [54]:
def run_swap_pipi_gain_sweep(sweep2Q_q, yaml_cfg, opt_gain, opt_len, opt_freq, span_f=2.0, test_pi_half=False, save_data=True, saved_files=None, good_data=False):
    print(f"-------- pi pi gain vs freq sweep --------")
    sweep_expt = Experiment(
        soccfg=soc,
        path=expt_path,
        prefix=f"eggf_pipi_gain_sweep_Q{qubits[0]}{qubits[1]}",
        config_file=config_path,
    )
    sweep_expt.cfg = AttrDict(set_yaml_params(sweep2Q_q, yaml_cfg, opt_gain, opt_len, opt_freq, test_pi_half))
    print("sweeping near freq", opt_freq, "gain", opt_gain, "at fixed len", opt_len)

    span_gain = 0.2*opt_gain
    npts_gain = 15
    start_gain = opt_gain - span_gain/2
    step_gain = span_gain/npts_gain
    expts_gain = npts_gain + 1
    gain_sweep = start_gain + step_gain * np.arange(expts_gain)

    start_N = 5
    step_N = 5 if not good_data else 1
    expts_N = 4 if not good_data else 10
    cycle_sweep = start_N + step_N * np.arange(expts_N)

    npts_f = 30
    start_f = opt_freq - span_f/2
    step_f = span_f/npts_f
    expts_f = npts_f + 1


    freq_sweep = start_f + step_f * np.arange(expts_f)
    
    print('gain_sweep', gain_sweep.tolist())
    print('cycle_sweep', cycle_sweep.tolist())
    print('freq_sweep', freq_sweep.tolist())
    
    sweep_expt.data = dict(
        avgi = np.zeros((2, len(gain_sweep), len(cycle_sweep), len(freq_sweep))),
        avgq = np.zeros((2, len(gain_sweep), len(cycle_sweep), len(freq_sweep))),
        amps = np.zeros((2, len(gain_sweep), len(cycle_sweep), len(freq_sweep))),
        products = np.zeros((2, len(gain_sweep), len(freq_sweep))),
        cycle_sweep = cycle_sweep,
        freq_sweep = freq_sweep,
        gain_sweep = gain_sweep,
    )
    
    angles = thresholds = ge_avgs = counts_calib = None
        
    
    for i_freq, freq in enumerate(tqdm(freq_sweep)):
        pipi = meas.PiTrainEgGfGainSweepExperiment(
            soccfg=soc,
            path=expt_path,
            prefix=f"PiTrainEgGfGainSweepExptQ{qubits[0]}{qubits[1]}",
            config_file=config_path,
        )
    
        pipi.cfg = AttrDict(deepcopy(sweep_expt.cfg))
    
        if not test_pi_half:
            pipi.cfg.device.qubit.f_EgGf_Q[sweep2Q_q] = freq
        else:
            pipi.cfg.device.qubit.f_EgGf_Q_half[sweep2Q_q] = freq
    
        pipi.cfg.expt = dict(
            start_gain=start_gain,
            step_gain=step_gain,
            expts_gain=expts_gain,
            start_N=start_N,
            step_N=step_N,
            expts_N=expts_N,
    
            reps=500 if not good_data else 1000,
            loops=1,
    
            pulse_type='flat_top',
            post_process='scale',
            singleshot_reps=10000,
            angles=angles,
            thresholds=thresholds,
            ge_avgs=ge_avgs,
            counts_calib=counts_calib,
    
            measure_f=False,
            init_state=init_state,
            qDrive=qDrive,
            qubits=qubits,
            measure_qubits=measure_qubits,
    
            test_pi_half=test_pi_half,
    
            readout_cool=False,
            n_init_readout=1,
            n_trig=1,
            init_read_wait_us=2,
            
            full_mux_expt=full_mux_expt,
            resonator_reset=resonator_reset,
        )
    
        try:
            pipi.acquire(progress=True, debug=False)
        except Exception:
            print("Pyro traceback:")
            print("".join(Pyro4.util.getPyroTraceback()))
    
        data = pipi.data
        pipi.analyze(fit=False)
        # pipi.display(fit=False)
    
        for data_name in ['avgi', 'avgq', 'amps']:
            sweep_expt.data[data_name][:, :, :, i_freq] = np.swapaxes(pipi.data[data_name], 1, 2)
        sweep_expt.data["products"][:, :, i_freq] = pipi.data["products"]
    
        angles = data["angles"]
        thresholds = data["thresholds"]
        ge_avgs = data["ge_avgs"]
        counts_calib = data["counts_calib"]

    if save_data:
        assert saved_files is not None
        saved_files.append(sweep_expt.save_data())
        print('Saved', saved_files[-1])

    data = sweep_expt.data
    qA, qB = qubits
    label = "($X_{\pi}, X_{-\pi})^N$"
    if test_pi_half:
        label = "($X_{\pi/2}, X_{\pi/2})^{2N}$"
    title = (
        f"Q{qA}/Q{qB} Product of Ns for {label}\n Len {opt_len:.3f}, Ns={cycle_sweep.tolist()}"
    )

    inner_sweep = data["freq_sweep"]
    outer_sweep = data["gain_sweep"]

    y_sweep = outer_sweep
    x_sweep = inner_sweep

    plt.figure(figsize=(8, 9))
    plt.suptitle(title, fontsize=20)
    data_name = "products"

    ax_qA = plt.subplot(211, title=f"QA ({qA})")
    ax_qA.set_ylabel(f"Gain [DAC units]", fontsize=18)
    ax_qA.tick_params(axis="both", which="major", labelsize=16)
    plot_data = data[data_name][0]
    plt.pcolormesh(x_sweep, y_sweep, plot_data, cmap="viridis", shading="auto")
    cbar = plt.colorbar()
    cbar.ax.tick_params(labelsize=18)

    ax_qB = plt.subplot(212, title=f"QB ({qB})")
    ax_qB.set_ylabel(f"Gain [DAC units]", fontsize=18)
    ax_qB.set_xlabel("Frequency [MHz]", fontsize=18)
    ax_qB.tick_params(axis="both", which="major", labelsize=16)
    plot_data = data[data_name][1]
    plt.pcolormesh(x_sweep, y_sweep, plot_data, cmap="viridis", shading="auto")
    cbar = plt.colorbar()
    cbar.ax.tick_params(labelsize=18)

    plt.tight_layout()
    plt.show()

    return sweep_expt.data

In [57]:
def calculate_gain_freq_tuples(pipi_data, piminuspi_data, opt_len, interpolate=True):
    if interpolate:
        pipi_freq_sweep, pipi_gain_sweep = pipi_data["freq_sweep"], pipi_data["gain_sweep"]
        piminuspi_freq_sweep, piminuspi_gain_sweep = piminuspi_data["freq_sweep"], piminuspi_data["gain_sweep"]
        reduced_freq_sweep = np.linspace(np.max([pipi_freq_sweep[0], piminuspi_freq_sweep[0]]), np.min([pipi_freq_sweep[-1], piminuspi_freq_sweep[-1]]), 20)
        reduced_gain_sweep = np.linspace(np.max([pipi_gain_sweep[0], piminuspi_gain_sweep[0]]), np.min([pipi_gain_sweep[-1], piminuspi_gain_sweep[-1]]), 25)
        reduced_y = reduced_gain_sweep
        reduced_x = reduced_freq_sweep
        
        print("reduced gain range", reduced_gain_sweep[0], "to", reduced_gain_sweep[-1])
        print("reduced freq range", reduced_freq_sweep[0], "to", reduced_freq_sweep[-1])

        fine_pipi_data = np.zeros((2, len(reduced_y), len(reduced_x)))
        fine_piminuspi_data = np.zeros((2, len(reduced_y), len(reduced_x)))
        for iq, q in enumerate(qubits):
            y, x = pipi_gain_sweep, pipi_freq_sweep
            pipi_interp = sp.interpolate.RegularGridInterpolator((y, x), pipi_data["products"][iq], bounds_error=False, fill_value=0)

            y, x = piminuspi_gain_sweep, piminuspi_freq_sweep
            piminuspi_interp = sp.interpolate.RegularGridInterpolator((y, x), piminuspi_data["products"][iq], bounds_error=False, fill_value=0)

            yg, xg = np.meshgrid(reduced_y, reduced_x, indexing='ij')
            fine_pipi_data[iq] = pipi_interp((yg, xg))
            fine_piminuspi_data[iq] = piminuspi_interp((yg, xg))

    else:
        assert np.all([pipi_data["freq_sweep"][i] == piminuspi_data["freq_sweep"][i] for i in range(len(pipi_data["freq_sweep"]))])
        assert np.all([pipi_data["gain_sweep"][i] == piminuspi_data["gain_sweep"][i] for i in range(len(pipi_data["gain_sweep"]))])
        assert len(pipi_data["freq_sweep"]) == len(piminuspi_data["freq_sweep"])
        assert len(pipi_data["gain_sweep"]) == len(piminuspi_data["gain_sweep"])
        fine_pipi_data = pipi_data["products"]
        fine_piminuspi_data = piminuspi_data["products"]
        reduced_x = pipi_data["freq_sweep"]
        reduced_y = pipi_data["gain_sweep"]

    x_sweep = reduced_x
    y_sweep = reduced_y

    prod_of_prods_data_q = np.array([fine_piminuspi_data[i] * fine_pipi_data[i] for i in range(2)])

    qA, qB = qubits

    # Interpolated pi pi data
    label_pipi = "($X_{\pi}, X_{\pi})^N$"
    if test_pi_half:
        label_pipi = "($X_{\pi/2}, X_{\pi/2})^{2N}$"
    title = (
        f"Interpolated Q{qA}/Q{qB} Product of Ns for {label_pipi}\n Len {opt_len:.3f}, Ns={pipi_data['cycle_sweep'].tolist()}"
    )

    plt.figure(figsize=(8, 9))
    plt.suptitle(title, fontsize=20)

    ax_qA = plt.subplot(211, title=f"QA ({qA})")
    ax_qA.set_ylabel(f"Gain", fontsize=18)
    ax_qA.tick_params(axis="both", which="major", labelsize=16)
    plot_data = fine_pipi_data[0]
    plt.pcolormesh(x_sweep, y_sweep, plot_data, cmap="viridis", shading="auto")
    cbar = plt.colorbar()
    cbar.ax.tick_params(labelsize=18)

    ax_qB = plt.subplot(212, title=f"QB ({qB})")
    ax_qB.set_ylabel(f"Gain", fontsize=18)
    ax_qB.set_xlabel("Frequency [MHz]", fontsize=18)
    ax_qB.tick_params(axis="both", which="major", labelsize=16)
    plot_data = fine_pipi_data[1]
    plt.pcolormesh(x_sweep, y_sweep, plot_data, cmap="viridis", shading="auto")
    cbar = plt.colorbar()
    cbar.ax.tick_params(labelsize=18)

    plt.tight_layout()
    plt.show()

    # Interpolated pi -pi data
    label_piminuspi = "($X_{\pi}, X_{-\pi})^N$"
    if test_pi_half:
        # label_piminuspi = "($X_{\pi/2}, X_{-\pi/2})^{N}$"
        label_piminuspi = "($X_{\pi/2}, X_{\pi/2}, X_{-\pi/2}, X_{-\pi/2})^N$"
    title = (
        f"Interpolated Q{qA}/Q{qB} Product of Ns for {label_piminuspi}\n Len {opt_len:.3f}, Ns={piminuspi_data['cycle_sweep'].tolist()}"
    )

    plt.figure(figsize=(8, 9))
    plt.suptitle(title, fontsize=20)

    ax_qA = plt.subplot(211, title=f"QA ({qA})")
    ax_qA.set_ylabel(f"Gain", fontsize=18)
    ax_qA.tick_params(axis="both", which="major", labelsize=16)
    plot_data = fine_piminuspi_data[0]
    plt.pcolormesh(x_sweep, y_sweep, plot_data, cmap="viridis", shading="auto")
    cbar = plt.colorbar()
    cbar.ax.tick_params(labelsize=18)

    ax_qB = plt.subplot(212, title=f"QB ({qB})")
    ax_qB.set_ylabel(f"Gain", fontsize=18)
    ax_qB.set_xlabel("Frequency [MHz]", fontsize=18)
    ax_qB.tick_params(axis="both", which="major", labelsize=16)
    plot_data = fine_piminuspi_data[1]
    plt.pcolormesh(x_sweep, y_sweep, plot_data, cmap="viridis", shading="auto")
    cbar = plt.colorbar()
    cbar.ax.tick_params(labelsize=18)

    plt.tight_layout()
    plt.show()

    # Product of the two inteprolated data sets
    title = (
        f"Interpolated Q{qA}/Q{qB} Product {label_piminuspi} $\\times$ {label_pipi}\n Len {opt_len:.3f}"
    )

    plt.figure(figsize=(8, 9))
    plt.suptitle(title, fontsize=20)

    ax_qA = plt.subplot(211, title=f"QA ({qA})")
    ax_qA.set_ylabel(f"Gain", fontsize=18)
    ax_qA.tick_params(axis="both", which="major", labelsize=16)
    plot_data = prod_of_prods_data_q[0]
    plt.pcolormesh(x_sweep, y_sweep, plot_data, cmap="viridis", shading="auto")
    cbar = plt.colorbar()
    cbar.ax.tick_params(labelsize=18)
    
    ax_qB = plt.subplot(212, title=f"QB ({qB})")
    ax_qB.set_ylabel(f"Gain", fontsize=18)
    ax_qB.set_xlabel("Frequency [MHz]", fontsize=18)
    ax_qB.tick_params(axis="both", which="major", labelsize=16)
    plot_data = prod_of_prods_data_q[1]
    plt.pcolormesh(x_sweep, y_sweep, plot_data, cmap="viridis", shading="auto")
    cbar = plt.colorbar()
    cbar.ax.tick_params(labelsize=18)

    plt.tight_layout()
    plt.show()

    return np.prod(prod_of_prods_data_q, axis=0), x_sweep, y_sweep

In [50]:
from skimage.feature import blob_dog, blob_log, blob_doh
from matplotlib.patches import Ellipse

def find_blobs(data_2d, x_sweep, y_sweep):
    data_2d = (np.copy(data_2d) - np.min(data_2d))/(np.max(data_2d) - np.min(data_2d))
    blobs_log = blob_log(data_2d, max_sigma=5, min_sigma=3, num_sigma=10, threshold=0.1, exclude_border=True)

    # Compute radii in the 3rd column.
    blobs_log[:, 2] = blobs_log[:, 2] * sqrt(2)

    blobs_dog = blob_dog(data_2d, max_sigma=5, min_sigma=3, threshold=0.1, exclude_border=True)
    blobs_dog[:, 2] = blobs_dog[:, 2] * sqrt(2)

    blobs_doh = blob_doh(data_2d, max_sigma=5, min_sigma=3, threshold=0.01)

    blobs_list = [blobs_log, blobs_dog, blobs_doh]
    colors = ['yellow', 'lime', 'red']
    titles = ['Laplacian of Gaussian', 'Difference of Gaussian', 'Determinant of Hessian']
    sequence = zip(blobs_list, colors, titles)

    fig, axes = plt.subplots(1, 3, figsize=(9, 3), sharex=True, sharey=True)
    ax = axes.ravel()

    for idx, (blobs, color, title) in enumerate(sequence):
        ax[idx].set_title(title)
        # ax[idx].imshow(data_2d)
        plt.sca(ax[idx])
        plt.pcolormesh(x_sweep, y_sweep, data_2d, cmap="viridis", shading="auto")
        for blob in blobs:
            r, c, sigma = blob
            opt_freq = x_sweep[int(c)]
            opt_gain = y_sweep[int(r)]
            sigma_x = sigma * (x_sweep[1] - x_sweep[0])
            sigma_y = sigma * (y_sweep[1] - y_sweep[0])
            c = Ellipse((opt_freq, opt_gain), width=sigma_x, height=sigma_y, color=color, linewidth=2, fill=False)
            ax[idx].add_patch(c)
        plt.colorbar()
        # ax[idx].set_axis_off()

    plt.tight_layout()
    plt.show()

    # return blobs_list
    return blobs_log


### V2 Start the fine calibration rigorous

#### Q3/Q1

In [ ]:
qubits = [3, 1] # order doesn't matter if you specify qDrive correctly
init_state = '|1>|1>' # for everything else
wrong_init_state = '|0>|1>'
setup_ZZ = 0 # for the amp rabi chevron
# measure_qubits = [qubits[0], qubits[1], 2]
measure_qubits = qubits
qDrive = qubits[0] # normal new protocol

In [ ]:
import yaml
with open(config_path, 'r') as cfg_file:
    yaml_cfg = yaml.safe_load(cfg_file)
yaml_cfg = AttrDict(yaml_cfg)

In [ ]:
do_amp_rabi_chevron = do_amp_rabi_chevron2 = do_prod_sweep = False

# SET THE EXPERIMENTS YOU WANT TO RUN HERE BY UNCOMMENTING!
do_amp_rabi_chevron = True
do_amp_rabi_chevron2 = True
do_prod_sweep = True


# put an override=True flag to force skipping certain qubits

save_files = True

good_data = False

full_mux_expt = False
resonator_reset = [0, 2, 3] if full_mux_expt else None
use_robust_pulses = True

test_pi_half = True

In [ ]:
npts = 20

# Lengths to test
opt_lens = np.linspace(0.8, 0.200, npts)
if test_pi_half:
    opt_lens /= 2

# Approximate gains for this length
opt_gains = np.linspace(7000, 32000, npts)

# Calculate the approximate frequencies corresponding to the gain sweep using the fit 
# opt_freqs = meas.fitting.quadraticfunc(opt_gains, *p)
opt_freqs = np.linspace(None, None, npts) # FILL THIS IN

In [ ]:
print('test lens opt_lens\n', opt_lens)
print('approx gains opt_gains\n', opt_gains)
print('approx freq for each gain\n', opt_freqs.tolist())

print('qubits', qubits)

new_opt_gains = []
new_opt_freqs = []
new_opt_gains2 = []
new_opt_freqs2 = []
new_opt_gains3 = []
new_opt_freqs3 = []

saved_files_amp_rabi_chevron = []
saved_files_amp_rabi_chevron2 = []
saved_files_piminuspi_sweep = []
saved_files_pipi_sweep = []

# opt_fit_index = 1 # index of [QA I, QA Q, QB I, QB Q] to use as saved optimal
signs = [1, 1, 1, 1] # whether the period occurs at the max (1) or min (-1)

opt_fit_index = 0 # index of [QA amps, QB amps] to use as saved optimal

test lens opt_lens
 [0.25]
approx gains opt_gains
 [12000]
approx freq for each gain
 [5885.6]
qubits [2, 1]


In [ ]:
# signs = [1, -1, -1, 1] # whether the period occurs at the max (1) or min (-1)
# opt_fit_index = 1

if not save_files: print('WARNING: NOT SAVING FILES!')
for i_len, pi_len in enumerate(opt_lens):
    # if i_gain != 7 and i_gain != 12: continue 
    # if gain < 15822: continue
    # if i_gain not in [3]: continue

    override = False

    pi_gain = opt_gains[i_len]
    pi_freq = opt_freqs[i_len]

    print(f'----------- STARTING LENGTH {pi_len} -----------')

    # ----- Amp Rabi Chevron ----- #
    # override = True
    # if gain > 15822 and gain < 15830:
    #     override = True
    #     pi_freq = yaml_cfg.device.qubit.f_EgGf_Q[qDrive]
    if do_amp_rabi_chevron and not override:
        pi_freq, pi_gain = run_amp_rabi_chevron(qDrive, yaml_cfg, opt_gain=pi_gain, opt_len=pi_len, opt_freq=pi_freq, test_pi_half=test_pi_half, span_gain=5000, span_f=15.0, save_data=save_files, saved_files=saved_files_amp_rabi_chevron, good_data=good_data)
        if not test_pi_half:
            yaml_cfg.device.qubit.f_EgGf_Q[qDrive] = pi_freq
            yaml_cfg.device.qubit.pulses.pi_EgGf_Q.gain[qDrive] = pi_gain
        else:
            yaml_cfg.device.qubit.f_EgGf_Q_half[qDrive] = pi_freq
            yaml_cfg.device.qubit.pulses.pi_EgGf_Q.half_gain[qDrive] = pi_gain
        if len(new_opt_freqs) > i_len:
            new_opt_freqs[i_len] = pi_freq
            print(f'for len {pi_len} updated swap pi freq from amp rabi chevron', pi_freq)
        else:
            new_opt_freqs.append(pi_freq)
            print(f'for len {pi_len} added swap pi freq from amp rabi chevron', pi_freq)
        if len(new_opt_gains) > i_len:
            new_opt_gains[i_len] = pi_gain
            print(f'for len {pi_len} updated swap pi gain from amp rabi chevron', pi_gain)
        else:
            new_opt_gains.append(pi_gain)
            print(f'for len {pi_len} added swap pi gain from amp rabi chevron', pi_gain)
    override = False

    # ----- Amp Rabi Chevron 2 ----- #
    # override = True
    # if gain > 15822 and gain < 15830:
    #     override = True
    #     pi_freq = yaml_cfg.device.qubit.f_EgGf_Q[qDrive]
    if do_amp_rabi_chevron2 and not override:
        pi_freq, pi_gain = run_amp_rabi_chevron(qDrive, yaml_cfg, opt_gain=pi_gain, opt_len=pi_len, opt_freq=pi_freq, test_pi_half=test_pi_half, span_gain=1000, span_f=5.0, save_data=save_files, saved_files=saved_files_amp_rabi_chevron2, good_data=good_data)
        if not test_pi_half:
            yaml_cfg.device.qubit.f_EgGf_Q[qDrive] = pi_freq
            yaml_cfg.device.qubit.pulses.pi_EgGf_Q.gain[qDrive] = pi_gain
        else:
            yaml_cfg.device.qubit.f_EgGf_Q_half[qDrive] = pi_freq
            yaml_cfg.device.qubit.pulses.pi_EgGf_Q.half_gain[qDrive] = pi_gain
        if len(new_opt_freqs2) > i_len:
            new_opt_freqs2[i_len] = pi_freq
            print(f'for len {pi_len} updated swap pi freq from amp rabi chevron fine', pi_freq)
        else:
            new_opt_freqs2.append(pi_freq)
            print(f'for len {pi_len} added swap pi freq from amp rabi chevron fine', pi_freq)
        if len(new_opt_gains2) > i_len:
            new_opt_gains2[i_len] = pi_gain
            print(f'for len {pi_len} updated swap pi gain from amp rabi chevron fine', pi_gain)
        else:
            new_opt_gains2.append(pi_gain)
            print(f'for len {pi_len} added swap pi gain from amp rabi chevron fine', pi_gain)
    override = False


    # ----- Pi Minus Pi Gain Sweep ----- #
    # if gain > 15822 and gain < 15830:
    #     override = True
    #     pi_len = yaml_cfg.device.qubit.pulses.pi_EgGf_Q.sigma[qDrive]
    if do_prod_sweep and not override:

        # override = True
        if not override:
            piminuspi_data = run_swap_piminuspi_gain_sweep(qDrive, yaml_cfg, opt_gain=pi_gain, opt_len=pi_len, opt_freq=pi_freq, span_f=1.5, test_pi_half=test_pi_half, save_data=save_files, saved_files=saved_files_piminuspi_sweep, good_data=good_data)
        override = False

        # override = True
        if not override:
            pipi_data = run_swap_pipi_gain_sweep(qDrive, yaml_cfg, opt_gain=pi_gain, opt_len=pi_len, opt_freq=pi_freq, span_f=1.5, test_pi_half=test_pi_half, save_data=save_files, saved_files=saved_files_pipi_sweep, good_data=good_data)
        override = False

        prod_of_prods_data, x_sweep, y_sweep = calculate_gain_freq_tuples(pipi_data, piminuspi_data, opt_len=pi_len, interpolate=True)
        blobs = find_blobs(prod_of_prods_data, x_sweep=x_sweep, y_sweep=y_sweep)

        r, c, sigma = blobs[0]
        pi_freq = x_sweep[int(c)]
        pi_gain = y_sweep[int(r)]
        sigma_x = sigma * (x_sweep[1] - x_sweep[0])
        sigma_y = sigma * (y_sweep[1] - y_sweep[0])
        print("best freq from blob", pi_freq)
        print("best gain from blob", pi_gain)

        idxy = np.argmax(prod_of_prods_data)
        ymax, xmax = np.unravel_index(idxy, prod_of_prods_data.shape)
        pi_freq_max = x_sweep[xmax]
        pi_gain_max = y_sweep[ymax]
        print("best freq from max", pi_freq_max)
        print("best gain from max", pi_gain_max)
        if np.abs(pi_freq_max - pi_freq) > 2*sigma_x or np.abs(pi_gain_max - pi_gain) > 2*sigma_y:
            print('WARNING: max product location is not close to found blob center!')
        
        if not test_pi_half:
            yaml_cfg.device.qubit.f_EgGf_Q[qDrive] = pi_freq
            yaml_cfg.device.qubit.pulses.pi_EgGf_Q.gain[qDrive] = pi_gain
        else:
            yaml_cfg.device.qubit.f_EgGf_Q_half[qDrive] = pi_freq
            yaml_cfg.device.qubit.pulses.pi_EgGf_Q.half_gain[qDrive] = pi_gain
        if len(new_opt_freqs3) > i_len:
            new_opt_freqs3[i_len] = pi_freq
            print(f'for len {pi_len} updated swap pi freq from product sweep', pi_freq)
        else:
            new_opt_freqs3.append(pi_freq)
            print(f'for len {pi_len} added swap pi freq from product sweep', pi_freq)
        if len(new_opt_gains3) > i_len:
            new_opt_gains3[i_len] = pi_gain
            print(f'for len {pi_len} updated swap pi gain from product sweep', pi_gain)
        else:
            new_opt_gains3.append(pi_gain)
            print(f'for len {pi_len} added swap pi gain from product sweep', pi_gain)
    override = False

----------- STARTING LENGTH 0.25 -----------
reduced gain range 10800.0 to 12384.900000000001
reduced freq range 5885.1 to 5886.35
best freq from blob 5885.955263157895
best gain from blob 11130.1875
best freq from max 5885.889473684211
best gain from max 11196.225
for len 0.25 added swap pi freq from product sweep 5885.955263157895
for len 0.25 added swap pi gain from product sweep 11130.1875


In [ ]:
print(f'opt_lens (test)={np.array(opt_lens).tolist()}')
print(f'opt_gains (initial guess)={np.array(opt_gains).tolist()}')
print(f'new_opt_gains (amp rabi chevron 1)={new_opt_gains}')
print(f'new_opt_gains2 (amp rabi chevron 2)={new_opt_gains2}')
print(f'new_opt_gains3 (pipi/piminuspi/product)={new_opt_gains3}')
print(f'opt_freqs (initial guess)={opt_freqs}')
print(f'new_opt_freqs (amp rabi chevron 1)={new_opt_freqs}')
print(f'new_opt_freqs2 (amp rabi chevron 2)={new_opt_freqs2}')
print(f'new_opt_freqs3 (pipi/piminuspi/product)={new_opt_freqs3}')
print(f'saved_files_amp_rabi_chevron={saved_files_amp_rabi_chevron}')
print(f'saved_files_amp_rabi_chevron2={saved_files_amp_rabi_chevron2}')
print(f'saved_files_piminuspi_sweep={saved_files_piminuspi_sweep}')
print(f'saved_files_pipi_sweep={saved_files_pipi_sweep}')

plt.figure(figsize=(8, 6))
plt.plot(opt_lens, opt_freqs, '.-', label='Initial guess')
if do_amp_rabi_chevron:
    plt.plot(opt_lens, new_opt_freqs, '.-', label='Amp Rabi Chevron')
if do_amp_rabi_chevron2:
    plt.plot(opt_lens, new_opt_freqs2, '.-', label='Amp Rabi Chevron Fine')
if do_prod_sweep:
    plt.plot(opt_lens, new_opt_freqs3, '.-', label='Product Sweep')
plt.xlabel('Length (us)')
plt.ylabel('Frequency (MHz)')
plt.legend()
plt.title(f'Q{qubits[0]}Q{qubits[1]}')
plt.show()

plt.figure(figsize=(8, 6))
plt.plot(opt_lens, opt_gains, '.-', label='Initial guess')
if do_amp_rabi_chevron:
    plt.plot(opt_lens, new_opt_gains, '.-', label='Amp Rabi Chevron')
if do_amp_rabi_chevron2:
    plt.plot(opt_lens, new_opt_gains2, '.-', label='Amp Rabi Chevron Fine')
if do_prod_sweep:
    plt.plot(opt_lens, new_opt_gains3, '.-', label='Product Sweep')
plt.xlabel('Length (us)')
plt.ylabel('Gains (DAC units)')
plt.legend()
plt.title(f'Q{qubits[0]}Q{qubits[1]}')
plt.show()


opt_lens (test)=[0.25]
opt_gains (initial guess)=[12000]
new_opt_gains (amp rabi chevron 1)=[11165]
new_opt_gains2 (amp rabi chevron 2)=[11259]
new_opt_gains3 (pipi/piminuspi/product)=[11130.1875]
opt_freqs (initial guess)=[5885.6]
new_opt_freqs (amp rabi chevron 1)=[5885.975]
new_opt_freqs2 (amp rabi chevron 2)=[5885.85]
new_opt_freqs3 (pipi/piminuspi/product)=[5885.955263157895]
saved_files_amp_rabi_chevron=['S:\\QRAM\\qram_4QR2\\data\\data_241025\\00004_rabi_EgGf_freqgain_chevron_qubit21.h5']
saved_files_amp_rabi_chevron2=['S:\\QRAM\\qram_4QR2\\data\\data_241025\\00005_rabi_EgGf_freqgain_chevron_qubit21.h5']
saved_files_piminuspi_sweep=['S:\\QRAM\\qram_4QR2\\data\\data_241025\\00002_eggf_piminuspi_gain_sweep_Q21.h5']
saved_files_pipi_sweep=['S:\\QRAM\\qram_4QR2\\data\\data_241025\\00003_eggf_pipi_gain_sweep_Q21.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00004_eggf_pipi_gain_sweep_Q21.h5']


#### Q2/Q1

In [21]:
qubits = [2, 1] # order doesn't matter if you specify qDrive correctly
init_state = '|0>|1>'
wrong_init_state = '|1>|1>'
setup_ZZ = None
# measure_qubits = [qubits[0], qubits[1], 3]
measure_qubits = qubits
qDrive = qubits[0] # normal new protocol

In [22]:
import yaml
with open(config_path, 'r') as cfg_file:
    yaml_cfg = yaml.safe_load(cfg_file)
yaml_cfg = AttrDict(yaml_cfg)

In [25]:
do_amp_rabi_chevron = do_amp_rabi_chevron2 = do_prod_sweep = False

# SET THE EXPERIMENTS YOU WANT TO RUN HERE BY UNCOMMENTING!
do_amp_rabi_chevron = True
do_amp_rabi_chevron2 = True
do_prod_sweep = True


# put an override=True flag to force skipping certain qubits

save_files = True

good_data = False

full_mux_expt = False
resonator_reset = [0, 2, 3] if full_mux_expt else None
use_robust_pulses = True

test_pi_half = True

In [26]:
npts = 20

# Lengths to test
opt_lens = np.linspace(0.8, 0.200, npts)
if test_pi_half:
    opt_lens /= 2

# Approximate gains for this length
opt_gains = np.linspace(7000, 32000, npts)

# Calculate the approximate frequencies corresponding to the gain sweep using the fit 
# opt_freqs = meas.fitting.quadraticfunc(opt_gains, *p)
opt_freqs = np.linspace(5887.0, 5880.7, npts)

In [29]:
# opt_lens = np.array([0.250])
# opt_gains = np.array([12000])
# opt_freqs = np.array([5885.6])

In [30]:
print('test lens opt_lens\n', opt_lens)
print('approx gains opt_gains\n', opt_gains)
print('approx freq for each gain\n', opt_freqs.tolist())

print('qubits', qubits)

new_opt_gains = []
new_opt_freqs = []
new_opt_gains2 = []
new_opt_freqs2 = []
new_opt_gains3 = []
new_opt_freqs3 = []

saved_files_amp_rabi_chevron = []
saved_files_amp_rabi_chevron2 = []
saved_files_piminuspi_sweep = []
saved_files_pipi_sweep = []

# opt_fit_index = 1 # index of [QA I, QA Q, QB I, QB Q] to use as saved optimal
signs = [1, 1, 1, 1] # whether the period occurs at the max (1) or min (-1)

opt_fit_index = 0 # index of [QA amps, QB amps] to use as saved optimal

test lens opt_lens
 [0.25]
approx gains opt_gains
 [12000]
approx freq for each gain
 [5885.6]
qubits [2, 1]


In [58]:
# signs = [1, -1, -1, 1] # whether the period occurs at the max (1) or min (-1)
# opt_fit_index = 1

if not save_files: print('WARNING: NOT SAVING FILES!')
for i_len, pi_len in enumerate(opt_lens):
    # if i_gain != 7 and i_gain != 12: continue 
    # if gain < 15822: continue
    # if i_gain not in [3]: continue

    override = False

    pi_gain = opt_gains[i_len]
    pi_freq = opt_freqs[i_len]

    print(f'----------- STARTING LENGTH {pi_len} -----------')

    # ----- Amp Rabi Chevron ----- #
    # override = True
    # if gain > 15822 and gain < 15830:
    #     override = True
    #     pi_freq = yaml_cfg.device.qubit.f_EgGf_Q[qDrive]
    if do_amp_rabi_chevron and not override:
        pi_freq, pi_gain = run_amp_rabi_chevron(qDrive, yaml_cfg, opt_gain=pi_gain, opt_len=pi_len, opt_freq=pi_freq, test_pi_half=test_pi_half, span_gain=5000, span_f=15.0, save_data=save_files, saved_files=saved_files_amp_rabi_chevron, good_data=good_data)
        if not test_pi_half:
            yaml_cfg.device.qubit.f_EgGf_Q[qDrive] = pi_freq
            yaml_cfg.device.qubit.pulses.pi_EgGf_Q.gain[qDrive] = pi_gain
        else:
            yaml_cfg.device.qubit.f_EgGf_Q_half[qDrive] = pi_freq
            yaml_cfg.device.qubit.pulses.pi_EgGf_Q.half_gain[qDrive] = pi_gain
        if len(new_opt_freqs) > i_len:
            new_opt_freqs[i_len] = pi_freq
            print(f'for len {pi_len} updated swap pi freq from amp rabi chevron', pi_freq)
        else:
            new_opt_freqs.append(pi_freq)
            print(f'for len {pi_len} added swap pi freq from amp rabi chevron', pi_freq)
        if len(new_opt_gains) > i_len:
            new_opt_gains[i_len] = pi_gain
            print(f'for len {pi_len} updated swap pi gain from amp rabi chevron', pi_gain)
        else:
            new_opt_gains.append(pi_gain)
            print(f'for len {pi_len} added swap pi gain from amp rabi chevron', pi_gain)
    override = False

    # ----- Amp Rabi Chevron 2 ----- #
    # override = True
    # if gain > 15822 and gain < 15830:
    #     override = True
    #     pi_freq = yaml_cfg.device.qubit.f_EgGf_Q[qDrive]
    if do_amp_rabi_chevron2 and not override:
        pi_freq, pi_gain = run_amp_rabi_chevron(qDrive, yaml_cfg, opt_gain=pi_gain, opt_len=pi_len, opt_freq=pi_freq, test_pi_half=test_pi_half, span_gain=1000, span_f=5.0, save_data=save_files, saved_files=saved_files_amp_rabi_chevron2, good_data=good_data)
        if not test_pi_half:
            yaml_cfg.device.qubit.f_EgGf_Q[qDrive] = pi_freq
            yaml_cfg.device.qubit.pulses.pi_EgGf_Q.gain[qDrive] = pi_gain
        else:
            yaml_cfg.device.qubit.f_EgGf_Q_half[qDrive] = pi_freq
            yaml_cfg.device.qubit.pulses.pi_EgGf_Q.half_gain[qDrive] = pi_gain
        if len(new_opt_freqs2) > i_len:
            new_opt_freqs2[i_len] = pi_freq
            print(f'for len {pi_len} updated swap pi freq from amp rabi chevron fine', pi_freq)
        else:
            new_opt_freqs2.append(pi_freq)
            print(f'for len {pi_len} added swap pi freq from amp rabi chevron fine', pi_freq)
        if len(new_opt_gains2) > i_len:
            new_opt_gains2[i_len] = pi_gain
            print(f'for len {pi_len} updated swap pi gain from amp rabi chevron fine', pi_gain)
        else:
            new_opt_gains2.append(pi_gain)
            print(f'for len {pi_len} added swap pi gain from amp rabi chevron fine', pi_gain)
    override = False


    # ----- Pi Minus Pi Gain Sweep ----- #
    # if gain > 15822 and gain < 15830:
    #     override = True
    #     pi_len = yaml_cfg.device.qubit.pulses.pi_EgGf_Q.sigma[qDrive]
    if do_prod_sweep and not override:

        # override = True
        if not override:
            piminuspi_data = run_swap_piminuspi_gain_sweep(qDrive, yaml_cfg, opt_gain=pi_gain, opt_len=pi_len, opt_freq=pi_freq, span_f=1.5, test_pi_half=test_pi_half, save_data=save_files, saved_files=saved_files_piminuspi_sweep, good_data=good_data)
        override = False

        # override = True
        if not override:
            pipi_data = run_swap_pipi_gain_sweep(qDrive, yaml_cfg, opt_gain=pi_gain, opt_len=pi_len, opt_freq=pi_freq, span_f=1.5, test_pi_half=test_pi_half, save_data=save_files, saved_files=saved_files_pipi_sweep, good_data=good_data)
        override = False

        prod_of_prods_data, x_sweep, y_sweep = calculate_gain_freq_tuples(pipi_data, piminuspi_data, opt_len=pi_len, interpolate=True)
        blobs = find_blobs(prod_of_prods_data, x_sweep=x_sweep, y_sweep=y_sweep)

        r, c, sigma = blobs[0]
        pi_freq = x_sweep[int(c)]
        pi_gain = y_sweep[int(r)]
        sigma_x = sigma * (x_sweep[1] - x_sweep[0])
        sigma_y = sigma * (y_sweep[1] - y_sweep[0])
        print("best freq from blob", pi_freq)
        print("best gain from blob", pi_gain)

        idxy = np.argmax(prod_of_prods_data)
        ymax, xmax = np.unravel_index(idxy, prod_of_prods_data.shape)
        pi_freq_max = x_sweep[xmax]
        pi_gain_max = y_sweep[ymax]
        print("best freq from max", pi_freq_max)
        print("best gain from max", pi_gain_max)
        if np.abs(pi_freq_max - pi_freq) > 2*sigma_x or np.abs(pi_gain_max - pi_gain) > 2*sigma_y:
            print('WARNING: max product location is not close to found blob center!')
        
        if not test_pi_half:
            yaml_cfg.device.qubit.f_EgGf_Q[qDrive] = pi_freq
            yaml_cfg.device.qubit.pulses.pi_EgGf_Q.gain[qDrive] = pi_gain
        else:
            yaml_cfg.device.qubit.f_EgGf_Q_half[qDrive] = pi_freq
            yaml_cfg.device.qubit.pulses.pi_EgGf_Q.half_gain[qDrive] = pi_gain
        if len(new_opt_freqs3) > i_len:
            new_opt_freqs3[i_len] = pi_freq
            print(f'for len {pi_len} updated swap pi freq from product sweep', pi_freq)
        else:
            new_opt_freqs3.append(pi_freq)
            print(f'for len {pi_len} added swap pi freq from product sweep', pi_freq)
        if len(new_opt_gains3) > i_len:
            new_opt_gains3[i_len] = pi_gain
            print(f'for len {pi_len} updated swap pi gain from product sweep', pi_gain)
        else:
            new_opt_gains3.append(pi_gain)
            print(f'for len {pi_len} added swap pi gain from product sweep', pi_gain)
    override = False

----------- STARTING LENGTH 0.25 -----------
reduced gain range 10800.0 to 12384.900000000001
reduced freq range 5885.1 to 5886.35
best freq from blob 5885.955263157895
best gain from blob 11130.1875
best freq from max 5885.889473684211
best gain from max 11196.225
for len 0.25 added swap pi freq from product sweep 5885.955263157895
for len 0.25 added swap pi gain from product sweep 11130.1875


In [59]:
print(f'opt_lens (test)={np.array(opt_lens).tolist()}')
print(f'opt_gains (initial guess)={np.array(opt_gains).tolist()}')
print(f'new_opt_gains (amp rabi chevron 1)={new_opt_gains}')
print(f'new_opt_gains2 (amp rabi chevron 2)={new_opt_gains2}')
print(f'new_opt_gains3 (pipi/piminuspi/product)={new_opt_gains3}')
print(f'opt_freqs (initial guess)={opt_freqs}')
print(f'new_opt_freqs (amp rabi chevron 1)={new_opt_freqs}')
print(f'new_opt_freqs2 (amp rabi chevron 2)={new_opt_freqs2}')
print(f'new_opt_freqs3 (pipi/piminuspi/product)={new_opt_freqs3}')
print(f'saved_files_amp_rabi_chevron={saved_files_amp_rabi_chevron}')
print(f'saved_files_amp_rabi_chevron2={saved_files_amp_rabi_chevron2}')
print(f'saved_files_piminuspi_sweep={saved_files_piminuspi_sweep}')
print(f'saved_files_pipi_sweep={saved_files_pipi_sweep}')

plt.figure(figsize=(8, 6))
plt.plot(opt_lens, opt_freqs, '.-', label='Initial guess')
if do_amp_rabi_chevron:
    plt.plot(opt_lens, new_opt_freqs, '.-', label='Amp Rabi Chevron')
if do_amp_rabi_chevron2:
    plt.plot(opt_lens, new_opt_freqs2, '.-', label='Amp Rabi Chevron Fine')
if do_prod_sweep:
    plt.plot(opt_lens, new_opt_freqs3, '.-', label='Product Sweep')
plt.xlabel('Length (us)')
plt.ylabel('Frequency (MHz)')
plt.legend()
plt.title(f'Q{qubits[0]}Q{qubits[1]}')
plt.show()

plt.figure(figsize=(8, 6))
plt.plot(opt_lens, opt_gains, '.-', label='Initial guess')
if do_amp_rabi_chevron:
    plt.plot(opt_lens, new_opt_gains, '.-', label='Amp Rabi Chevron')
if do_amp_rabi_chevron2:
    plt.plot(opt_lens, new_opt_gains2, '.-', label='Amp Rabi Chevron Fine')
if do_prod_sweep:
    plt.plot(opt_lens, new_opt_gains3, '.-', label='Product Sweep')
plt.xlabel('Length (us)')
plt.ylabel('Gains (DAC units)')
plt.legend()
plt.title(f'Q{qubits[0]}Q{qubits[1]}')
plt.show()


opt_lens (test)=[0.25]
opt_gains (initial guess)=[12000]
new_opt_gains (amp rabi chevron 1)=[11165]
new_opt_gains2 (amp rabi chevron 2)=[11259]
new_opt_gains3 (pipi/piminuspi/product)=[11130.1875]
opt_freqs (initial guess)=[5885.6]
new_opt_freqs (amp rabi chevron 1)=[5885.975]
new_opt_freqs2 (amp rabi chevron 2)=[5885.85]
new_opt_freqs3 (pipi/piminuspi/product)=[5885.955263157895]
saved_files_amp_rabi_chevron=['S:\\QRAM\\qram_4QR2\\data\\data_241025\\00004_rabi_EgGf_freqgain_chevron_qubit21.h5']
saved_files_amp_rabi_chevron2=['S:\\QRAM\\qram_4QR2\\data\\data_241025\\00005_rabi_EgGf_freqgain_chevron_qubit21.h5']
saved_files_piminuspi_sweep=['S:\\QRAM\\qram_4QR2\\data\\data_241025\\00002_eggf_piminuspi_gain_sweep_Q21.h5']
saved_files_pipi_sweep=['S:\\QRAM\\qram_4QR2\\data\\data_241025\\00003_eggf_pipi_gain_sweep_Q21.h5', 'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00004_eggf_pipi_gain_sweep_Q21.h5']


### V1 Start the fine calibration

#### Q3/Q1

In [54]:
qubits = [3, 1] # order doesn't matter if you specify qDrive correctly
init_state = '|1>|1>' # for everything else
wrong_init_state = '|0>|1>'
setup_ZZ = 0 # for the amp rabi chevron
# measure_qubits = [qubits[0], qubits[1], 2]
measure_qubits = qubits
qDrive = qubits[0] # normal new protocol

In [12]:
do_piminuspi = do_length_rabi = do_error_amp = do_pi_train = False

do_piminuspi = True
do_length_rabi = True
# do_error_amp = True
# do_pi_train = True

# put an override=True flag to force skipping certain qubits

save_files = True

good_data = False

full_mux_expt = False
resonator_reset = [0, 2, 3] if full_mux_expt else None
use_robust_pulses = True

test_pi_half = True

In [127]:
import yaml
with open(config_path, 'r') as cfg_file:
    yaml_cfg = yaml.safe_load(cfg_file)
yaml_cfg = AttrDict(yaml_cfg)

In [138]:
# Input the freqs fitted from the amp rabi chevron and the gain sweep used in that chevron

fit_gain_sweep = [5000, 5466, 5932, 6398, 6864, 7330, 7796, 8262, 8728, 9194, 9660, 10126, 10592, 11058, 11524]
fit_freqs = [5147.314867441374, 5146.791028104514, 5146.251280336857, 5145.695624138401, 5145.1240595091485, 5144.536586449097, 5143.933204958248, 5143.313915036601, 5142.678716684156, 5142.027609900913, 5141.360594686873, 5140.6776710420345, 5139.978838966397, 5139.264098459963, 5138.533449522731]

fit_gain_sweep=[5000.0, 5368.421052631579, 5736.842105263158, 6105.263157894737, 6473.684210526316, 6842.105263157895, 7210.526315789473, 7578.9473684210525, 7947.368421052632, 8315.78947368421, 8684.21052631579, 9052.631578947368, 9421.052631578947, 9789.473684210527, 10157.894736842105, 10526.315789473683, 10894.736842105263, 11263.157894736843, 11631.578947368422, 12000.0]
fit_freqs=[5146.789574259002, 5146.412882589542, 5146.021416670141, 5145.606973899523, 5145.1563842852165, 5144.697617412424, 5144.203198597103, 5143.7108805224125, 5143.1757994866575, 5142.616613546966, 5142.051963629243, 5141.466751340055, 5140.864129408276, 5140.159388480326, 5139.591682895483, 5138.944711406807, 5138.364109972535, 5137.7439350382165, 5137.163794127236, 5136.556006672831]


# Fit those to a quadratic function and use this to extrapolate frequencies for the gain sweep of the actual experiment
fitparams = [fit_gain_sweep[0], fit_freqs[0], fit_freqs[-1] - fit_freqs[0]]
p, pCov = meas.fitting.fitquadratic(fit_gain_sweep, fit_freqs, fitparams=fitparams)

In [ ]:
fit_lens = [0.18023128711677291, 0.17101304980492513, 0.16118489225225882, 0.15120995977739157, 0.14536089985545117, 0.1377674476950781, 0.13309149699515288, 0.1284193605788396, 0.12501948048116884, 0.12038339136860676, 0.11747007510218732, 0.11440299529003421, 0.11195940862488307, 0.10983460548263342, 0.10632642500320487, 0.10466575396824139, 0.10242294977468908, 0.10060998701110126, 0.09762767999394902, 0.09642304743241509]
fitparams = [fit_gain_sweep[0], fit_lens[0], fit_lens[-1] - fit_lens[0]]
plen, plenCov = meas.fitting.fitquadratic(fit_gain_sweep, fit_lens, fitparams=fitparams)

plt.figure()
plt.plot(fit_gain_sweep, fit_lens)
plt.plot(fit_gain_sweep, meas.fitting.quadraticfunc(fit_gain_sweep, *plen))
plt.show()
opt_lens = meas.fitting.quadraticfunc(gain_sweep, *plen)

In [ ]:
npts = 3

# gain_sweep = np.linspace(5000, 12000, npts)
gain_sweep = np.linspace(12000, 18000, npts)
print('gain_sweep\n', gain_sweep)

# Guess the approximate lengths corresponding to the gain sweep
# opt_lens = np.linspace(0.4, 0.200, npts)
opt_lens = np.linspace(0.2, 0.140, npts)
if test_pi_half:
    opt_lens /= 2
# opt_lens = meas.fitting.quadraticfunc(gain_sweep, *plen)
print('approx len for each gain (us)\n', opt_lens.tolist())

# For pi/-pi
# span_fs = np.linspace(2.0, 3.5, npts)
span_fs = [4.5]*npts

# Calculate the approximate frequencies corresponding to the gain sweep using the fit 
# opt_freqs = meas.fitting.quadraticfunc(gain_sweep, *p)
# opt_freqs = np.linspace(5136.5, 5128, npts)
opt_freqs = np.linspace(5136.5, 5112.5, npts)
print('approx freq for each gain\n', opt_freqs.tolist())

print('qubits', qubits)

new_opt_lens = [] # length rabi
new_opt_lens2 = [] # error amp
new_opt_lens3 = [] # pi pulse train

new_opt_freqs = []

saved_files_lengthrabi = []
saved_files_piminuspi = []
saved_files_error_amp = []
saved_files_pitrain = []

# opt_fit_index = 1 # index of [QA I, QA Q, QB I, QB Q] to use as saved optimal
signs = [1, 1, 1, 1] # whether the period occurs at the max (1) or min (-1)

opt_fit_index = 0 # index of [QA amps, QB amps] to use as saved optimal

In [198]:
# new_opt_lens = [] # length rabi
# new_opt_lens2 = [] # error amp
# new_opt_lens3 = [] # pi pulse train
# new_opt_freqs = []

# saved_files_lengthrabi = []
# saved_files_piminuspi = []
# saved_files_error_amp = []
# saved_files_pitrain = []

# # opt_fit_index = 1 # index of [QA I, QA Q, QB I, QB Q] to use as saved optimal
# signs = [1, 1, 1, 1] # whether the period occurs at the max (1) or min (-1)

# opt_fit_index = 0 # index of [QA amps, QB amps] to use as saved optimal

# gain_sweep = [yaml_cfg.device.qubit.pulses.pi_EgGf_Q.gain[qubits[0]]]
# opt_lens = [yaml_cfg.device.qubit.pulses.pi_EgGf_Q.sigma[qubits[0]]]
# opt_freqs = [yaml_cfg.device.qubit.f_EgGf_Q[qubits[0]]]

# if test_pi_half:
#     gain_sweep = [yaml_cfg.device.qubit.pulses.pi_EgGf_Q.half_gain[qubits[0]]]
#     opt_lens = [yaml_cfg.device.qubit.pulses.pi_EgGf_Q.half_sigma[qubits[0]]]
#     opt_freqs = [yaml_cfg.device.qubit.f_EgGf_Q_half[qubits[0]]]

In [ ]:
# signs = [1, -1, -1, 1] # whether the period occurs at the max (1) or min (-1)
# opt_fit_index = 1

if not save_files: print('WARNING: NOT SAVING FILES!')
for i_gain, gain in enumerate(gain_sweep):


    if i_gain != 1: continue
    # if i_gain != 7 and i_gain != 12: continue 
    # if gain < 15822: continue
    # if i_gain not in [3]: continue

    override = False

    print(f'----------- STARTING GAIN {gain} -----------')
    pi_len = opt_lens[i_gain]
    pi_freq = opt_freqs[i_gain]

    # ----- Pi minus pi----- #
    # if gain > 15822 and gain < 15830:
    #     override = True
    #     pi_freq = yaml_cfg.device.qubit.f_EgGf_Q[qDrive]
    if do_piminuspi and not override:
        pi_freq = run_swap_piminuspi(qDrive, yaml_cfg, gain=gain, opt_len=pi_len, opt_freq=pi_freq, test_pi_half=test_pi_half, span_f=span_fs[i_gain], save_data=save_files, saved_files=saved_files_piminuspi, good_data=good_data)
        if not test_pi_half:
            yaml_cfg.device.qubit.f_EgGf_Q[qDrive] = pi_freq
        else:
            yaml_cfg.device.qubit.f_EgGf_Q_half[qDrive] = pi_freq
        if len(new_opt_freqs) > i_gain:
            new_opt_freqs[i_gain] = pi_freq
            print(f'for gain {gain} updated swap pi freq from pi minus pi', pi_freq)
        else:
            new_opt_freqs.append(pi_freq)
            print(f'for gain {gain} added swap pi freq from pi minus pi', pi_freq)
    override = False

    # ----- Length rabi ----- #
    # if gain > 15822 and gain < 15830:
    #     override = True
    #     pi_len = yaml_cfg.device.qubit.pulses.pi_EgGf_Q.sigma[qDrive]
    if do_length_rabi and not override:
        pi_len = run_swap_length_rabi(qDrive, yaml_cfg, gain=gain, opt_len=pi_len, opt_freq=pi_freq, test_pi_half=test_pi_half, save_data=save_files, saved_files=saved_files_lengthrabi, good_data=good_data)
        if not test_pi_half:
            yaml_cfg.device.qubit.pulses.pi_EgGf_Q.sigma[qDrive] = pi_len
        else:
            yaml_cfg.device.qubit.pulses.pi_EgGf_Q.half_sigma[qDrive] = pi_len
        if len(new_opt_lens) > i_gain:
            new_opt_lens[i_gain] = pi_len
            print(f'for gain {gain} updated swap pi len from length rabi', pi_len)
        else:
            new_opt_lens.append(pi_len)
            print(f'for gain {gain} added swap pi len from length rabi', pi_len)
    override = False


    # ----- Error amplification ----- #
    if do_error_amp and not override:
        print('using opt freq', pi_freq)
        pi_len = run_swap_error_amp(qDrive, yaml_cfg, gain=gain, opt_len=pi_len, opt_freq=pi_freq, test_pi_half=test_pi_half, save_data=save_files, saved_files=saved_files_error_amp, good_data=good_data)
        if not test_pi_half:
            yaml_cfg.device.qubit.pulses.pi_EgGf_Q.sigma[qDrive] = pi_len
        else:
            yaml_cfg.device.qubit.pulses.pi_EgGf_Q.half_sigma[qDrive] = pi_len
        if len(new_opt_lens2) > i_gain:
            new_opt_lens2[i_gain] = pi_len
            print(f'for gain {gain} updated swap pi len from error amplification', pi_len)
        else:
            new_opt_lens2.append(pi_len)
            print(f'for gain {gain} added swap pi len from error amplification', pi_len)
    override = False

    # ----- Pi Train ----- #
    if do_pi_train and not override:
        print('using opt freq', pi_freq)
        pi_len = run_swap_pi_train(qDrive, yaml_cfg, gain=gain, opt_len=pi_len, opt_freq=pi_freq, test_pi_half=test_pi_half, save_data=save_files, saved_files=saved_files_pitrain, good_data=good_data)
        if not test_pi_half:
            yaml_cfg.device.qubit.pulses.pi_EgGf_Q.sigma[qDrive] = pi_len
        else:
            yaml_cfg.device.qubit.pulses.pi_EgGf_Q.half_sigma[qDrive] = pi_len
        if len(new_opt_lens3) > i_gain:
            new_opt_lens3[i_gain] = pi_len
            print(f'for gain {gain} updated swap pi len from pi train', pi_len)
        else:
            new_opt_lens3.append(pi_len)
            print(f'for gain {gain} added swap pi len from pi train', pi_len)
    override = False

In [ ]:
print(f'gain_sweep={np.array(gain_sweep).tolist()}')
print(f'new_opt_lens (fine length rabi)={new_opt_lens}')
print(f'new_opt_lens2 (error amp)={new_opt_lens2}')
print(f'new_opt_lens3 (pi train)={new_opt_lens3}')
print(f'opt_freqs (initial guess)={opt_freqs}')
print(f'new_opt_freqs={new_opt_freqs}')
print(f'saved_files_lengthrabi={saved_files_lengthrabi}')
print(f'saved_files_piminuspi={saved_files_piminuspi}')
print(f'saved_files_error_amp={saved_files_error_amp}')

plt.figure(figsize=(8, 6))
plt.plot(gain_sweep, opt_lens, '.-', label='Initial guess')
if do_length_rabi:
    plt.plot(gain_sweep, new_opt_lens, '.-', label='Length Rabi Fine')
if do_error_amp:
    plt.plot(gain_sweep, new_opt_lens2, '.-', label='Error Amplfication')
if do_pi_train:
    plt.plot(gain_sweep, new_opt_lens3, '.-', label='Pi Train')
plt.xlabel('Gain')
plt.ylabel('Length (us)')
plt.legend()
plt.title(f'Q{qubits[0]}Q{qubits[1]}')
plt.show()

plt.figure(figsize=(8, 6))
plt.plot(gain_sweep, opt_freqs, '.-', label='Guess freq')
if do_piminuspi:
    plt.plot(gain_sweep, new_opt_freqs, '.-', label='$\pi/-\pi$ freq')
plt.xlabel('Gain')
plt.ylabel('Frequency (MHz)')
plt.legend()
plt.title(f'Q{qubits[0]}Q{qubits[1]}')
plt.show()


#### Q2/Q1

In [202]:
qubits = [2, 1] # order doesn't matter if you specify qDrive correctly
init_state = '|0>|1>'
wrong_init_state = '|1>|1>'
setup_ZZ = None
# measure_qubits = [qubits[0], qubits[1], 3]
measure_qubits = qubits
qDrive = qubits[0] # normal new protocol

In [ ]:
do_piminuspi = do_length_rabi = do_error_amp = do_pi_train = False

do_piminuspi = True
do_length_rabi = True
# do_error_amp = True
do_pi_train = True

# put an override=True flag to force skipping certain qubits

save_files = True

good_data = False

full_mux_expt = False
resonator_reset = [0, 2, 3] if full_mux_expt else None
use_robust_pulses = True

test_pi_half = True

In [204]:
import yaml
with open(config_path, 'r') as cfg_file:
    yaml_cfg = yaml.safe_load(cfg_file)
yaml_cfg = AttrDict(yaml_cfg)

In [85]:
# # Input the freqs fitted from the amp rabi chevron and the gain sweep used in that chevron

# fit_gain_sweep = [7000, 8666, 10332, 11998, 13664, 15330, 16996, 18662, 20328, 21994, 23660, 25326, 26992, 28658, 30324]
# fit_freqs = [5891.426975764686, 5890.140864964922, 5888.935582757715, 5887.811129143062, 5886.767504120967, 5885.804707691427, 5884.9227398544435, 5884.121600610016, 5883.401289958145, 5882.76180789883, 5882.203154432071, 5881.725329557868, 5881.328333276221, 5881.012165587131, 5880.776826490596]

# # Fit those to a quadratic function and use this to extrapolate frequencies for the gain sweep of the actual experiment
# fitparams = [fit_gain_sweep[0], fit_freqs[0], (fit_freqs[-1] - fit_freqs[0])/(fit_gain_sweep[-1] - fit_gain_sweep[0])**2]
# fitparams = None
# p, pCov = meas.fitting.fitquadratic(fit_gain_sweep, fit_freqs, fitparams=fitparams)

In [ ]:
npts = 35

gain_sweep = np.linspace(7000, 32000, npts)

# Guess the approximate lengths corresponding to the gain sweep
opt_lens = np.linspace(0.8, 0.200, npts)
if test_pi_half:
    opt_lens /= 2

# For pi/-pi
span_fs = np.linspace(2.0, 3.0, npts)

# Calculate the approximate frequencies corresponding to the gain sweep using the fit 
# opt_freqs = meas.fitting.quadraticfunc(gain_sweep, *p)
opt_freqs = np.linspace(5887.0, 5880.7, npts)

print('gain_sweep\n', gain_sweep)
print('approx len for each gain (us)\n', opt_lens)
print('approx freq for each gain\n', opt_freqs.tolist())

print('qubits', qubits)

new_opt_lens = [] # length rabi
new_opt_lens2 = [] # error amp
new_opt_lens3 = [] # pi pulse train

new_opt_freqs = []

saved_files_lengthrabi = []
saved_files_piminuspi = []
saved_files_error_amp = []
saved_files_pitrain = []

# opt_fit_index = 1 # index of [QA I, QA Q, QB I, QB Q] to use as saved optimal
signs = [1, 1, 1, 1] # whether the period occurs at the max (1) or min (-1)

opt_fit_index = 0 # index of [QA amps, QB amps] to use as saved optimal

In [ ]:
save_files = False

new_opt_lens = [] # length rabi
new_opt_lens2 = [] # error amp
new_opt_lens3 = [] # pi pulse train
new_opt_freqs = []

saved_files_lengthrabi = []
saved_files_piminuspi = []
saved_files_error_amp = []
saved_files_pitrain = []

# opt_fit_index = 1 # index of [QA I, QA Q, QB I, QB Q] to use as saved optimal
signs = [1, 1, 1, 1] # whether the period occurs at the max (1) or min (-1)

opt_fit_index = 0 # index of [QA amps, QB amps] to use as saved optimal

gain_sweep = [yaml_cfg.device.qubit.pulses.pi_EgGf_Q.gain[qubits[0]]]
opt_lens = [yaml_cfg.device.qubit.pulses.pi_EgGf_Q.sigma[qubits[0]]]
opt_freqs = [yaml_cfg.device.qubit.f_EgGf_Q[qubits[0]]]

if test_pi_half:
    gain_sweep = [yaml_cfg.device.qubit.pulses.pi_EgGf_Q.half_gain[qubits[0]]]
    opt_lens = [yaml_cfg.device.qubit.pulses.pi_EgGf_Q.half_sigma[qubits[0]]]
    opt_freqs = [yaml_cfg.device.qubit.f_EgGf_Q_half[qubits[0]]]

print('gain_sweep\n', gain_sweep)
print('approx len for each gain (us)\n', opt_lens)
print('approx freq for each gain\n', opt_freqs)


In [ ]:
# signs = [1, -1, -1, 1] # whether the period occurs at the max (1) or min (-1)
# opt_fit_index = 1

if not save_files: print('WARNING: NOT SAVING FILES!')
for i_gain, gain in enumerate(gain_sweep):
    # if i_gain != 7 and i_gain != 12: continue 
    # if gain < 15822: continue
    # if i_gain not in [3]: continue

    override = False

    print(f'----------- STARTING GAIN {gain} -----------')
    pi_len = opt_lens[i_gain]
    pi_freq = opt_freqs[i_gain]

    # ----- Pi minus pi----- #
    # if gain > 15822 and gain < 15830:
    #     override = True
    #     pi_freq = yaml_cfg.device.qubit.f_EgGf_Q[qDrive]
    if do_piminuspi and not override:
        pi_freq = run_swap_piminuspi(qDrive, yaml_cfg, gain=gain, opt_len=pi_len, opt_freq=pi_freq, test_pi_half=test_pi_half, span_f=span_fs[i_gain], save_data=save_files, saved_files=saved_files_piminuspi, good_data=good_data)
        if not test_pi_half:
            yaml_cfg.device.qubit.f_EgGf_Q[qDrive] = pi_freq
        else:
            yaml_cfg.device.qubit.f_EgGf_Q_half[qDrive] = pi_freq
        if len(new_opt_freqs) > i_gain:
            new_opt_freqs[i_gain] = pi_freq
            print(f'for gain {gain} updated swap pi freq from pi minus pi', pi_freq)
        else:
            new_opt_freqs.append(pi_freq)
            print(f'for gain {gain} added swap pi freq from pi minus pi', pi_freq)
    override = False

    # ----- Length rabi ----- #
    # if gain > 15822 and gain < 15830:
    #     override = True
    #     pi_len = yaml_cfg.device.qubit.pulses.pi_EgGf_Q.sigma[qDrive]
    if do_length_rabi and not override:
        pi_len = run_swap_length_rabi(qDrive, yaml_cfg, gain=gain, opt_len=pi_len, opt_freq=pi_freq, test_pi_half=test_pi_half, save_data=save_files, saved_files=saved_files_lengthrabi, good_data=good_data)
        if not test_pi_half:
            yaml_cfg.device.qubit.pulses.pi_EgGf_Q.sigma[qDrive] = pi_len
        else:
            yaml_cfg.device.qubit.pulses.pi_EgGf_Q.half_sigma[qDrive] = pi_len
        if len(new_opt_lens) > i_gain:
            new_opt_lens[i_gain] = pi_len
            print(f'for gain {gain} updated swap pi len from length rabi', pi_len)
        else:
            new_opt_lens.append(pi_len)
            print(f'for gain {gain} added swap pi len from length rabi', pi_len)
    override = False


    # ----- Error amplification ----- #
    if do_error_amp and not override:
        print('using opt freq', pi_freq)
        pi_len = run_swap_error_amp(qDrive, yaml_cfg, gain=gain, opt_len=pi_len, opt_freq=pi_freq, test_pi_half=test_pi_half, save_data=save_files, saved_files=saved_files_error_amp, good_data=good_data)
        if not test_pi_half:
            yaml_cfg.device.qubit.pulses.pi_EgGf_Q.sigma[qDrive] = pi_len
        else:
            yaml_cfg.device.qubit.pulses.pi_EgGf_Q.half_sigma[qDrive] = pi_len
        if len(new_opt_lens2) > i_gain:
            new_opt_lens2[i_gain] = pi_len
            print(f'for gain {gain} updated swap pi len from error amplification', pi_len)
        else:
            new_opt_lens2.append(pi_len)
            print(f'for gain {gain} added swap pi len from error amplification', pi_len)
    override = False

    # ----- Pi Train ----- #
    if do_pi_train and not override:
        print('using opt freq', pi_freq)
        pi_len = run_swap_pi_train(qDrive, yaml_cfg, gain=gain, opt_len=pi_len, opt_freq=pi_freq, test_pi_half=test_pi_half, save_data=save_files, saved_files=saved_files_pitrain, good_data=good_data)
        if not test_pi_half:
            yaml_cfg.device.qubit.pulses.pi_EgGf_Q.sigma[qDrive] = pi_len
        else:
            yaml_cfg.device.qubit.pulses.pi_EgGf_Q.half_sigma[qDrive] = pi_len
        if len(new_opt_lens3) > i_gain:
            new_opt_lens3[i_gain] = pi_len
            print(f'for gain {gain} updated swap pi len from pi train', pi_len)
        else:
            new_opt_lens3.append(pi_len)
            print(f'for gain {gain} added swap pi len from pi train', pi_len)
    override = False

In [ ]:
print(f'gain_sweep={np.array(gain_sweep).tolist()}')
print(f'new_opt_lens (fine length rabi)={new_opt_lens}')
print(f'new_opt_lens2 (error amp)={new_opt_lens2}')
print(f'new_opt_lens3 (pi train)={new_opt_lens3}')
print(f'opt_freqs (initial guess)={opt_freqs}')
print(f'new_opt_freqs={new_opt_freqs}')
print(f'saved_files_lengthrabi={saved_files_lengthrabi}')
print(f'saved_files_piminuspi={saved_files_piminuspi}')
print(f'saved_files_error_amp={saved_files_error_amp}')
print(f'saved_files_pitrain={saved_files_pitrain}')

plt.figure(figsize=(8, 6))
plt.plot(gain_sweep, opt_lens, '.-', label='Initial guess')
if do_length_rabi:
    plt.plot(gain_sweep, new_opt_lens, '.-', label='Length Rabi Fine')
if do_error_amp:
    plt.plot(gain_sweep, new_opt_lens2, '.-', label='Error Amplfication')
if do_pi_train:
    plt.plot(gain_sweep, new_opt_lens3, '.-', label='Pi Train')
plt.xlabel('Gain')
plt.ylabel('Length (us)')
plt.legend()
plt.title(f'Q{qubits[0]}Q{qubits[1]}')
plt.show()

plt.figure(figsize=(8, 6))
plt.plot(gain_sweep, opt_freqs, '.-', label='Guess freq')
if do_piminuspi:
    plt.plot(gain_sweep, new_opt_freqs, '.-', label='$\pi/-\pi$ freq')
plt.xlabel('Gain')
plt.ylabel('Frequency (MHz)')
plt.legend()
plt.title(f'Q{qubits[0]}Q{qubits[1]}')
plt.show()


### Replot data above

In [106]:
saved_files_lengthrabi = saved_files_lengthrabi[20:26]
saved_files_piminuspi = saved_files_piminuspi[20:26]
saved_files_error_amp = saved_files_error_amp[20:26]

In [42]:
# Reconstruct saved files list
saved_files_lengthrabi = []
saved_files_piminuspi = []
saved_files_error_amp = []

qubits = [3, 1]

start_i = 10
stop_i = 14
base_path = 's:\QRAM\qram_4QR2\data\data_241025'
filename = f'length_rabi_EgGf_qubit{qubits[0]}{qubits[1]}.h5'
for i in range(start_i, stop_i+1):
    if i < 10:
        saved_files_lengthrabi.append(f'{base_path}\\0000{i}_{filename}')
    elif i < 100:
        saved_files_lengthrabi.append(f'{base_path}\\000{i}_{filename}')
    else:
        saved_files_lengthrabi.append(f'{base_path}\\00{i}_{filename}')

start_i = 10
stop_i = 14
filename = f'PiMinusPiEgGfExptQ{qubits[0]}{qubits[1]}.h5'
for i in range(start_i, stop_i+1):
    if i < 10:
        saved_files_piminuspi.append(f'{base_path}\\0000{i}_{filename}')
    elif i < 100:
        saved_files_piminuspi.append(f'{base_path}\\000{i}_{filename}')
    else:
        saved_files_piminuspi.append(f'{base_path}\\00{i}_{filename}')

start_i = 10
stop_i = 14
filename = f'NPulseExptEgGfQ{qubits[0]}{qubits[1]}.h5'
for i in range(start_i, stop_i+1):
    if i < 10:
        saved_files_error_amp.append(f'{base_path}\\0000{i}_{filename}')
    elif i < 100:
        saved_files_error_amp.append(f'{base_path}\\000{i}_{filename}')
    else:
        saved_files_error_amp.append(f'{base_path}\\00{i}_{filename}')

In [ ]:
opt_fit_index = 0

new_opt_lens = []
new_opt_lens2 = []
new_opt_freqs = []

# Going through length rabi files again to get opt lens
for i_file, saved_file in enumerate(saved_files_lengthrabi):
    file_name = saved_file.split('\\')[-1]
    print(file_name)

    lengthrabiEgGf = meas.LengthRabiEgGfExperiment(config_file=config_path)
    # # rabiEgGfChevron.cfg.expt = dict(sigma_test=0.400, singleshot=False,)
    temp_data, attrs = prev_data(expt_path, file_name)
    lengthrabiEgGf.cfg = AttrDict(attrs['config'])
    lengthrabiEgGf.data = temp_data

    data = lengthrabiEgGf.analyze(fit=True)
    lengthrabiEgGf.display(fit=True)
    p, pcov = meas.fitting.get_best_fit(data, prefixes=['fitA', 'fitB'], check_measures=['amps'])
    if p[2] > 180:
        p[2] = p[2] - 360
    elif p[2] < -180:
        p[2] = p[2] + 360
    if p[2] < 0:
        pi_len = (1 / 2 - p[2] / 180) / 2 / p[1]
    else:
        pi_len = (3 / 2 - p[2] / 180) / 2 / p[1]

    new_opt_lens.append(pi_len)
    print('pi len', new_opt_lens[-1])


# Going through piminuspi files again to get opt freqs
for i_file, saved_file in enumerate(saved_files_piminuspi):
    file_name = saved_file.split('\\')[-1]
    print(file_name)

    piminuspi = meas.PiMinusPiEgGfExperiment(config_file=config_path)
    # # rabiEgGfChevron.cfg.expt = dict(sigma_test=0.400, singleshot=False,)
    temp_data, attrs = prev_data(expt_path, file_name)
    piminuspi.cfg = AttrDict(attrs['config'])
    piminuspi.data = temp_data

    data = piminuspi.analyze(fit=True)
    piminuspi.display(fit=True)
    best_freq = data['best_freq']
    new_opt_freqs.append(best_freq)
    print('best freq', new_opt_freqs[-1])


# Going through error amp files again to get opt lens 2
for i_file, saved_file in enumerate(saved_files_error_amp):
    file_name = saved_file.split('\\')[-1]
    print(file_name)

    npulsecalib = meas.NPulseEgGfExperiment(config_file=config_path)
    # # rabiEgGfChevron.cfg.expt = dict(sigma_test=0.400, singleshot=False,)
    temp_data, attrs = prev_data(expt_path, file_name)
    npulsecalib.cfg = AttrDict(attrs['config'])
    npulsecalib.data = temp_data

    data = npulsecalib.analyze(fit=True)
    npulsecalib.display(fit=True)

    npulsecalib_fit, npulsecalib_fit_err = meas.fitting.get_best_fit(npulsecalib.data, prefixes=['fitA', 'fitB'], check_measures=['avgi'])

    if np.all(npulsecalib_fit == data['fitA_avgi']): use_q =  npulsecalib.cfg.expt.measure_qubits[0]
    else: use_q = npulsecalib.cfg.expt.measure_qubits[1]
    if use_q == qDrive: sign = 1
    else: sign = -1
    angle_err = npulsecalib_fit[1]
    amp_ratio = (180 - sign*angle_err)/180
    new_opt_len = npulsecalib.cfg.device.qubit.pulses.pi_EgGf_Q.sigma[qDrive] / amp_ratio

    print('pi len precise', new_opt_len, f'adjustment ratio {amp_ratio} from Q{use_q}')

    new_opt_lens2.append(new_opt_len)

In [ ]:
print(new_opt_lens)
print(saved_files)

In [ ]:
new_opt_lens[1] = 1.1987019004033739
new_opt_lens[4] = 0.4716335769416996

Sweep ramp times

In [ ]:
gain = 25066
freq = 5885.080333333333
default_len = 0.28170921764829654
default_sigma_ramp_cycles = 3
sigma_n = 4

# len_approx_gains = 1.5*np.linspace(0.470, 0.210, 25)
# # len_approx_gains = 1.5*np.linspace(0.500, 0.300, 21)
# print('max len for each gain (us)\n', len_approx_gains)


# freq_approx_gains = np.linspace(5889.675, 5882.583, 25) # approx freq for each gain
# # freq_approx_gains = np.linspace(5151.5, 5147, 21) # approx freq for each gain

sigma_ramp_cycle_sweep = np.linspace(2, 20, 10)
sigma_ramp_us_sweep = np.array([soc.cycles2us(c, gen_ch=yaml_cfg.hw.soc.dacs.swap_Q.ch[sweep2Q_q]) for c in sigma_ramp_cycle_sweep])
print('sigma_ramp_cycle_sweep', sigma_ramp_cycle_sweep)
print('sigma_ramp in ns', 1e3*sigma_ramp_us_sweep)


default_area = default_len * gain + np.sqrt(2*np.pi)*soc.cycles2us(default_sigma_ramp_cycles, gen_ch=yaml_cfg.hw.soc.dacs.swap_Q.ch[sweep2Q_q])*sp.special.erf(sigma_n/2 / np.sqrt(2))
expected_flat_len = (default_area - np.sqrt(2*np.pi)*sigma_ramp_us_sweep*sp.special.erf(sigma_n/2 / np.sqrt(2))) / gain

print('expected_flat_lens', expected_flat_len)


# print('approx freq for each gain\n', freq_approx_gains)

# opt_lens = []
# opt_freqs = []
# saved_files = []
# save_files = True

# opt_fit_index = 2 # index of [QA I, QA Q, QB I, QB Q] to use as saved optimal

In [ ]:
saved_files = []
save_files = True
opt_lens_ramps = []
opt_fit_index = 2

for i_ramp, ramp_cycles in enumerate(sigma_ramp_cycle_sweep):
    lengthrabiEgGf = meas.LengthRabiEgGfExperiment(
        soccfg=soc,
        path=expt_path,
        prefix=f"length_rabi_EgGf_qubit{qubits[0]}{qubits[1]}",
        config_file=config_path,
    )
    
    stop_len = default_len*1.5

    start_len = 0
    span_len = stop_len - start_len
    npts_len = 151

    sweep2Q_q = qubits[0]
    lengthrabiEgGf.cfg.device.qubit.pulses.pi_EgGf_Q.gain[sweep2Q_q] = int(gain)
    lengthrabiEgGf.cfg.device.qubit.f_EgGf_Q[sweep2Q_q] = freq

    lengthrabiEgGf.cfg.expt = dict(
        start=start_len, # pulse length (sigma) [us]
        step=span_len/npts_len, # [us]
        expts=npts_len,
        reps=2000,
        post_process='threshold',
        # post_process=None,
        singleshot_reps=20000,
        measure_f=False,
        init_state=init_state,
        qDrive=qDrive,
        qubits=qubits,
        measure_qubits=measure_qubits,
        sigma_ramp_cycles=ramp_cycles,
    )
    
    # lengthrabiEgGf.cfg.device.readout.readout_length = [10,5] # [us]
    print(lengthrabiEgGf.cfg)
    # lengthrabiEgGf.im[lengthrabiEgGf.cfg.aliases.soc].reset()
    try:
        lengthrabiEgGf.acquire(progress=True, debug=True)
    except Exception:
        print("Pyro traceback:")
        print("".join(Pyro4.util.getPyroTraceback()))

    if save_files: saved_files.append(lengthrabiEgGf.save_data())
    print('Saved', saved_files[-1])
    data = lengthrabiEgGf.analyze(data=None, fit=True)
    pi_lens = lengthrabiEgGf.display(data=data, fit=True)

    new_opt_lens.append(pi_lens[opt_fit_index])
    print('pi len', new_opt_lens[-1])

## Randomized benchmarking

In [ ]:
rbprog = meas.SimultaneousRBEgGfExperiment(
    soccfg=soc,
    path=expt_path,
    prefix=f"rb_EgGf_qubit{qubits[0]}{qubits[1]}",
    config_file=config_path,
)

gate_char = None
# gate_char = 'X'


max_gates = 225
expts = 30
# expts = 5

if gate_char is None: step = max_gates // expts
else: step = max_gates // (2*expts)
print('step', step)

rbprog.cfg.expt = dict(
    # start=20,
    # step=0,
    # expts=1,
    # variations=1000,
    # variations=10, # number different sequences per depth
    start=1, # rb depth start
    step=step, # step rb depth
    expts=expts+1, # number steps
    # variations=3,
    variations=20,
    reps=2000, # number averages per unique sequence need at least 10,000 for a good amount of error per point!!
    loops=5,
    # reps=100, # number averages per unique sequence need at least 10,000 for a good amount of error per point!!
    gate_char=gate_char, # single qubit clifford gate (str) to characterize. if not None, runs interleaved RB instead of regular RB
    qubits=qubits,
    qDrive=qDrive,
    singleshot_reps=20000, # reps per state for singleshot calibration
    # singleshot_reps=100, # reps per state for singleshot calibration
    post_process='threshold', # 'threshold' (uses single shot binning), 'scale' (scale by ge_avgs), or None

    measure_f=[qDrive],
    add_phase=True,

    # cool_qubits=[0, 1],

)
# rbprog.cfg.device.qubit.pulses.pi_ge.gain[qubit_i] = int(0.8*rbprog.cfg.device.qubit.pulses.pi_ge.gain[qubit_i])
# rbprog.cfg.device.readout.relax_delay = 1
# rbprog.cfg.device.readout.readout_length = 5

print(rbprog.cfg)

try:
    rbprog.go(analyze=False, display=False, progress=True, save=False)
except Exception:
    print("Pyro traceback:")
    print("".join(Pyro4.util.getPyroTraceback()))

# rbprog.go(analyze=False, display=False, progress=True, save=False)

In [ ]:
rbprog.save_data()

In [ ]:
# # TESTING
rbprog = meas.SimultaneousRBEgGfExperiment(config_file=config_path)
temp_data, attrs = prev_data('s:\Connie\experiments\qramLL_4QR2\data\data_240617\\00011_rb_EgGf_qubit31.h5')
rbprog.data = temp_data
rbprog.cfg = AttrDict(attrs['config'])
rbprog.calib_order = attrs['calib_order']

rbprog.analyze(fit=True)
# rbprog.display(fit=True, show_all_vars=False)

In [ ]:
rbprog.save_data()

In [ ]:
# p_irb = 0.9590834452029213 
# p_irb_err = 0.0002387037509980582
# p_rb = 0.9833366468001751 
# p_rb_err = 0.00011879294449350527

p_irb = 0.9378643954169977
p_irb_err = 0.02184963356373672
p_rb = 0.9736931040310971
p_rb_err = 0.01541381511169818 
print(100*meas.fitting.rb_error(p_rb, 2),'% average error')
gate_fid, gate_fid_err = meas.fitting.rb_gate_fidelity(p_rb=p_rb, p_irb=p_irb, d=2, p_rb_err=p_rb_err, p_irb_err=p_irb_err)
print(100*gate_fid, '+/-', 100*gate_fid_err, '% gate fidelity')

In [ ]:
rbprog = meas.SimultaneousRBEgGfExperiment(
    soccfg=soc,
    path=expt_path,
    prefix=f"rb_EgGf_qubit{qubits[0]}{qubits[1]}",
    config_file=config_path,
)

# gate_char = None
gate_char = 'X'


max_gates = 225
expts = 30
# expts = 5

if gate_char is None: step = max_gates // expts
else: step = max_gates // (2*expts)
print('step', step)

rbprog.cfg.expt = dict(
    # start=20,
    # step=0,
    # expts=1,
    # variations=1000,
    # variations=10, # number different sequences per depth
    start=1, # rb depth start
    step=step, # step rb depth
    expts=expts+1, # number steps
    variations=20,
    reps=2000, # number averages per unique sequence need at least 10,000 for a good amount of error per point!!
    loops=5,
    # reps=5, # number averages per unique sequence need at least 10,000 for a good amount of error per point!!
    gate_char=gate_char, # single qubit clifford gate (str) to characterize. if not None, runs interleaved RB instead of regular RB
    qubits=qubits,
    qDrive=qDrive,
    singleshot_reps=20000, # reps per state for singleshot calibration
    # singleshot_reps=100, # reps per state for singleshot calibration
    post_process='threshold', # 'threshold' (uses single shot binning), 'scale' (scale by ge_avgs), or None

    measure_f=[qDrive],
    add_phase=True,

    # cool_qubits=[0, 1],

)
# rbprog.cfg.device.qubit.pulses.pi_ge.gain[qubit_i] = int(0.8*rbprog.cfg.device.qubit.pulses.pi_ge.gain[qubit_i])
# rbprog.cfg.device.readout.relax_delay = 1
# rbprog.cfg.device.readout.readout_length = 5

print(rbprog.cfg)

try:
    rbprog.go(analyze=False, display=False, progress=True, save=False)
except Exception:
    print("Pyro traceback:")
    print("".join(Pyro4.util.getPyroTraceback()))

# rbprog.go(analyze=False, display=False, progress=True, save=False)

In [ ]:
rbprog.save_data()

In [ ]:
# # TESTING
# rbprog = meas.SimultaneousRBEgGfExperiment(config_file=config_path)
# temp_data, attrs = prev_data('s:\Connie\experiments\qramLL_4QR2\data\data_240617\\00010_rb_EgGf_qubit21.h5')
# rbprog.data = temp_data
# rbprog.cfg = AttrDict(attrs['config'])
# rbprog.calib_order = attrs['calib_order']

rbprog.analyze(fit=True)
rbprog.display(fit=True, show_all_vars=False)

Q2/Q1


In [ ]:
qubits = [2, 1] # order doesn't matter if you specify qDrive correctly
init_state = '|0>|1>'
setup_ZZ = None
# measure_qubits = [qubits[0], qubits[1], 3]
measure_qubits = qubits
qDrive = qubits[0] # normal new protocol

In [ ]:
rbprog = meas.SimultaneousRBEgGfExperiment(
    soccfg=soc,
    path=expt_path,
    prefix=f"rb_EgGf_qubit{qubits[0]}{qubits[1]}",
    config_file=config_path,
)

gate_char = None
# gate_char = 'X'


max_gates = 225
expts = 30
# expts = 5

if gate_char is None: step = max_gates // expts
else: step = max_gates // (2*expts)
print('step', step)

rbprog.cfg.expt = dict(
    # start=20,
    # step=0,
    # expts=1,
    # variations=1000,
    # variations=10, # number different sequences per depth
    start=1, # rb depth start
    step=step, # step rb depth
    expts=expts+1, # number steps
    variations=20,
    reps=2000, # number averages per unique sequence need at least 10,000 for a good amount of error per point!!
    loops=5,
    # reps=5, # number averages per unique sequence need at least 10,000 for a good amount of error per point!!
    gate_char=gate_char, # single qubit clifford gate (str) to characterize. if not None, runs interleaved RB instead of regular RB
    qubits=qubits,
    qDrive=qDrive,
    singleshot_reps=20000, # reps per state for singleshot calibration
    # singleshot_reps=100, # reps per state for singleshot calibration
    post_process='threshold', # 'threshold' (uses single shot binning), 'scale' (scale by ge_avgs), or None

    measure_f=[qDrive],
    add_phase=True,

    # cool_qubits=[0, 1],

)
# rbprog.cfg.device.qubit.pulses.pi_ge.gain[qubit_i] = int(0.8*rbprog.cfg.device.qubit.pulses.pi_ge.gain[qubit_i])
# rbprog.cfg.device.readout.relax_delay = 1
# rbprog.cfg.device.readout.readout_length = 5

print(rbprog.cfg)

try:
    rbprog.go(analyze=False, display=False, progress=True, save=False)
except Exception:
    print("Pyro traceback:")
    print("".join(Pyro4.util.getPyroTraceback()))

# rbprog.go(analyze=False, display=False, progress=True, save=False)

In [ ]:
rbprog.save_data()

In [ ]:
# # TESTING
# rbprog = meas.SimultaneousRBEgGfExperiment(config_file=config_path)
# temp_data, attrs = prev_data('s:\Connie\experiments\qramLL_4QR2\data\data_240617\\00010_rb_EgGf_qubit21.h5')
# rbprog.data = temp_data
# rbprog.cfg = AttrDict(attrs['config'])
# rbprog.calib_order = attrs['calib_order']

rbprog.analyze(fit=True)
rbprog.display(fit=True, show_all_vars=False)

In [ ]:
rbprog = meas.SimultaneousRBEgGfExperiment(
    soccfg=soc,
    path=expt_path,
    prefix=f"rb_EgGf_qubit{qubits[0]}{qubits[1]}",
    config_file=config_path,
)

# gate_char = None
gate_char = 'X'


max_gates = 225
expts = 30
# expts = 5

if gate_char is None: step = max_gates // expts
else: step = max_gates // (2*expts)
print('step', step)

rbprog.cfg.expt = dict(
    # start=20,
    # step=0,
    # expts=1,
    # variations=1000,
    # variations=10, # number different sequences per depth
    start=1, # rb depth start
    step=step, # step rb depth
    expts=expts+1, # number steps
    variations=20,
    reps=2000, # number averages per unique sequence need at least 10,000 for a good amount of error per point!!
    loops=5,
    # reps=5, # number averages per unique sequence need at least 10,000 for a good amount of error per point!!
    gate_char=gate_char, # single qubit clifford gate (str) to characterize. if not None, runs interleaved RB instead of regular RB
    qubits=qubits,
    qDrive=qDrive,
    singleshot_reps=20000, # reps per state for singleshot calibration
    # singleshot_reps=100, # reps per state for singleshot calibration
    post_process='threshold', # 'threshold' (uses single shot binning), 'scale' (scale by ge_avgs), or None

    measure_f=[qDrive],
    add_phase=True,

    # cool_qubits=[0, 1],

)
# rbprog.cfg.device.qubit.pulses.pi_ge.gain[qubit_i] = int(0.8*rbprog.cfg.device.qubit.pulses.pi_ge.gain[qubit_i])
# rbprog.cfg.device.readout.relax_delay = 1
# rbprog.cfg.device.readout.readout_length = 5

print(rbprog.cfg)

try:
    rbprog.go(analyze=False, display=False, progress=True, save=False)
except Exception:
    print("Pyro traceback:")
    print("".join(Pyro4.util.getPyroTraceback()))

# rbprog.go(analyze=False, display=False, progress=True, save=False)

In [ ]:
rbprog.save_data()

In [ ]:
# # TESTING
# rbprog = meas.SimultaneousRBEgGfExperiment(config_file=config_path)
# temp_data, attrs = prev_data('s:\Connie\experiments\qramLL_4QR2\data\data_240617\\00010_rb_EgGf_qubit21.h5')
# rbprog.data = temp_data
# rbprog.cfg = AttrDict(attrs['config'])
# rbprog.calib_order = attrs['calib_order']

rbprog.analyze(fit=True)
rbprog.display(fit=True, show_all_vars=False)

## 3D sweep to optimize length, gain, freq

In [ ]:
2027 - np.arange(expts_gain)*(10/8000*step_gain)

In [ ]:
start_gain = 2000
step_gain = 2000
expts_gain = 5
gain_sweep = start_gain + step_gain * np.arange(expts_gain)
print(gain_sweep)

sweep2d_expt_progs = []

for ig, gain in enumerate(gain_sweep):
    print('Now setting gain to', gain)
    rabiEgGfFreqLenChevron = meas.EgGfFreqLenChevronExperiment(
        soccfg=soc,
        path=expt_path,
        prefix=f"rabi_EgGf_freqlen_chevron_qubit{qubits[0]}{qubits[1]}",
        config_file=config_path,
    )

    span_f = 6 # MHz
    npts_f = 20

    freq_center_pts = 2412 - np.arange(expts_gain)*(10/6000*step_gain)
    if ig==0: print('Freq sweep will be centered at', freq_center_pts)

    rabiEgGfFreqLenChevron.cfg.expt = dict(
        start_f=freq_center_pts[ig] - span_f/2,
        step_f=span_f/npts_f, # freq step, 
        expts_f=npts_f, # number of different freq experiments, 
        start_len=soc.cycles2us(10), # pulse length (sigma) [us]
        step_len=soc.cycles2us(12), # [us]
        expts_len=30,
        gain=gain,
        reps=1000,
        pulse_type='flat_top',
        post_process='scale',
        singleshot_reps=15000,
        qubits=qubits,
    )

    # rabiEgGfFreqLenChevron.cfg.device.readout.readout_length = [15, 15] # [us]
    # rabiEgGfFreqLenChevron.cfg.device.readout.relax_delay = [600, 50] # Wait time between experiments [us]

    # print(rabiEgGfFreqLenChevron.cfg)

    try:
        rabiEgGfFreqLenChevron.go(analyze=False, display=False, progress=True, save=False)
    except Exception:
        print("Pyro traceback:")
        print("".join(Pyro4.util.getPyroTraceback()))
    
    rabiEgGfFreqLenChevron.analyze(data=None)
    rabiEgGfFreqLenChevron.display(data=None)
    sweep2d_expt_progs.append(rabiEgGfFreqLenChevron)

# Crosstalk calibrations

In [ ]:
crosstalkEcho = meas.CrosstalkEchoExperiment(
    soccfg=soc,
    path=expt_path,
    config_file=config_path,
)

crosstalkEcho.cfg.expt = dict(
    start=soc.cycles2us(60), # wait time tau [us]
    step=soc.cycles2us(90), # [us] make sure nyquist freq = 0.5 * (1/step) > ramsey (signal) freq!
    # step=soc.cycles2us(15), # [us] make sure nyquist freq = 0.5 * (1/step) > ramsey (signal) freq!
    expts=200,
    ramsey_freq=0.0, # [MHz]
    cpmg=False, # cp or cpmg
    reps=1500,
    qTest=1,
    qDrives=[2, 3],
    gain_x=10000,
    gain_y=0,
)
print(crosstalkEcho.cfg.expt.step)

print(crosstalkEcho.cfg)

try:
    crosstalkEcho.go(analyze=False, display=False, progress=True, save=False)
except Exception:
    print("Pyro traceback:")
    print("".join(Pyro4.util.getPyroTraceback()))

In [ ]:
# # TESTING
# crosstalkEcho = meas.RamseyExperiment(config_file=config_path)
# crosstalkEcho.cfg.expt = dict(ramsey_freq=1, qubit=qubit_i)
# temp_data, attrs = prev_data(expt_path, '00000_ramsey_qubit1.h5')
# data = crosstalkEcho.analyze(data=temp_data, fit=True)
# crosstalkEcho.display(data=data, fit=True)

data = crosstalkEcho.data
fitparams=[None, 5/data['xpts'][-1], None, None, None]
crosstalkEcho.analyze(fit=False, fit_num_sin=1, fitparams=fitparams)
crosstalkEcho.display(fit=False, fit_num_sin=1)

qTest = crosstalkEcho.cfg.expt.qTest

data = crosstalkEcho.data
xpts = data['xpts']
fourier = np.fft.fftshift(np.abs(np.fft.fft(data['avgi'])))
freqs = np.fft.fftshift(np.fft.fftfreq(len(fourier), d=(xpts[1]-xpts[0])))
plt.plot(freqs, fourier)
plt.xlabel('Frequency [MHz]')
plt.xlim(-6, 6)
# plt.ylim(0, 20000)
plt.title(f'Fourier Transform of Ramsey Echo on Q{qTest} (Ramsey Freq: {crosstalkEcho.cfg.expt.ramsey_freq})')
plt.show()

In [ ]:
# # TESTING
# t2r = meas.RamseyExperiment(config_file=config_path)
# t2r.cfg.expt = dict(ramsey_freq=2, qubit=qubit_i)
# temp_data, attrs = prev_data(expt_path, '00000_ramsey_qubit1.h5')
# data = t2r.analyze(data=temp_data, fit=True)
# t2r.display(data=data, fit=True)

t2r.save_data()

## Chevron for gain sweep vs time sweep

In [ ]:
crosstalkEchoTimeChevron = meas.CrosstalkEchoTimeChevronExperiment(
    soccfg=soc,
    path=expt_path,
    config_file=config_path,
)

span_i = 40000
npts_i = 150


crosstalkEchoTimeChevron.cfg.expt = dict(
    start_i=-span_i//2,
    step_i=int(span_i/npts_i), # [dac level]
    expts_i=npts_i+1,
    # start_t=soc.cycles2us(60), # wait time tau [us]
    # step_t=soc.cycles2us(30), # [us] make sure nyquist freq = 0.5 * (1/step) > ramsey (signal) freq!
    # expts_t=250,
    start_t=3.5,
    step_t=0,
    expts_t=1,
    sweep_axis='x',
    ramsey_freq=0.0, # [MHz]
    cpmg=False, # cp or cpmg
    reps=4000,
    qTest=1,
    qDrives=[2, 3],
    post_process='scale',
    singleshot_reps=20000,
    thresholds=[0, 42.250179073420334, 0, 0],
    angles=[0, -27.653262746238827, 0, 0],
    ge_avgs=[array([0., 0., 0., 0.]), [29.582075536759927, 24.37702111255693, 38.68021955107352, 29.11635689655172], array([0., 0., 0., 0.]), array([0., 0., 0., 0.])],
    counts_calib=[[8046, 6063, 3431, 2460], [3095, 2322, 8364, 6219]]
)

print(crosstalkEchoTimeChevron.cfg)

try:
    crosstalkEchoTimeChevron.go(analyze=False, display=False, progress=True, save=False)
except Exception:
    print("Pyro traceback:")
    print("".join(Pyro4.util.getPyroTraceback()))

In [ ]:
# amprabi = meas.AmplitudeRabiExperiment(config_file=config_path,)
temp_data, attrs = prev_data('s:\Connie\experiments\qramLL_4QR2\data\data_240218\\00002_CrosstalkEchoTime.h5', filename=None)
crosstalkEchoTimeChevron.data = temp_data
crosstalkEchoTimeChevron.cfg = AttrDict(attrs['config'])
print(crosstalkEchoTimeChevron.cfg.expt)

crosstalkEchoTimeChevron.analyze(fit=True)
crosstalkEchoTimeChevron.display(fit=True)

data = crosstalkEchoTimeChevron.data
plt.figure()
t_ind = np.argmin(np.abs(data['wait_times']-3))
# e_val = np.min(data['amps'])
# g_val = np.max(data['amps'])
plt.plot(data['gain_i_sweep'], data['amps'][:,t_ind], 'o-')
plt.title(f'Crosstalk Calibration on Q{crosstalkEchoTimeChevron.qTest} with drive on {crosstalkEchoTimeChevron.qDrives}')
plt.xlabel(f'Gains ({crosstalkEchoTimeChevron.cfg.expt.sweep_axis} axis) (DAC units)')
plt.ylabel('Population')
plt.axvline(0)
plt.axvline(12500)
plt.axvline(17750)
plt.axvline(-12500)
plt.axvline(-17750)
plt.show()

In [ ]:
crosstalkEchoTimeChevron.save_data()

## Chevron for fixed time

In [ ]:
crosstalkEchoChevron = meas.CrosstalkEchoChevronExperiment(
    soccfg=soc,
    path=expt_path,
    config_file=config_path,
)

span_x = 40000
npts_x = 60

span_y = 40000
npts_y = 60

crosstalkEchoChevron.cfg.expt = dict(
    start_x=-span_x//2,
    step_x=int(span_x/npts_x), # [dac level]
    expts_x=npts_x,
    start_y=-span_y//2,
    step_y=int(span_y/npts_y), # [dac level]
    expts_y=npts_y,
    ramsey_freq=0.0, # [MHz]
    cpmg=False, # cp or cpmg
    reps=2000,
    qTest=1,
    qDrives=[2, 3],
    wait_us=4,
    post_process='scale',
)

print(crosstalkEchoChevron.cfg)

try:
    crosstalkEchoChevron.go(analyze=False, display=False, progress=True, save=False)
except Exception:
    print("Pyro traceback:")
    print("".join(Pyro4.util.getPyroTraceback()))

In [ ]:
data = crosstalkEchoChevron.data
for k, a in data.items():
    data[k]=np.array(a)
    if np.shape(data[k]) == (100,):
        data[k] = np.reshape(data[k], (10, 10))
crosstalkEchoChevron.analyze(fit=True)
crosstalkEchoChevron.display(fit=True)